<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#option-A:-On-your-own-machine" data-toc-modified-id="option-A:-On-your-own-machine-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>option A: On your own machine</a></span></li><li><span><a href="#option-B:-In-the-docker-container" data-toc-modified-id="option-B:-In-the-docker-container-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>option B: In the docker container</a></span></li><li><span><a href="#For-the-moment-we-follow-option-A." data-toc-modified-id="For-the-moment-we-follow-option-A.-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>For the moment we follow option A.</a></span></li></ul></div>

In [1]:
import collections
import os
import operator
from functools import reduce
from datetime import datetime, date
from pymongo import MongoClient
from bson.objectid import ObjectId

In [2]:
DUMP_DIR = os.path.expanduser('~/local/narcis/MongoDB-NARCIS-Dec17')

# Start Docker

In case you want to stop Docker

In [3]:
!docker stop NARCIS
!docker rm NARCIS

NARCIS
NARCIS


In [4]:
!docker run --name NARCIS -v {DUMP_DIR}:/data/db -p 27019:27017 -d mongo --logpath /data/db/log.log

7309b02d9a3cab3e6356cb6c294ee349dc14f07be5289e6d9655c59eebec31d9


# Connect to MongoDB

In [5]:
client = MongoClient('mongodb://localhost:27019/')

In [6]:
client.database_names()

['NARCIS', 'admin', 'config', 'local']

Navigate to the Dec2017 collection

In [7]:
DBN = client.NARCIS
DBN.collection_names()

['Dec2017']

In [8]:
DBND = DBN.Dec2017
DBND.count()

1622397

# Inventory of fields
We count how often each field occurs.
We make separate counts for empty and non-empty values.

The troughput of docs is not very fast. The cell below takes several minutes. 

In [42]:
nonEmptyFields = collections.Counter()
emptyFields = collections.Counter()

i = 0
j = 0
chunk = 10000

for doc in DBND.find():
    j += 1
    i += 1
    if j == chunk:
        j = 0
        print(f'\t{i:>7}')
    for (key, val) in doc.items():
        if val:
            nonEmptyFields[key] += 1
        else:
            emptyFields[key] += 1
print(f'\t{i:>7}')

	  10000
	  20000
	  30000
	  40000
	  50000
	  60000
	  70000
	  80000
	  90000
	 100000
	 110000
	 120000
	 130000
	 140000
	 150000
	 160000
	 170000
	 180000
	 190000
	 200000
	 210000
	 220000
	 230000
	 240000
	 250000
	 260000
	 270000
	 280000
	 290000
	 300000
	 310000
	 320000
	 330000
	 340000
	 350000
	 360000
	 370000
	 380000
	 390000
	 400000
	 410000
	 420000
	 430000
	 440000
	 450000
	 460000
	 470000
	 480000
	 490000
	 500000
	 510000
	 520000
	 530000
	 540000
	 550000
	 560000
	 570000
	 580000
	 590000
	 600000
	 610000
	 620000
	 630000
	 640000
	 650000
	 660000
	 670000
	 680000
	 690000
	 700000
	 710000
	 720000
	 730000
	 740000
	 750000
	 760000
	 770000
	 780000
	 790000
	 800000
	 810000
	 820000
	 830000
	 840000
	 850000
	 860000
	 870000
	 880000
	 890000
	 900000
	 910000
	 920000
	 930000
	 940000
	 950000
	 960000
	 970000
	 980000
	 990000
	1000000
	1010000
	1020000
	1030000
	1040000
	1050000
	1060000
	1070000
	1080000
	1090000
	1100000
	1110000
	

In [43]:
len(nonEmptyFields)

15

In [44]:
len(emptyFields)

0

In [46]:
for (field, amount) in sorted(nonEmptyFields.items(), key=lambda x: (-x[1], x[0])):
    print(f'{field:<20} {amount:>7}x')

DAI                  1622397x
GlobalIDs            1622397x
ID                   1622397x
Journal              1622397x
Keywords             1622397x
NumberofIDs          1622397x
_id                  1622397x
access               1622397x
date_harv            1622397x
date_header          1622397x
date_orig            1622397x
filenr               1622397x
nldidlnorm           1622397x
originURL            1622397x
setSpec              1622397x


# Nested fields

We want to explore a distribution of nested fields.

This takes 21 mins.
It is the Pythonic way: you draw in every document, walk through its structure, and collect all the keys.

That said, it is not clear how to do this faster with a mapReduce or other javascript function from within the mongodb server.

In [9]:
def getLeaves(key, val):
    if type(val) is list:
        return reduce(operator.add, (getLeaves(key, v) for v in val), ())
    elif type(val) is dict:
        return reduce(operator.add, (getLeaves(key + (ky,), v) for (ky, v) in val.items()), ())
    else:
        return (key,)

In [19]:
nestedFields = collections.Counter()
i = 0
j = 0
chunk = 10000
limit = -1

for doc in DBND.find():
    j += 1
    i += 1
    for k in getLeaves((), doc):
        nestedFields[k] += 1
    if j == chunk:
        j = 0
        print(f'\t{i:>7}')
    if limit > 0 and i >= limit:
        break
print(f'\t{i:>7}')

	  10000
	  20000
	  30000
	  40000
	  50000
	  60000
	  70000
	  80000
	  90000
	 100000
	 110000
	 120000
	 130000
	 140000
	 150000
	 160000
	 170000
	 180000
	 190000
	 200000
	 210000
	 220000
	 230000
	 240000
	 250000
	 260000
	 270000
	 280000
	 290000
	 300000
	 310000
	 320000
	 330000
	 340000
	 350000
	 360000
	 370000
	 380000
	 390000
	 400000
	 410000
	 420000
	 430000
	 440000
	 450000
	 460000
	 470000
	 480000
	 490000
	 500000
	 510000
	 520000
	 530000
	 540000
	 550000
	 560000
	 570000
	 580000
	 590000
	 600000
	 610000
	 620000
	 630000
	 640000
	 650000
	 660000
	 670000
	 680000
	 690000
	 700000
	 710000
	 720000
	 730000
	 740000
	 750000
	 760000
	 770000
	 780000
	 790000
	 800000
	 810000
	 820000
	 830000
	 840000
	 850000
	 860000
	 870000
	 880000
	 890000
	 900000
	 910000
	 920000
	 930000
	 940000
	 950000
	 960000
	 970000
	 980000
	 990000
	1000000
	1010000
	1020000
	1030000
	1040000
	1050000
	1060000
	1070000
	1080000
	1090000
	1100000
	1110000
	

In [20]:
len(nestedFields)

1331

In [23]:
for (k, v) in sorted(nestedFields.items(), key=lambda x: (-x[1], x[0])):
    print(f'{v:>7}x {'.'.join(k)}')

SyntaxError: invalid syntax (<ipython-input-23-4f8b9af58be5>, line 2)

# With variety

Now the mongodb way: we could use mapReduce as indicated on
[stack overflow](https://stackoverflow.com/questions/2298870/mongodb-get-names-of-all-keys-in-collection).
But that would require adaptations, because the accepted answer does not do  nested keys.
So we have to write a MongoDb function for mapReduce, which means writing something in Javascript.

It turns out that somebody has already done this, and made a nice library out of it:
[variety](https://github.com/variety/variety). 
This is a tool to explore the dominant scheme of a MongoDB and its outlier documents.

## option A: On your own machine

You must have mongodb and node installed.

Install *variety*:

```
npm install variety-cli -g
```

Yes, it is a javascript program, and it runs in node.

## option B: In the docker container

Maybe Emile can prepare a docker container with node and variety-cli installed.

## For the moment we follow option A.
It takes nearly 80 minutes. 
So much slower than the Pythonic way. Not really handy, except maybe for small subsets.

In [24]:
!variety --port=27019 NARCIS/Dec2017

MongoDB shell version v3.4.10
connecting to: mongodb://127.0.0.1:27019/NARCIS
MongoDB server version: 3.6.2
Variety: A MongoDB Schema Analyzer
Version 1.5.1, released 02 October 2017
Using collection of "Dec2017"
Using query of { }
Using limit of 1622397
Using maxDepth of 99
Using sort of { "_id" : -1 }
Using outputFormat of "ascii"
Using persistResults of false
Using resultsDatabase of "varietyResults"
Using resultsCollection of "Dec2017Keys"
Using resultsUser of null
Using resultsPass of null
Using logKeysContinuously of false
Using excludeSubkeys of [ ]
Using arrayEscape of "XX"
Using plugins of [ ]
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| key                                                                                       | types                           | occurrences | percents                 |
| --------------------------------------------------

| nldidlnorm.XX.Item.Component.Resource.mods.name3.role.roleTerm1._attrs                    | Array                           |       44489 |   2.74217716132364630610 |
| nldidlnorm.XX.Item.Component.Resource.mods.name3.role.roleTerm1.text                      | Array                           |       44489 |   2.74217716132364630610 |
| nldidlnorm.XX.Item.Component.Resource.mods.name1.affiliation1                             | Array                           |       43935 |   2.70803015538120428474 |
| nldidlnorm.XX.Item.Component.Resource.mods.name9.role1                                    | Object                          |       43415 |   2.67597881406338888866 |
| nldidlnorm.XX.Item.Component.Resource.mods.name9.role1.roleTerm                           | Object                          |       43415 |   2.67597881406338888866 |
| nldidlnorm.XX.Item.Component.Resource.mods.name9.role1.roleTerm._attrs                    | Array                           |       43415 |   2.675978814

| nldidlnorm.XX.Item.Component.Resource.mods.name9.affiliation8                             | Array                           |           5 |   0.00030818597420976493 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject0.topic1._attrs                         | Array                           |           5 |   0.00030818597420976493 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject0.topic1.text                           | Array                           |           5 |   0.00030818597420976493 |
| nldidlnorm.XX.Item.Component.Resource.mods.name1.role.roleTerm2                           | Object                          |           4 |   0.00024654877936781196 |
| nldidlnorm.XX.Item.Component.Resource.mods.name1.role.roleTerm2._attrs                    | Array                           |           4 |   0.00024654877936781196 |
| nldidlnorm.XX.Item.Component.Resource.mods.name1.role.roleTerm2.text                      | Array                           |           4 |   0.000246548

```
MongoDB shell version v3.4.10
connecting to: mongodb://127.0.0.1:27019/NARCIS
MongoDB server version: 3.6.2
WARNING: shell and server versions do not match
Variety: A MongoDB Schema Analyzer
Version 1.5.1, released 02 October 2017
Using collection of "Dec2017"
Using query of { }
Using limit of 1622397
Using maxDepth of 99
Using sort of { "_id" : -1 }
Using outputFormat of "ascii"
Using persistResults of false
Using resultsDatabase of "varietyResults"
Using resultsCollection of "Dec2017Keys"
Using resultsUser of null
Using resultsPass of null
Using logKeysContinuously of false
Using excludeSubkeys of [ ]
Using arrayEscape of "XX"
Using plugins of [ ]
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| key                                                                                       | types                           | occurrences | percents                 |
| ----------------------------------------------------------------------------------------- | ------------------------------- | ----------- | ------------------------ |
| DAI                                                                                       | Array                           |     1622397 | 100.00000000000000000000 |
| GlobalIDs                                                                                 | Array                           |     1622397 | 100.00000000000000000000 |
| ID                                                                                        | Array                           |     1622397 | 100.00000000000000000000 |
| Journal                                                                                   | Array                           |     1622397 | 100.00000000000000000000 |
| Keywords                                                                                  | Array                           |     1622397 | 100.00000000000000000000 |
| NumberofIDs                                                                               | Array                           |     1622397 | 100.00000000000000000000 |
| _id                                                                                       | ObjectId                        |     1622397 | 100.00000000000000000000 |
| access                                                                                    | Array                           |     1622397 | 100.00000000000000000000 |
| date_harv                                                                                 | Array                           |     1622397 | 100.00000000000000000000 |
| date_header                                                                               | Array                           |     1622397 | 100.00000000000000000000 |
| date_orig                                                                                 | Array                           |     1622397 | 100.00000000000000000000 |
| filenr                                                                                    | Array                           |     1622397 | 100.00000000000000000000 |
| nldidlnorm                                                                                | Array                           |     1622397 | 100.00000000000000000000 |
| nldidlnorm.XX.Component                                                                   | Object                          |     1622397 | 100.00000000000000000000 |
| nldidlnorm.XX.Component.Resource                                                          | Array                           |     1622397 | 100.00000000000000000000 |
| nldidlnorm.XX.Descriptor                                                                  | Object                          |     1622397 | 100.00000000000000000000 |
| nldidlnorm.XX.Descriptor.Statement                                                        | Object                          |     1622397 | 100.00000000000000000000 |
| nldidlnorm.XX.Descriptor.Statement.Identifier                                             | Array                           |     1622397 | 100.00000000000000000000 |
| nldidlnorm.XX.Descriptor.Statement._attrs                                                 | Array                           |     1622397 | 100.00000000000000000000 |
| nldidlnorm.XX.Descriptor1                                                                 | Object                          |     1622397 | 100.00000000000000000000 |
| nldidlnorm.XX.Descriptor1.Statement                                                       | Object                          |     1622397 | 100.00000000000000000000 |
| nldidlnorm.XX.Descriptor1.Statement._attrs                                                | Array                           |     1622397 | 100.00000000000000000000 |
| nldidlnorm.XX.Descriptor1.Statement.modified                                              | Array                           |     1622397 | 100.00000000000000000000 |
| nldidlnorm.XX.Item                                                                        | Object                          |     1622397 | 100.00000000000000000000 |
| nldidlnorm.XX.Item.Component                                                              | Object                          |     1622397 | 100.00000000000000000000 |
| nldidlnorm.XX.Item.Component.Resource                                                     | Object                          |     1622397 | 100.00000000000000000000 |
| nldidlnorm.XX.Item.Component.Resource._attrs                                              | Array                           |     1622397 | 100.00000000000000000000 |
| nldidlnorm.XX.Item.Component.Resource.mods                                                | Object                          |     1622397 | 100.00000000000000000000 |
| nldidlnorm.XX.Item.Component.Resource.mods._attrs                                         | Array                           |     1622397 | 100.00000000000000000000 |
| nldidlnorm.XX.Item.Component.Resource.mods.genre                                          | Array (1584485),Object (37912)  |     1622397 | 100.00000000000000000000 |
| nldidlnorm.XX.Item.Component.Resource.mods.name                                           | Object                          |     1622397 | 100.00000000000000000000 |
| nldidlnorm.XX.Item.Component.Resource.mods.name.role                                      | Object                          |     1622397 | 100.00000000000000000000 |
| nldidlnorm.XX.Item.Component.Resource.mods.name.role.roleTerm                             | Object                          |     1622397 | 100.00000000000000000000 |
| nldidlnorm.XX.Item.Component.Resource.mods.name.role.roleTerm._attrs                      | Array                           |     1622397 | 100.00000000000000000000 |
| nldidlnorm.XX.Item.Component.Resource.mods.name.role.roleTerm.text                        | Array                           |     1622397 | 100.00000000000000000000 |
| nldidlnorm.XX.Item.Component.Resource.mods.titleInfo                                      | Object                          |     1622397 | 100.00000000000000000000 |
| nldidlnorm.XX.Item.Component.Resource.mods.titleInfo.title                                | Array                           |     1622397 | 100.00000000000000000000 |
| nldidlnorm.XX.Item.Component.Resource.mods.typeOfResource                                 | Array (1584485),Object (37912)  |     1622397 | 100.00000000000000000000 |
| nldidlnorm.XX.Item.Descriptor                                                             | Object                          |     1622397 | 100.00000000000000000000 |
| nldidlnorm.XX.Item.Descriptor.Statement                                                   | Object                          |     1622397 | 100.00000000000000000000 |
| nldidlnorm.XX.Item.Descriptor.Statement._attrs                                            | Array                           |     1622397 | 100.00000000000000000000 |
| nldidlnorm.XX.Item.Descriptor.Statement.type                                              | Array                           |     1622397 | 100.00000000000000000000 |
| nldidlnorm.XX.Item1                                                                       | Object                          |     1622397 | 100.00000000000000000000 |
| nldidlnorm.XX.Item1.Component                                                             | Object                          |     1622397 | 100.00000000000000000000 |
| nldidlnorm.XX.Item1.Component.Resource                                                    | Array                           |     1622397 | 100.00000000000000000000 |
| nldidlnorm.XX.Item1.Descriptor                                                            | Object                          |     1622397 | 100.00000000000000000000 |
| nldidlnorm.XX.Item1.Descriptor.Statement                                                  | Object                          |     1622397 | 100.00000000000000000000 |
| nldidlnorm.XX.Item1.Descriptor.Statement._attrs                                           | Array                           |     1622397 | 100.00000000000000000000 |
| nldidlnorm.XX.Item1.Descriptor.Statement.type                                             | Array                           |     1622397 | 100.00000000000000000000 |
| originURL                                                                                 | Array                           |     1622397 | 100.00000000000000000000 |
| setSpec                                                                                   | Array                           |     1622397 | 100.00000000000000000000 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo                                     | Object                          |     1622285 |  99.99309663417770366323 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.dateIssued                          | Object (1622284),Array (1)      |     1622285 |  99.99309663417770366323 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.dateIssued._attrs                   | Array                           |     1622284 |  99.99303499698285691011 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.dateIssued.text                     | Array                           |     1622284 |  99.99303499698285691011 |
| nldidlnorm.XX.Item.Component.Resource.mods.name.namePart                                  | Object (1515452),Array (105738) |     1621190 |  99.92560390582576701490 |
| nldidlnorm.XX.Item.Component.Resource.mods.name._attrs                                    | Array                           |     1600433 |  98.64620065249134484020 |
| nldidlnorm.XX.Item.Component.Resource.mods.name.namePart._attrs                           | Array                           |     1515452 |  93.40821019762734067626 |
| nldidlnorm.XX.Item.Component.Resource.mods.name.namePart.text                             | Array                           |     1515452 |  93.40821019762734067626 |
| nldidlnorm.XX.Item.Component.Resource.mods.name.namePart1                                 | Object (1506423),Array (2223)   |     1508646 |  92.98870744953300970792 |
| nldidlnorm.XX.Item.Component.Resource.mods.name.namePart1._attrs                          | Array                           |     1506423 |  92.85168796539934987777 |
| nldidlnorm.XX.Item.Component.Resource.mods.name.namePart1.text                            | Array                           |     1506423 |  92.85168796539934987777 |
| nldidlnorm.XX.Item.Component.Resource.mods.name1                                          | Object                          |     1390731 |  85.72075761974411989286 |
| nldidlnorm.XX.Item.Component.Resource.mods.name1.role                                     | Object                          |     1390731 |  85.72075761974411989286 |
| nldidlnorm.XX.Item.Component.Resource.mods.name1.role.roleTerm                            | Object                          |     1390731 |  85.72075761974411989286 |
| nldidlnorm.XX.Item.Component.Resource.mods.name1.role.roleTerm._attrs                     | Array                           |     1390731 |  85.72075761974411989286 |
| nldidlnorm.XX.Item.Component.Resource.mods.name1.role.roleTerm.text                       | Array                           |     1390731 |  85.72075761974411989286 |
| nldidlnorm.XX.Item.Component.Resource.mods.name1.namePart                                 | Object (987481),Array (402694)  |     1390175 |  85.68648733941199679975 |
| nldidlnorm.XX.Item.Component.Resource.mods.name1._attrs                                   | Array                           |     1362434 |  83.97660991730137425293 |
| nldidlnorm.XX.Item.Component.Resource.mods.language                                       | Object                          |     1275432 |  78.61405069166178805062 |
| nldidlnorm.XX.Item.Component.Resource.mods.language.languageTerm                          | Object                          |     1275432 |  78.61405069166178805062 |
| nldidlnorm.XX.Item.Component.Resource.mods.language.languageTerm._attrs                   | Array                           |     1275432 |  78.61405069166178805062 |
| nldidlnorm.XX.Item.Component.Resource.mods.language.languageTerm.text                     | Array                           |     1275432 |  78.61405069166178805062 |
| nldidlnorm.XX.Item.Descriptor1                                                            | Object                          |     1148045 |  70.76227335232991322300 |
| nldidlnorm.XX.Item.Descriptor1.Statement                                                  | Object                          |     1148045 |  70.76227335232991322300 |
| nldidlnorm.XX.Item.Descriptor1.Statement.Identifier                                       | Array                           |     1148045 |  70.76227335232991322300 |
| nldidlnorm.XX.Item.Descriptor1.Statement._attrs                                           | Array                           |     1148045 |  70.76227335232991322300 |
| Journal.XX.relatedItem                                                                    | Object                          |     1124764 |  69.32729781921440803671 |
| Journal.XX.relatedItem._attrs                                                             | Array                           |     1124764 |  69.32729781921440803671 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem                                    | Object                          |     1124764 |  69.32729781921440803671 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem._attrs                             | Array                           |     1124764 |  69.32729781921440803671 |
| Journal.XX.relatedItem.titleInfo                                                          | Object (1102299),Array (202)    |     1102501 |  67.95506895044800899086 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.titleInfo                          | Object (1102299),Array (202)    |     1102501 |  67.95506895044800899086 |
| Journal.XX.relatedItem.titleInfo.title                                                    | Array                           |     1102299 |  67.94261823708993119908 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.titleInfo.title                    | Array                           |     1102299 |  67.94261823708993119908 |
| nldidlnorm.XX.Item.Component.Resource.mods.name2                                          | Object                          |     1086115 |  66.94508187576776947481 |
| nldidlnorm.XX.Item.Component.Resource.mods.name2.role                                     | Object                          |     1086115 |  66.94508187576776947481 |
| nldidlnorm.XX.Item.Component.Resource.mods.name2.role.roleTerm                            | Object                          |     1086115 |  66.94508187576776947481 |
| nldidlnorm.XX.Item.Component.Resource.mods.name2.role.roleTerm._attrs                     | Array                           |     1086115 |  66.94508187576776947481 |
| nldidlnorm.XX.Item.Component.Resource.mods.name2.role.roleTerm.text                       | Array                           |     1086115 |  66.94508187576776947481 |
| nldidlnorm.XX.Item.Component.Resource.mods.name2.namePart                                 | Object (787874),Array (297680)  |     1085554 |  66.91050340946142682697 |
| nldidlnorm.XX.Item.Component.Resource.mods.identifier                                     | Object                          |     1079981 |  66.56699932260723073796 |
| nldidlnorm.XX.Item.Component.Resource.mods.identifier._attrs                              | Array                           |     1079981 |  66.56699932260723073796 |
| nldidlnorm.XX.Item.Component.Resource.mods.identifier.text                                | Array                           |     1079981 |  66.56699932260723073796 |
| nldidlnorm.XX.Item.Component.Resource.mods.name2._attrs                                   | Array                           |     1063662 |  65.56114193998139683117 |
| Journal.XX.relatedItem.part                                                               | Object (1050936),Array (6271)   |     1057207 |  65.16327384727658511565 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.part                               | Object (1050936),Array (6271)   |     1057207 |  65.16327384727658511565 |
| Journal.XX.relatedItem.part.extent                                                        | Object (941774),Array (63576)   |     1005350 |  61.96695383435743309519 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.part.extent                        | Object (941774),Array (63576)   |     1005350 |  61.96695383435743309519 |
| nldidlnorm.XX.Item.Component.Resource.mods.name1.namePart._attrs                          | Array                           |      987481 |  60.86555879972657834287 |
| nldidlnorm.XX.Item.Component.Resource.mods.name1.namePart.text                            | Array                           |      987481 |  60.86555879972657834287 |
| nldidlnorm.XX.Item.Component.Resource.mods.name1.namePart1                                | Object (985677),Array (695)     |      986372 |  60.79720315064685109974 |
| nldidlnorm.XX.Item.Component.Resource.mods.name1.namePart1._attrs                         | Array                           |      985677 |  60.75436530023169723336 |
| nldidlnorm.XX.Item.Component.Resource.mods.name1.namePart1.text                           | Array                           |      985677 |  60.75436530023169723336 |
| Journal.XX.relatedItem.part.extent._attrs                                                 | Array                           |      941774 |  58.04830753508543494945 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.part.extent._attrs                 | Array                           |      941774 |  58.04830753508543494945 |
| nldidlnorm.XX.Item.Component.Resource.mods.identifier1                                    | Object                          |      937256 |  57.76983068878949012515 |
| nldidlnorm.XX.Item.Component.Resource.mods.identifier1._attrs                             | Array                           |      937256 |  57.76983068878949012515 |
| nldidlnorm.XX.Item.Component.Resource.mods.identifier1.text                               | Array                           |      937256 |  57.76983068878949012515 |
| nldidlnorm.XX.Item.Component.Resource.mods.identifier2                                    | Object                          |      863526 |  53.22532031309229694216 |
| nldidlnorm.XX.Item.Component.Resource.mods.identifier2._attrs                             | Array                           |      863526 |  53.22532031309229694216 |
| nldidlnorm.XX.Item.Component.Resource.mods.identifier2.text                               | Array                           |      863526 |  53.22532031309229694216 |
| Journal.XX.relatedItem.part.extent.start                                                  | Array                           |      862794 |  53.18020188646798374066 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.part.extent.start                  | Array                           |      862794 |  53.18020188646798374066 |
| Journal.XX.relatedItem.identifier                                                         | Object                          |      810909 |  49.98215603209325763601 |
| Journal.XX.relatedItem.identifier._attrs                                                  | Array                           |      810909 |  49.98215603209325763601 |
| Journal.XX.relatedItem.identifier.text                                                    | Array                           |      810909 |  49.98215603209325763601 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.identifier                         | Object                          |      810909 |  49.98215603209325763601 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.identifier._attrs                  | Array                           |      810909 |  49.98215603209325763601 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.identifier.text                    | Array                           |      810909 |  49.98215603209325763601 |
| Journal.XX.relatedItem.part.detail                                                        | Object                          |      804303 |  49.57498072296731095321 |
| Journal.XX.relatedItem.part.detail._attrs                                                 | Array                           |      804303 |  49.57498072296731095321 |
| Journal.XX.relatedItem.part.detail.number                                                 | Array                           |      804303 |  49.57498072296731095321 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.part.detail                        | Object                          |      804303 |  49.57498072296731095321 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.part.detail._attrs                 | Array                           |      804303 |  49.57498072296731095321 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.part.detail.number                 | Array                           |      804303 |  49.57498072296731095321 |
| nldidlnorm.XX.Item.Component.Resource.mods.name3                                          | Object                          |      798600 |  49.22346380078365513100 |
| nldidlnorm.XX.Item.Component.Resource.mods.name3.role                                     | Object                          |      798600 |  49.22346380078365513100 |
| nldidlnorm.XX.Item.Component.Resource.mods.name3.role.roleTerm                            | Object                          |      798600 |  49.22346380078365513100 |
| nldidlnorm.XX.Item.Component.Resource.mods.name3.role.roleTerm._attrs                     | Array                           |      798600 |  49.22346380078365513100 |
| nldidlnorm.XX.Item.Component.Resource.mods.name3.role.roleTerm.text                       | Array                           |      798600 |  49.22346380078365513100 |
| nldidlnorm.XX.Item.Component.Resource.mods.name3.namePart                                 | Object (577499),Array (220766)  |      798265 |  49.20281534051159866294 |
| nldidlnorm.XX.Item.Component.Resource.mods.name2.namePart._attrs                          | Array                           |      787874 |  48.56234324890886711046 |
| nldidlnorm.XX.Item.Component.Resource.mods.name2.namePart.text                            | Array                           |      787874 |  48.56234324890886711046 |
| nldidlnorm.XX.Item.Component.Resource.mods.name2.namePart1                                | Object (786344),Array (153)     |      786497 |  48.47746883161149611396 |
| nldidlnorm.XX.Item.Component.Resource.mods.name2.namePart1._attrs                         | Array                           |      786344 |  48.46803834080068185131 |
| nldidlnorm.XX.Item.Component.Resource.mods.name2.namePart1.text                           | Array                           |      786344 |  48.46803834080068185131 |
| nldidlnorm.XX.Item.Component.Resource.mods.name3._attrs                                   | Array                           |      786175 |  48.45762165487239059303 |
| Journal.XX.relatedItem.part.extent.end                                                    | Array                           |      756364 |  46.62015523943892958414 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.part.extent.end                    | Array                           |      756364 |  46.62015523943892958414 |
| nldidlnorm.XX.Item2                                                                       | Object                          |      751758 |  46.33625431999689681106 |
| nldidlnorm.XX.Item2.Component                                                             | Object                          |      751758 |  46.33625431999689681106 |
| nldidlnorm.XX.Item2.Component.Resource                                                    | Array                           |      751758 |  46.33625431999689681106 |
| nldidlnorm.XX.Item2.Descriptor                                                            | Object                          |      751758 |  46.33625431999689681106 |
| nldidlnorm.XX.Item2.Descriptor.Statement                                                  | Object                          |      751758 |  46.33625431999689681106 |
| nldidlnorm.XX.Item2.Descriptor.Statement._attrs                                           | Array                           |      751758 |  46.33625431999689681106 |
| nldidlnorm.XX.Item2.Descriptor.Statement.type                                             | Array                           |      751758 |  46.33625431999689681106 |
| nldidlnorm.XX.Item1.Descriptor1                                                           | Object                          |      750070 |  46.23221073510367773451 |
| nldidlnorm.XX.Item1.Descriptor1.Statement                                                 | Object                          |      750070 |  46.23221073510367773451 |
| nldidlnorm.XX.Item1.Descriptor1.Statement._attrs                                          | Array                           |      750070 |  46.23221073510367773451 |
| Keywords.XX.topic                                                                         | Array (708378),Object (37857)   |      746235 |  45.99583209288478968801 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject                                        | Object                          |      746235 |  45.99583209288478968801 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject.topic                                  | Array (708378),Object (37857)   |      746235 |  45.99583209288478968801 |
| nldidlnorm.XX.Item.Component.Resource.mods.name.nameIdentifier                            | Object (625647),Array (2)       |      625649 |  38.56324931567304759028 |
| nldidlnorm.XX.Item.Component.Resource.mods.name.nameIdentifier._attrs                     | Array                           |      625647 |  38.56312604128336118947 |
| nldidlnorm.XX.Item.Component.Resource.mods.name.nameIdentifier.text                       | Array                           |      625647 |  38.56312604128336118947 |
| nldidlnorm.XX.Item.Component.Resource.mods.abstract                                       | Array (597217),Object (14656)   |      611873 |  37.71413531953029973920 |
| nldidlnorm.XX.Item.Component.Resource.mods.identifier3                                    | Object                          |      607715 |  37.45784786337745941864 |
| nldidlnorm.XX.Item.Component.Resource.mods.identifier3._attrs                             | Array                           |      607715 |  37.45784786337745941864 |
| nldidlnorm.XX.Item.Component.Resource.mods.identifier3.text                               | Array                           |      607715 |  37.45784786337745941864 |
| nldidlnorm.XX.Item.Component.Resource.mods.name3.namePart._attrs                          | Array                           |      577499 |  35.59541838403301028393 |
| nldidlnorm.XX.Item.Component.Resource.mods.name3.namePart.text                            | Array                           |      577499 |  35.59541838403301028393 |
| Journal.XX.relatedItem.part.detail1                                                       | Object                          |      576697 |  35.54598535376975831923 |
| Journal.XX.relatedItem.part.detail1._attrs                                                | Array                           |      576697 |  35.54598535376975831923 |
| Journal.XX.relatedItem.part.detail1.number                                                | Array                           |      576697 |  35.54598535376975831923 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.part.detail1                       | Object                          |      576697 |  35.54598535376975831923 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.part.detail1._attrs                | Array                           |      576697 |  35.54598535376975831923 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.part.detail1.number                | Array                           |      576697 |  35.54598535376975831923 |
| nldidlnorm.XX.Item.Component.Resource.mods.name3.namePart1                                | Object (575532),Array (218)     |      575750 |  35.48761493025443058968 |
| nldidlnorm.XX.Item.Component.Resource.mods.name3.namePart1._attrs                         | Array                           |      575532 |  35.47417802177888290771 |
| nldidlnorm.XX.Item.Component.Resource.mods.name3.namePart1.text                           | Array                           |      575532 |  35.47417802177888290771 |
| nldidlnorm.XX.Item.Component.Resource.mods.name4                                          | Object                          |      569730 |  35.11655901730587459042 |
| nldidlnorm.XX.Item.Component.Resource.mods.name4.role                                     | Object                          |      569730 |  35.11655901730587459042 |
| nldidlnorm.XX.Item.Component.Resource.mods.name4.role.roleTerm                            | Object                          |      569730 |  35.11655901730587459042 |
| nldidlnorm.XX.Item.Component.Resource.mods.name4.role.roleTerm._attrs                     | Array                           |      569730 |  35.11655901730587459042 |
| nldidlnorm.XX.Item.Component.Resource.mods.name4.role.roleTerm.text                       | Array                           |      569730 |  35.11655901730587459042 |
| nldidlnorm.XX.Item.Component.Resource.mods.name4.namePart                                 | Object (402974),Array (166477)  |      569451 |  35.09936223994497339618 |
| nldidlnorm.XX.Item.Component.Resource.mods.name4._attrs                                   | Array                           |      563914 |  34.75807709210507567832 |
| Journal.XX.relatedItem.part.extent.list                                                   | Array                           |      549830 |  33.88997883995101290111 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.part.extent.list                   | Array                           |      549830 |  33.88997883995101290111 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.publisher                           | Array                           |      546325 |  33.67394047202996887336 |
| nldidlnorm.XX.Item1.Descriptor2                                                           | Object                          |      527131 |  32.49087615423351849131 |
| nldidlnorm.XX.Item1.Descriptor2.Statement                                                 | Object                          |      527131 |  32.49087615423351849131 |
| nldidlnorm.XX.Item1.Descriptor2.Statement._attrs                                          | Array                           |      527131 |  32.49087615423351849131 |
| Keywords.XX.topic1                                                                        | Array                           |      464625 |  28.63818165344240540549 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject.topic1                                 | Array                           |      464625 |  28.63818165344240540549 |
| nldidlnorm.XX.Item.Component.Resource.mods.name1.role1                                    | Object                          |      446177 |  27.50109868299805881975 |
| nldidlnorm.XX.Item.Component.Resource.mods.name1.role1.roleTerm                           | Object                          |      446177 |  27.50109868299805881975 |
| nldidlnorm.XX.Item.Component.Resource.mods.name1.role1.roleTerm._attrs                    | Array                           |      446177 |  27.50109868299805881975 |
| nldidlnorm.XX.Item.Component.Resource.mods.name1.role1.roleTerm.text                      | Array                           |      446177 |  27.50109868299805881975 |
| Journal.XX.relatedItem.part.detail.caption                                                | Array                           |      434693 |  26.79325713743306991432 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.part.detail.caption                | Array                           |      434693 |  26.79325713743306991432 |
| nldidlnorm.XX.Item.Component.Resource.mods.name4.namePart._attrs                          | Array                           |      402974 |  24.83818695424116285153 |
| nldidlnorm.XX.Item.Component.Resource.mods.name4.namePart.text                            | Array                           |      402974 |  24.83818695424116285153 |
| nldidlnorm.XX.Item.Component.Resource.mods.name4.namePart1                                | Object (401575),Array (165)     |      401740 |  24.76212665580619187722 |
| nldidlnorm.XX.Item.Component.Resource.mods.name4.namePart1._attrs                         | Array                           |      401575 |  24.75195651865726986784 |
| nldidlnorm.XX.Item.Component.Resource.mods.name4.namePart1.text                           | Array                           |      401575 |  24.75195651865726986784 |
| nldidlnorm.XX.Item.Component.Resource.mods.name5                                          | Object                          |      397050 |  24.47304821199743329885 |
| nldidlnorm.XX.Item.Component.Resource.mods.name5.role                                     | Object                          |      397050 |  24.47304821199743329885 |
| nldidlnorm.XX.Item.Component.Resource.mods.name5.role.roleTerm                            | Object                          |      397050 |  24.47304821199743329885 |
| nldidlnorm.XX.Item.Component.Resource.mods.name5.role.roleTerm._attrs                     | Array                           |      397050 |  24.47304821199743329885 |
| nldidlnorm.XX.Item.Component.Resource.mods.name5.role.roleTerm.text                       | Array                           |      397050 |  24.47304821199743329885 |
| nldidlnorm.XX.Item.Component.Resource.mods.name5.namePart                                 | Object (275773),Array (121199)  |      396972 |  24.46824051079975959055 |
| nldidlnorm.XX.Item.Component.Resource.mods.name5._attrs                                   | Array                           |      394789 |  24.33368651445977803860 |
| nldidlnorm.XX.Item1.Descriptor1.Statement.Identifier                                      | Array                           |      387436 |  23.88046822078689856994 |
| nldidlnorm.XX.Item1.Descriptor2.Statement.accessRights                                    | Array                           |      387436 |  23.88046822078689856994 |
| Keywords.XX.topic2                                                                        | Array                           |      370606 |  22.84311423159682874484 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject.topic2                                 | Array                           |      370606 |  22.84311423159682874484 |
| nldidlnorm.XX.Item1.Descriptor1.Statement.accessRights                                    | Array                           |      362298 |  22.33103241684988304883 |
| nldidlnorm.XX.Item.Component.Resource.mods.name.role1                                     | Object                          |      360553 |  22.22347551185067615620 |
| nldidlnorm.XX.Item.Component.Resource.mods.name.role1.roleTerm                            | Object                          |      360553 |  22.22347551185067615620 |
| nldidlnorm.XX.Item.Component.Resource.mods.name.role1.roleTerm._attrs                     | Array                           |      360553 |  22.22347551185067615620 |
| nldidlnorm.XX.Item.Component.Resource.mods.name.role1.roleTerm.text                       | Array                           |      360553 |  22.22347551185067615620 |
| nldidlnorm.XX.Item.Component.Resource.mods.name2.role1                                    | Object                          |      338916 |  20.88983152705533896665 |
| nldidlnorm.XX.Item.Component.Resource.mods.name2.role1.roleTerm                           | Object                          |      338916 |  20.88983152705533896665 |
| nldidlnorm.XX.Item.Component.Resource.mods.name2.role1.roleTerm._attrs                    | Array                           |      338916 |  20.88983152705533896665 |
| nldidlnorm.XX.Item.Component.Resource.mods.name2.role1.roleTerm.text                      | Array                           |      338916 |  20.88983152705533896665 |
| nldidlnorm.XX.Item.Component.Resource.mods.name1.nameIdentifier                           | Object                          |      333074 |  20.52974703478864881845 |
| nldidlnorm.XX.Item.Component.Resource.mods.name1.nameIdentifier._attrs                    | Array                           |      333074 |  20.52974703478864881845 |
| nldidlnorm.XX.Item.Component.Resource.mods.name1.nameIdentifier.text                      | Array                           |      333074 |  20.52974703478864881845 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.place                               | Object                          |      329611 |  20.31629742905096591699 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.place.placeTerm                     | Object (157208),Array (172403)  |      329611 |  20.31629742905096591699 |
| nldidlnorm.XX.Item.Component.Resource.mods.name.affiliation                               | Array                           |      325955 |  20.09095184470878692196 |
| Journal.XX.relatedItem.part.extent.total                                                  | Array                           |      313089 |  19.29792769587221812344 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.part.extent.total                  | Array                           |      313089 |  19.29792769587221812344 |
| Keywords.XX.topic3                                                                        | Array                           |      311622 |  19.20750593103907277737 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject.topic3                                 | Array                           |      311622 |  19.20750593103907277737 |
| Journal.XX.relatedItem.part.detail1.caption                                               | Array                           |      302203 |  18.62694519282271699012 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.part.detail1.caption               | Array                           |      302203 |  18.62694519282271699012 |
| nldidlnorm.XX.Item1.Descriptor3                                                           | Object                          |      302004 |  18.61467939104916879955 |
| nldidlnorm.XX.Item1.Descriptor3.Statement                                                 | Object                          |      302004 |  18.61467939104916879955 |
| nldidlnorm.XX.Item1.Descriptor3.Statement._attrs                                          | Array                           |      302004 |  18.61467939104916879955 |
| nldidlnorm.XX.Item.Component.Resource.mods.name6                                          | Object                          |      282989 |  17.44264813112943457440 |
| nldidlnorm.XX.Item.Component.Resource.mods.name6.role                                     | Object                          |      282989 |  17.44264813112943457440 |
| nldidlnorm.XX.Item.Component.Resource.mods.name6.role.roleTerm                            | Object                          |      282989 |  17.44264813112943457440 |
| nldidlnorm.XX.Item.Component.Resource.mods.name6.role.roleTerm._attrs                     | Array                           |      282989 |  17.44264813112943457440 |
| nldidlnorm.XX.Item.Component.Resource.mods.name6.role.roleTerm.text                       | Array                           |      282989 |  17.44264813112943457440 |
| nldidlnorm.XX.Item.Component.Resource.mods.name6.namePart                                 | Object (190284),Array (92646)   |      282930 |  17.43901153663375680480 |
| nldidlnorm.XX.Item.Component.Resource.mods.name6._attrs                                   | Array                           |      281924 |  17.37700451862275485837 |
| nldidlnorm.XX.Item.Component.Resource.mods.name5.namePart._attrs                          | Array                           |      275773 |  16.99787413314989947821 |
| nldidlnorm.XX.Item.Component.Resource.mods.name5.namePart.text                            | Array                           |      275773 |  16.99787413314989947821 |
| nldidlnorm.XX.Item.Component.Resource.mods.name5.namePart1                                | Object (274856),Array (157)     |      275013 |  16.95102986507001574523 |
| nldidlnorm.XX.Item.Component.Resource.mods.name5.namePart1._attrs                         | Array                           |      274856 |  16.94135282547982868095 |
| nldidlnorm.XX.Item.Component.Resource.mods.name5.namePart1.text                           | Array                           |      274856 |  16.94135282547982868095 |
| nldidlnorm.XX.Item.Component.Resource.mods.identifier4                                    | Object                          |      272250 |  16.78072629572169915946 |
| nldidlnorm.XX.Item.Component.Resource.mods.identifier4._attrs                             | Array                           |      272250 |  16.78072629572169915946 |
| nldidlnorm.XX.Item.Component.Resource.mods.identifier4.text                               | Array                           |      272250 |  16.78072629572169915946 |
| Keywords.XX.topic4                                                                        | Array                           |      271121 |  16.71113790274513632994 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject.topic4                                 | Array                           |      271121 |  16.71113790274513632994 |
| nldidlnorm.XX.Item.Component.Resource.mods.name3.role1                                    | Object                          |      251679 |  15.51278756062788666270 |
| nldidlnorm.XX.Item.Component.Resource.mods.name3.role1.roleTerm                           | Object                          |      251679 |  15.51278756062788666270 |
| nldidlnorm.XX.Item.Component.Resource.mods.name3.role1.roleTerm._attrs                    | Array                           |      251679 |  15.51278756062788666270 |
| nldidlnorm.XX.Item.Component.Resource.mods.name3.role1.roleTerm.text                      | Array                           |      251679 |  15.51278756062788666270 |
| nldidlnorm.XX.Item1.Descriptor3.Statement.modified                                        | Array                           |      236390 |  14.57041648868926664306 |
| Keywords.XX.topic5                                                                        | Array                           |      230270 |  14.19319685625651494831 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject.topic5                                 | Array                           |      230270 |  14.19319685625651494831 |
| nldidlnorm.XX.Item.Component.Resource.mods.name2.nameIdentifier                           | Object (230037),Array (2)       |      230039 |  14.17895866424802342465 |
| nldidlnorm.XX.Item.Component.Resource.mods.name2.nameIdentifier._attrs                    | Array                           |      230037 |  14.17883538985833880020 |
| nldidlnorm.XX.Item.Component.Resource.mods.name2.nameIdentifier.text                      | Array                           |      230037 |  14.17883538985833880020 |
| nldidlnorm.XX.Item.Component.Resource.mods.titleInfo._attrs                               | Array                           |      225370 |  13.89117460153094540942 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject1                                       | Object                          |      225130 |  13.87638167476887574026 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject1.topic                                 | Array (219041),Object (6089)    |      225130 |  13.87638167476887574026 |
| nldidlnorm.XX.Item.Component.Resource.mods.name7                                          | Object                          |      205774 |  12.68333213140803472641 |
| nldidlnorm.XX.Item.Component.Resource.mods.name7.role                                     | Object                          |      205774 |  12.68333213140803472641 |
| nldidlnorm.XX.Item.Component.Resource.mods.name7.role.roleTerm                            | Object                          |      205774 |  12.68333213140803472641 |
| nldidlnorm.XX.Item.Component.Resource.mods.name7.role.roleTerm._attrs                     | Array                           |      205774 |  12.68333213140803472641 |
| nldidlnorm.XX.Item.Component.Resource.mods.name7.role.roleTerm.text                       | Array                           |      205774 |  12.68333213140803472641 |
| nldidlnorm.XX.Item.Component.Resource.mods.name7.namePart                                 | Object (135307),Array (70429)   |      205736 |  12.68098991800403929631 |
| nldidlnorm.XX.Item.Component.Resource.mods.name7._attrs                                   | Array                           |      205127 |  12.64345286634529053060 |
| Journal.XX.relatedItem.originInfo                                                         | Object                          |      200982 |  12.38796669372539582810 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.originInfo                         | Object                          |      200982 |  12.38796669372539582810 |
| Keywords.XX._attrs                                                                        | Array                           |      200578 |  12.36306526700924557360 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject._attrs                                 | Array                           |      200578 |  12.36306526700924557360 |
| Keywords.XX.topic6                                                                        | Array                           |      192460 |  11.86269451928227169901 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject.topic6                                 | Array                           |      192460 |  11.86269451928227169901 |
| nldidlnorm.XX.Item.Component.Resource.mods.name6.namePart._attrs                          | Array                           |      190284 |  11.72857198330618189175 |
| nldidlnorm.XX.Item.Component.Resource.mods.name6.namePart.text                            | Array                           |      190284 |  11.72857198330618189175 |
| nldidlnorm.XX.Item.Component.Resource.mods.name6.namePart1                                | Object (189609),Array (108)     |      189717 |  11.69362369383079425234 |
| nldidlnorm.XX.Item.Component.Resource.mods.name6.namePart1._attrs                         | Array                           |      189609 |  11.68696687678786361175 |
| nldidlnorm.XX.Item.Component.Resource.mods.name6.namePart1.text                           | Array                           |      189609 |  11.68696687678786361175 |
| nldidlnorm.XX.Item.Component.Resource.mods.name4.role1                                    | Object                          |      185367 |  11.42550189626829926226 |
| nldidlnorm.XX.Item.Component.Resource.mods.name4.role1.roleTerm                           | Object                          |      185367 |  11.42550189626829926226 |
| nldidlnorm.XX.Item.Component.Resource.mods.name4.role1.roleTerm._attrs                    | Array                           |      185367 |  11.42550189626829926226 |
| nldidlnorm.XX.Item.Component.Resource.mods.name4.role1.roleTerm.text                      | Array                           |      185367 |  11.42550189626829926226 |
| Journal.XX.relatedItem.originInfo.dateIssued                                              | Object                          |      181351 |  11.17796692178301576348 |
| Journal.XX.relatedItem.originInfo.dateIssued._attrs                                       | Array                           |      181351 |  11.17796692178301576348 |
| Journal.XX.relatedItem.originInfo.dateIssued.text                                         | Array                           |      181351 |  11.17796692178301576348 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.originInfo.dateIssued              | Object                          |      181351 |  11.17796692178301576348 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.originInfo.dateIssued._attrs       | Array                           |      181351 |  11.17796692178301576348 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.originInfo.dateIssued.text         | Array                           |      181351 |  11.17796692178301576348 |
| nldidlnorm.XX.Item.Component.Resource.mods.identifier5                                    | Object                          |      175758 |  10.83323009103197342995 |
| nldidlnorm.XX.Item.Component.Resource.mods.identifier5._attrs                             | Array                           |      175758 |  10.83323009103197342995 |
| nldidlnorm.XX.Item.Component.Resource.mods.identifier5.text                               | Array                           |      175758 |  10.83323009103197342995 |
| Journal.XX.relatedItem.originInfo.publisher                                               | Array                           |      166464 |  10.26037400217086137388 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.originInfo.publisher               | Array                           |      166464 |  10.26037400217086137388 |
| Keywords.XX.topic7                                                                        | Array                           |      165878 |  10.22425460599347779578 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject.topic7                                 | Array                           |      165878 |  10.22425460599347779578 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject1._attrs                                | Array                           |      159491 |   9.83057784253792377172 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.place.placeTerm._attrs              | Array                           |      157208 |   9.68986012671374474792 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.place.placeTerm.text                | Array                           |      157208 |   9.68986012671374474792 |
| Journal.XX.relatedItem.titleInfo.subTitle                                                 | Array                           |      156295 |   9.63358536782304142321 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.titleInfo.subTitle                 | Array                           |      156295 |   9.63358536782304142321 |
| nldidlnorm.XX.Item.Component.Resource.mods.name1.affiliation                              | Array                           |      154105 |   9.49859991111916457385 |
| nldidlnorm.XX.Item.Component.Resource.mods.name8                                          | Object                          |      153015 |   9.43141536874143682212 |
| nldidlnorm.XX.Item.Component.Resource.mods.name8.role                                     | Object                          |      153015 |   9.43141536874143682212 |
| nldidlnorm.XX.Item.Component.Resource.mods.name8.role.roleTerm                            | Object                          |      153015 |   9.43141536874143682212 |
| nldidlnorm.XX.Item.Component.Resource.mods.name8.role.roleTerm._attrs                     | Array                           |      153015 |   9.43141536874143682212 |
| nldidlnorm.XX.Item.Component.Resource.mods.name8.role.roleTerm.text                       | Array                           |      153015 |   9.43141536874143682212 |
| nldidlnorm.XX.Item.Component.Resource.mods.name8.namePart                                 | Object (98818),Array (54164)    |      152982 |   9.42938134131165206497 |
| nldidlnorm.XX.Item.Component.Resource.mods.name8._attrs                                   | Array                           |      152528 |   9.40139805485340573910 |
| nldidlnorm.XX.Item.Component.Resource.mods.name3.nameIdentifier                           | Object (147797),Array (1)       |      147798 |   9.10985412325096710617 |
| nldidlnorm.XX.Item.Component.Resource.mods.name3.nameIdentifier._attrs                    | Array                           |      147797 |   9.10979248605612568213 |
| nldidlnorm.XX.Item.Component.Resource.mods.name3.nameIdentifier.text                      | Array                           |      147797 |   9.10979248605612568213 |
| Journal.XX.relatedItem.identifier1                                                        | Object                          |      144370 |   8.89856181933275358631 |
| Journal.XX.relatedItem.identifier1._attrs                                                 | Array                           |      144370 |   8.89856181933275358631 |
| Journal.XX.relatedItem.identifier1.text                                                   | Array                           |      144370 |   8.89856181933275358631 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.identifier1                        | Object                          |      144370 |   8.89856181933275358631 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.identifier1._attrs                 | Array                           |      144370 |   8.89856181933275358631 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.identifier1.text                   | Array                           |      144370 |   8.89856181933275358631 |
| Keywords.XX.topic8                                                                        | Array                           |      142988 |   8.81337921606117369322 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject.topic8                                 | Array                           |      142988 |   8.81337921606117369322 |
| nldidlnorm.XX.Item.Component.Resource.mods.name7.namePart._attrs                          | Array                           |      135307 |   8.33994392248013305391 |
| nldidlnorm.XX.Item.Component.Resource.mods.name7.namePart.text                            | Array                           |      135307 |   8.33994392248013305391 |
| nldidlnorm.XX.Item.Component.Resource.mods.name5.role1                                    | Object                          |      135260 |   8.33704697432256125467 |
| nldidlnorm.XX.Item.Component.Resource.mods.name5.role1.roleTerm                           | Object                          |      135260 |   8.33704697432256125467 |
| nldidlnorm.XX.Item.Component.Resource.mods.name5.role1.roleTerm._attrs                    | Array                           |      135260 |   8.33704697432256125467 |
| nldidlnorm.XX.Item.Component.Resource.mods.name5.role1.roleTerm.text                      | Array                           |      135260 |   8.33704697432256125467 |
| nldidlnorm.XX.Item.Component.Resource.mods.name7.namePart1                                | Object (134772),Array (78)      |      134850 |   8.31177572443736067953 |
| nldidlnorm.XX.Item.Component.Resource.mods.name7.namePart1._attrs                         | Array                           |      134772 |   8.30696802323968874759 |
| nldidlnorm.XX.Item.Component.Resource.mods.name7.namePart1.text                           | Array                           |      134772 |   8.30696802323968874759 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject1.topic1                                | Array (129480),Object (2277)    |      131757 |   8.12113188079120007501 |
| nldidlnorm.XX.Item.Component.Resource.mods.extension                                      | Object                          |      131012 |   8.07521217063394480817 |
| nldidlnorm.XX.Item.Component.Resource.mods.extension.rights                               | Object                          |      131005 |   8.07478071027005128713 |
| nldidlnorm.XX.Item.Component.Resource.mods.extension.rights._attrs                        | Array                           |      131005 |   8.07478071027005128713 |
| nldidlnorm.XX.Item.Component.Resource.mods.extension.rights.description                   | Array                           |      130942 |   8.07089756699500782133 |
| nldidlnorm.XX.Item1.Descriptor2.Statement.modified                                        | Array                           |      128934 |   7.94713007975236607194 |
| nldidlnorm.XX.Item.Component.Resource.mods.name1.role2                                    | Object                          |      127769 |   7.87532274776149154860 |
| nldidlnorm.XX.Item.Component.Resource.mods.name1.role2.roleTerm                           | Object                          |      127769 |   7.87532274776149154860 |
| nldidlnorm.XX.Item.Component.Resource.mods.name1.role2.roleTerm._attrs                    | Array                           |      127769 |   7.87532274776149154860 |
| nldidlnorm.XX.Item.Component.Resource.mods.name1.role2.roleTerm.text                      | Array                           |      127769 |   7.87532274776149154860 |
| Keywords.XX.topic9                                                                        | Array                           |      123995 |   7.64270397442796056708 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject.topic9                                 | Array                           |      123995 |   7.64270397442796056708 |
| nldidlnorm.XX.Item.Component.Resource.mods.name9                                          | Object                          |      116060 |   7.15361283335706321651 |
| nldidlnorm.XX.Item.Component.Resource.mods.name9.role                                     | Object                          |      116060 |   7.15361283335706321651 |
| nldidlnorm.XX.Item.Component.Resource.mods.name9.role.roleTerm                            | Object                          |      116060 |   7.15361283335706321651 |
| nldidlnorm.XX.Item.Component.Resource.mods.name9.role.roleTerm._attrs                     | Array                           |      116060 |   7.15361283335706321651 |
| nldidlnorm.XX.Item.Component.Resource.mods.name9.role.roleTerm.text                       | Array                           |      116060 |   7.15361283335706321651 |
| nldidlnorm.XX.Item.Component.Resource.mods.name9.namePart                                 | Object (74435),Array (41538)    |      115973 |   7.14825039740581402725 |
| nldidlnorm.XX.Item.Component.Resource.mods.name9._attrs                                   | Array                           |      115701 |   7.13148508040880280134 |
| nldidlnorm.XX.Item.Component.Resource.mods.name1.displayForm                              | Array                           |      115586 |   7.12439680300197775153 |
| nldidlnorm.XX.Item.Component.Resource.mods.name2.affiliation                              | Array                           |      108085 |   6.66205620449248847592 |
| nldidlnorm.XX.Item.Component.Resource.mods.name.displayForm                               | Array                           |      107818 |   6.64559907346968703479 |
| nldidlnorm.XX.Item.Component.Resource.mods.titleInfo.subTitle                             | Array                           |      107711 |   6.63900389362159781825 |
| nldidlnorm.XX.Item.Component.Resource.mods.name.affiliation1                              | Array                           |      106710 |   6.57730506158480299206 |
| nldidlnorm.XX.Item.Component.Resource.mods.name.role.roleTerm1                            | Object                          |      104911 |   6.46641974806412989096 |
| nldidlnorm.XX.Item.Component.Resource.mods.name.role.roleTerm1._attrs                     | Array                           |      104911 |   6.46641974806412989096 |
| nldidlnorm.XX.Item.Component.Resource.mods.name.role.roleTerm1.text                       | Array                           |      104911 |   6.46641974806412989096 |
| nldidlnorm.XX.Item.Component.Resource.mods.name6.role1                                    | Object                          |      101789 |   6.27398842576755289002 |
| nldidlnorm.XX.Item.Component.Resource.mods.name6.role1.roleTerm                           | Object                          |      101789 |   6.27398842576755289002 |
| nldidlnorm.XX.Item.Component.Resource.mods.name6.role1.roleTerm._attrs                    | Array                           |      101789 |   6.27398842576755289002 |
| nldidlnorm.XX.Item.Component.Resource.mods.name6.role1.roleTerm.text                      | Array                           |      101789 |   6.27398842576755289002 |
| nldidlnorm.XX.Item.Component.Resource.mods.name8.namePart._attrs                          | Array                           |       98818 |   6.09086431989210996818 |
| nldidlnorm.XX.Item.Component.Resource.mods.name8.namePart.text                            | Array                           |       98818 |   6.09086431989210996818 |
| nldidlnorm.XX.Item.Component.Resource.mods.identifier6                                    | Object                          |       98454 |   6.06842838096963976824 |
| nldidlnorm.XX.Item.Component.Resource.mods.identifier6._attrs                             | Array                           |       98454 |   6.06842838096963976824 |
| nldidlnorm.XX.Item.Component.Resource.mods.identifier6.text                               | Array                           |       98454 |   6.06842838096963976824 |
| nldidlnorm.XX.Item.Component.Resource.mods.name8.namePart1                                | Object (98365),Array (65)       |       98430 |   6.06694908829343226842 |
| nldidlnorm.XX.Item.Component.Resource.mods.name8.namePart1._attrs                         | Array                           |       98365 |   6.06294267062870595453 |
| nldidlnorm.XX.Item.Component.Resource.mods.name8.namePart1.text                           | Array                           |       98365 |   6.06294267062870595453 |
| nldidlnorm.XX.Item.Component.Resource.mods.name1.role.roleTerm1                           | Object                          |       96799 |   5.96641882350620722519 |
| nldidlnorm.XX.Item.Component.Resource.mods.name1.role.roleTerm1._attrs                    | Array                           |       96799 |   5.96641882350620722519 |
| nldidlnorm.XX.Item.Component.Resource.mods.name1.role.roleTerm1.text                      | Array                           |       96799 |   5.96641882350620722519 |
| nldidlnorm.XX.Item.Component.Resource.mods.name4.nameIdentifier                           | Object                          |       94621 |   5.83217301314043368166 |
| nldidlnorm.XX.Item.Component.Resource.mods.name4.nameIdentifier._attrs                    | Array                           |       94621 |   5.83217301314043368166 |
| nldidlnorm.XX.Item.Component.Resource.mods.name4.nameIdentifier.text                      | Array                           |       94621 |   5.83217301314043368166 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject1.topic2                                | Array                           |       92512 |   5.70218016921875481273 |
| nldidlnorm.XX.Item.Component.Resource.mods.name2.displayForm                              | Array                           |       90609 |   5.58488458743451854360 |
| nldidlnorm.XX.Item.Component.Resource.mods.name0                                          | Object                          |       89435 |   5.51252252069006498658 |
| nldidlnorm.XX.Item.Component.Resource.mods.name0.role                                     | Object                          |       89435 |   5.51252252069006498658 |
| nldidlnorm.XX.Item.Component.Resource.mods.name0.role.roleTerm                            | Object                          |       89435 |   5.51252252069006498658 |
| nldidlnorm.XX.Item.Component.Resource.mods.name0.role.roleTerm._attrs                     | Array                           |       89435 |   5.51252252069006498658 |
| nldidlnorm.XX.Item.Component.Resource.mods.name0.role.roleTerm.text                       | Array                           |       89435 |   5.51252252069006498658 |
| nldidlnorm.XX.Item.Component.Resource.mods.name0.namePart                                 | Object (57411),Array (31947)    |       89358 |   5.50777645668723536687 |
| nldidlnorm.XX.Item.Component.Resource.mods.name0._attrs                                   | Array                           |       89123 |   5.49329171589937637066 |
| nldidlnorm.XX.Item.Component.Resource.mods.name2.role2                                    | Object                          |       82472 |   5.08334273300554695396 |
| nldidlnorm.XX.Item.Component.Resource.mods.name2.role2.roleTerm                           | Object                          |       82472 |   5.08334273300554695396 |
| nldidlnorm.XX.Item.Component.Resource.mods.name2.role2.roleTerm._attrs                    | Array                           |       82472 |   5.08334273300554695396 |
| nldidlnorm.XX.Item.Component.Resource.mods.name2.role2.roleTerm.text                      | Array                           |       82472 |   5.08334273300554695396 |
| Keywords.XX.topic0                                                                        | Array                           |       81378 |   5.01591164184844995333 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject.topic0                                 | Array                           |       81378 |   5.01591164184844995333 |
| nldidlnorm.XX.Item.Component.Resource.mods.classification                                 | Object                          |       80495 |   4.96148599880300533727 |
| nldidlnorm.XX.Item.Component.Resource.mods.classification._attrs                          | Array                           |       80495 |   4.96148599880300533727 |
| nldidlnorm.XX.Item.Component.Resource.mods.classification.text                            | Array                           |       80495 |   4.96148599880300533727 |
| nldidlnorm.XX.Item.Component.Resource.mods.name7.role1                                    | Object                          |       75972 |   4.68270096653285250454 |
| nldidlnorm.XX.Item.Component.Resource.mods.name7.role1.roleTerm                           | Object                          |       75972 |   4.68270096653285250454 |
| nldidlnorm.XX.Item.Component.Resource.mods.name7.role1.roleTerm._attrs                    | Array                           |       75972 |   4.68270096653285250454 |
| nldidlnorm.XX.Item.Component.Resource.mods.name7.role1.roleTerm.text                      | Array                           |       75972 |   4.68270096653285250454 |
| nldidlnorm.XX.Item.Component.Resource.mods.name3.affiliation                              | Array                           |       75904 |   4.67850963728359925398 |
| nldidlnorm.XX.Item.Component.Resource.mods.name9.namePart._attrs                          | Array                           |       74435 |   4.58796459806077017163 |
| nldidlnorm.XX.Item.Component.Resource.mods.name9.namePart.text                            | Array                           |       74435 |   4.58796459806077017163 |
| nldidlnorm.XX.Item.Component.Resource.mods.name9.namePart1                                | Object (73999),Array (59)       |       74058 |   4.56472737560535435364 |
| nldidlnorm.XX.Item.Component.Resource.mods.name9.namePart1._attrs                         | Array                           |       73999 |   4.56109078110967924857 |
| nldidlnorm.XX.Item.Component.Resource.mods.name9.namePart1.text                           | Array                           |       73999 |   4.56109078110967924857 |
| nldidlnorm.XX.Item.Component.Resource.mods.extension1                                     | Object                          |       72263 |   4.45408861086404872509 |
| nldidlnorm.XX.Item.Component.Resource.mods.extension1.rights                              | Object                          |       72263 |   4.45408861086404872509 |
| nldidlnorm.XX.Item.Component.Resource.mods.extension1.rights._attrs                       | Array                           |       72263 |   4.45408861086404872509 |
| nldidlnorm.XX.Item.Component.Resource.mods.extension1.rights.description                  | Array                           |       72263 |   4.45408861086404872509 |
| nldidlnorm.XX.Item.Component.Resource.mods.name2.role.roleTerm1                           | Object                          |       69252 |   4.26849901719492841323 |
| nldidlnorm.XX.Item.Component.Resource.mods.name2.role.roleTerm1._attrs                    | Array                           |       69252 |   4.26849901719492841323 |
| nldidlnorm.XX.Item.Component.Resource.mods.name2.role.roleTerm1.text                      | Array                           |       69252 |   4.26849901719492841323 |
| nldidlnorm.XX.Item3                                                                       | Object                          |       66399 |   4.09264810031083658970 |
| nldidlnorm.XX.Item3.Component                                                             | Object                          |       66399 |   4.09264810031083658970 |
| nldidlnorm.XX.Item3.Component.Resource                                                    | Array                           |       66399 |   4.09264810031083658970 |
| nldidlnorm.XX.Item3.Descriptor                                                            | Object                          |       66399 |   4.09264810031083658970 |
| nldidlnorm.XX.Item3.Descriptor.Statement                                                  | Object                          |       66399 |   4.09264810031083658970 |
| nldidlnorm.XX.Item3.Descriptor.Statement._attrs                                           | Array                           |       66399 |   4.09264810031083658970 |
| nldidlnorm.XX.Item3.Descriptor.Statement.type                                             | Array                           |       66399 |   4.09264810031083658970 |
| nldidlnorm.XX.Item1.Descriptor3.Statement.description                                     | Array                           |       65614 |   4.04426290235990304467 |
| nldidlnorm.XX.Item.Component.Resource.mods.name3.role2                                    | Object                          |       65564 |   4.04118104261780519693 |
| nldidlnorm.XX.Item.Component.Resource.mods.name3.role2.roleTerm                           | Object                          |       65564 |   4.04118104261780519693 |
| nldidlnorm.XX.Item.Component.Resource.mods.name3.role2.roleTerm._attrs                    | Array                           |       65564 |   4.04118104261780519693 |
| nldidlnorm.XX.Item.Component.Resource.mods.name3.role2.roleTerm.text                      | Array                           |       65564 |   4.04118104261780519693 |
| nldidlnorm.XX.Item2.Descriptor1                                                           | Object                          |       64708 |   3.98841960383309368510 |
| nldidlnorm.XX.Item2.Descriptor1.Statement                                                 | Object                          |       64708 |   3.98841960383309368510 |
| nldidlnorm.XX.Item2.Descriptor1.Statement._attrs                                          | Array                           |       64708 |   3.98841960383309368510 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject1.topic3                                | Array                           |       63865 |   3.93645944858132734723 |
| nldidlnorm.XX.Item.Component.Resource.mods.name5.nameIdentifier                           | Object                          |       60976 |   3.75838959268292516214 |
| nldidlnorm.XX.Item.Component.Resource.mods.name5.nameIdentifier._attrs                    | Array                           |       60976 |   3.75838959268292516214 |
| nldidlnorm.XX.Item.Component.Resource.mods.name5.nameIdentifier.text                      | Array                           |       60976 |   3.75838959268292516214 |
| nldidlnorm.XX.Item.Component.Resource.mods.name3.displayForm                              | Array                           |       59300 |   3.65508565412781205595 |
| nldidlnorm.XX.Item.Component.Resource.mods.name8.role1                                    | Object                          |       57685 |   3.55554158445805823519 |
| nldidlnorm.XX.Item.Component.Resource.mods.name8.role1.roleTerm                           | Object                          |       57685 |   3.55554158445805823519 |
| nldidlnorm.XX.Item.Component.Resource.mods.name8.role1.roleTerm._attrs                    | Array                           |       57685 |   3.55554158445805823519 |
| nldidlnorm.XX.Item.Component.Resource.mods.name8.role1.roleTerm.text                      | Array                           |       57685 |   3.55554158445805823519 |
| nldidlnorm.XX.Item.Component.Resource.mods.name0.namePart._attrs                          | Array                           |       57411 |   3.53865299307136282891 |
| nldidlnorm.XX.Item.Component.Resource.mods.name0.namePart.text                            | Array                           |       57411 |   3.53865299307136282891 |
| nldidlnorm.XX.Item.Component.Resource.mods.name0.namePart1                                | Object (56861),Array (50)       |       56911 |   3.50783439565038657193 |
| nldidlnorm.XX.Item.Component.Resource.mods.name0.namePart1._attrs                         | Array                           |       56861 |   3.50475253590828872419 |
| nldidlnorm.XX.Item.Component.Resource.mods.name0.namePart1.text                           | Array                           |       56861 |   3.50475253590828872419 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject1.topic4                                | Array                           |       55544 |   3.42357635030143692134 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.comment                             | Array                           |       54957 |   3.38739531692921014283 |
| nldidlnorm.XX.Item.Component.Resource.mods.extension2                                     | Object                          |       54280 |   3.34566693602120812656 |
| nldidlnorm.XX.Item.Component.Resource.mods.extension2.rights                              | Object                          |       54280 |   3.34566693602120812656 |
| nldidlnorm.XX.Item.Component.Resource.mods.extension2.rights._attrs                       | Array                           |       54280 |   3.34566693602120812656 |
| nldidlnorm.XX.Item.Component.Resource.mods.extension2.rights.description                  | Array                           |       54280 |   3.34566693602120812656 |
| nldidlnorm.XX.Item.Component.Resource.mods.name4.affiliation                              | Array                           |       53574 |   3.30215107646278926978 |
| nldidlnorm.XX.Item.Component.Resource.mods.language.languageTerm1                         | Object                          |       52471 |   3.23416525055211501183 |
| nldidlnorm.XX.Item.Component.Resource.mods.language.languageTerm1._attrs                  | Array                           |       52471 |   3.23416525055211501183 |
| nldidlnorm.XX.Item.Component.Resource.mods.language.languageTerm1.text                    | Array                           |       52471 |   3.23416525055211501183 |
| nldidlnorm.XX.Item.Component.Resource.mods.name4.role2                                    | Object                          |       51309 |   3.16254283014576564881 |
| nldidlnorm.XX.Item.Component.Resource.mods.name4.role2.roleTerm                           | Object                          |       51309 |   3.16254283014576564881 |
| nldidlnorm.XX.Item.Component.Resource.mods.name4.role2.roleTerm._attrs                    | Array                           |       51309 |   3.16254283014576564881 |
| nldidlnorm.XX.Item.Component.Resource.mods.name4.role2.roleTerm.text                      | Array                           |       51309 |   3.16254283014576564881 |
| Journal.XX.relatedItem1                                                                   | Object                          |       50637 |   3.12112263521197341731 |
| Journal.XX.relatedItem1._attrs                                                            | Array                           |       50637 |   3.12112263521197341731 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem1                                   | Object                          |       50637 |   3.12112263521197341731 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem1._attrs                            | Array                           |       50637 |   3.12112263521197341731 |
| nldidlnorm.XX.Item2.Descriptor2                                                           | Object                          |       47694 |   2.93972437079210591193 |
| nldidlnorm.XX.Item2.Descriptor2.Statement                                                 | Object                          |       47694 |   2.93972437079210591193 |
| nldidlnorm.XX.Item2.Descriptor2.Statement._attrs                                          | Array                           |       47694 |   2.93972437079210591193 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject1.topic5                                | Array                           |       46907 |   2.89121589845148863063 |
| nldidlnorm.XX.Item.Component.Resource.mods.name3.role.roleTerm1                           | Object                          |       44489 |   2.74217716132364630610 |
| nldidlnorm.XX.Item.Component.Resource.mods.name3.role.roleTerm1._attrs                    | Array                           |       44489 |   2.74217716132364630610 |
| nldidlnorm.XX.Item.Component.Resource.mods.name3.role.roleTerm1.text                      | Array                           |       44489 |   2.74217716132364630610 |
| nldidlnorm.XX.Item.Component.Resource.mods.name1.affiliation1                             | Array                           |       43935 |   2.70803015538120428474 |
| nldidlnorm.XX.Item.Component.Resource.mods.name9.role1                                    | Object                          |       43415 |   2.67597881406338888866 |
| nldidlnorm.XX.Item.Component.Resource.mods.name9.role1.roleTerm                           | Object                          |       43415 |   2.67597881406338888866 |
| nldidlnorm.XX.Item.Component.Resource.mods.name9.role1.roleTerm._attrs                    | Array                           |       43415 |   2.67597881406338888866 |
| nldidlnorm.XX.Item.Component.Resource.mods.name9.role1.roleTerm.text                      | Array                           |       43415 |   2.67597881406338888866 |
| nldidlnorm.XX.Item.Component.Resource.mods.name4.displayForm                              | Array                           |       43405 |   2.67536244211496931911 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.dateOther                           | Object                          |       42934 |   2.64633132334440945854 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.dateOther._attrs                    | Array                           |       42934 |   2.64633132334440945854 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.dateOther.text                      | Array                           |       42934 |   2.64633132334440945854 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject2                                       | Object                          |       42413 |   2.61421834483175219432 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject2.topic                                 | Array (29957),Object (12456)    |       42413 |   2.61421834483175219432 |
| nldidlnorm.XX.Item.Component.Resource.mods.name5.role2                                    | Object                          |       41908 |   2.58309156143656570848 |
| nldidlnorm.XX.Item.Component.Resource.mods.name5.role2.roleTerm                           | Object                          |       41908 |   2.58309156143656570848 |
| nldidlnorm.XX.Item.Component.Resource.mods.name5.role2.roleTerm._attrs                    | Array                           |       41908 |   2.58309156143656570848 |
| nldidlnorm.XX.Item.Component.Resource.mods.name5.role2.roleTerm.text                      | Array                           |       41908 |   2.58309156143656570848 |
| Journal.XX.relatedItem1.part                                                              | Object                          |       41580 |   2.56287456152840498191 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem1.part                              | Object                          |       41580 |   2.56287456152840498191 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject1.topic6                                | Array                           |       39671 |   2.44520915657511705987 |
| nldidlnorm.XX.Item.Component.Resource.mods.name.affiliation2                              | Array                           |       39670 |   2.44514751938027519174 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.dateCreated                         | Object (39288),Array (147)      |       39435 |   2.43066277859241619552 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.dateCreated._attrs                  | Array                           |       39288 |   2.42160211095064914488 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.dateCreated.text                    | Array                           |       39288 |   2.42160211095064914488 |
| nldidlnorm.XX.Item.Component.Resource.mods.name6.nameIdentifier                           | Object                          |       39124 |   2.41149361099656855956 |
| nldidlnorm.XX.Item.Component.Resource.mods.name6.nameIdentifier._attrs                    | Array                           |       39124 |   2.41149361099656855956 |
| nldidlnorm.XX.Item.Component.Resource.mods.name6.nameIdentifier.text                      | Array                           |       39124 |   2.41149361099656855956 |
| nldidlnorm.XX.Item.Component.Resource.mods.name5.affiliation                              | Array                           |       38393 |   2.36643682156710122300 |
| nldidlnorm.XX.Item.Component.Resource.mods.genre._attrs                                   | Array                           |       37912 |   2.33678933084812179288 |
| nldidlnorm.XX.Item.Component.Resource.mods.genre.text                                     | Array                           |       37912 |   2.33678933084812179288 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.issuance                            | Array                           |       37912 |   2.33678933084812179288 |
| nldidlnorm.XX.Item.Component.Resource.mods.typeOfResource._attrs                          | Array                           |       37912 |   2.33678933084812179288 |
| nldidlnorm.XX.Item.Component.Resource.mods.typeOfResource.text                            | Array                           |       37912 |   2.33678933084812179288 |
| Keywords.XX.topic._attrs                                                                  | Array                           |       37857 |   2.33339928513181416037 |
| Keywords.XX.topic.text                                                                    | Array                           |       37857 |   2.33339928513181416037 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject.topic._attrs                           | Array                           |       37857 |   2.33339928513181416037 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject.topic.text                             | Array                           |       37857 |   2.33339928513181416037 |
| nldidlnorm.XX.Item.Component.Resource.mods.name6.role2                                    | Object                          |       35926 |   2.21437786189200291886 |
| nldidlnorm.XX.Item.Component.Resource.mods.name6.role2.roleTerm                           | Object                          |       35926 |   2.21437786189200291886 |
| nldidlnorm.XX.Item.Component.Resource.mods.name6.role2.roleTerm._attrs                    | Array                           |       35926 |   2.21437786189200291886 |
| nldidlnorm.XX.Item.Component.Resource.mods.name6.role2.roleTerm.text                      | Array                           |       35926 |   2.21437786189200291886 |
| nldidlnorm.XX.Item2.Descriptor1.Statement.Identifier                                      | Array                           |       34267 |   2.11212175564920290327 |
| nldidlnorm.XX.Item2.Descriptor2.Statement.accessRights                                    | Array                           |       34267 |   2.11212175564920290327 |
| nldidlnorm.XX.Item.Component.Resource.mods.extension3                                     | Object                          |       34215 |   2.10891662151742131925 |
| nldidlnorm.XX.Item.Component.Resource.mods.extension3.rights                              | Object                          |       34215 |   2.10891662151742131925 |
| nldidlnorm.XX.Item.Component.Resource.mods.extension3.rights._attrs                       | Array                           |       34215 |   2.10891662151742131925 |
| nldidlnorm.XX.Item.Component.Resource.mods.extension3.rights.description                  | Array                           |       34215 |   2.10891662151742131925 |
| nldidlnorm.XX.Item.Component.Resource.mods.identifier7                                    | Object                          |       33546 |   2.06768133816815469217 |
| nldidlnorm.XX.Item.Component.Resource.mods.identifier7._attrs                             | Array                           |       33546 |   2.06768133816815469217 |
| nldidlnorm.XX.Item.Component.Resource.mods.identifier7.text                               | Array                           |       33546 |   2.06768133816815469217 |
| nldidlnorm.XX.Item.Component.Resource.mods.name0.role1                                    | Object                          |       33285 |   2.05159403031440534804 |
| nldidlnorm.XX.Item.Component.Resource.mods.name0.role1.roleTerm                           | Object                          |       33285 |   2.05159403031440534804 |
| nldidlnorm.XX.Item.Component.Resource.mods.name0.role1.roleTerm._attrs                    | Array                           |       33285 |   2.05159403031440534804 |
| nldidlnorm.XX.Item.Component.Resource.mods.name0.role1.roleTerm.text                      | Array                           |       33285 |   2.05159403031440534804 |
| nldidlnorm.XX.Item.Component.Resource.mods.name4.role.roleTerm1                           | Object                          |       32715 |   2.01646082925449210421 |
| nldidlnorm.XX.Item.Component.Resource.mods.name4.role.roleTerm1._attrs                    | Array                           |       32715 |   2.01646082925449210421 |
| nldidlnorm.XX.Item.Component.Resource.mods.name4.role.roleTerm1.text                      | Array                           |       32715 |   2.01646082925449210421 |
| nldidlnorm.XX.Item.Component.Resource.mods.titleInfo1                                     | Object                          |       32468 |   2.00123644212852935809 |
| nldidlnorm.XX.Item.Component.Resource.mods.titleInfo1.title                               | Array                           |       32468 |   2.00123644212852935809 |
| Journal.XX.relatedItem1.titleInfo                                                         | Object                          |       32397 |   1.99686020129475094720 |
| Journal.XX.relatedItem1.titleInfo.title                                                   | Array                           |       32397 |   1.99686020129475094720 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem1.titleInfo                         | Object                          |       32397 |   1.99686020129475094720 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem1.titleInfo.title                   | Array                           |       32397 |   1.99686020129475094720 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject1.topic7                                | Array                           |       32330 |   1.99273050924034000886 |
| nldidlnorm.XX.Item4                                                                       | Object                          |       32314 |   1.99174431412286878640 |
| nldidlnorm.XX.Item4.Component                                                             | Object                          |       32314 |   1.99174431412286878640 |
| nldidlnorm.XX.Item4.Component.Resource                                                    | Array                           |       32314 |   1.99174431412286878640 |
| nldidlnorm.XX.Item4.Descriptor                                                            | Object                          |       32314 |   1.99174431412286878640 |
| nldidlnorm.XX.Item4.Descriptor.Statement                                                  | Object                          |       32314 |   1.99174431412286878640 |
| nldidlnorm.XX.Item4.Descriptor.Statement._attrs                                           | Array                           |       32314 |   1.99174431412286878640 |
| nldidlnorm.XX.Item4.Descriptor.Statement.type                                             | Array                           |       32314 |   1.99174431412286878640 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject3                                       | Object                          |       31112 |   1.91765640592284136723 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject3.topic                                 | Array (22472),Object (8640)     |       31112 |   1.91765640592284136723 |
| nldidlnorm.XX.Item.Component.Resource.mods.name5.displayForm                              | Array                           |       31049 |   1.91377326264779834553 |
| nldidlnorm.XX.Item3.Descriptor1                                                           | Object                          |       30552 |   1.88313957681134769295 |
| nldidlnorm.XX.Item3.Descriptor1.Statement                                                 | Object                          |       30552 |   1.88313957681134769295 |
| nldidlnorm.XX.Item3.Descriptor1.Statement._attrs                                          | Array                           |       30552 |   1.88313957681134769295 |
| nldidlnorm.XX.Item2.Descriptor1.Statement.accessRights                                    | Array                           |       30412 |   1.87451036953347416336 |
| nldidlnorm.XX.Item.Component.Resource.mods.name7.role2                                    | Object                          |       29748 |   1.83358327215841754310 |
| nldidlnorm.XX.Item.Component.Resource.mods.name7.role2.roleTerm                           | Object                          |       29748 |   1.83358327215841754310 |
| nldidlnorm.XX.Item.Component.Resource.mods.name7.role2.roleTerm._attrs                    | Array                           |       29748 |   1.83358327215841754310 |
| nldidlnorm.XX.Item.Component.Resource.mods.name7.role2.roleTerm.text                      | Array                           |       29748 |   1.83358327215841754310 |
| nldidlnorm.XX.Item.Component.Resource.mods.name2.affiliation1                             | Array                           |       29155 |   1.79703241561713933372 |
| nldidlnorm.XX.Item.Component.Resource.mods.name6.affiliation                              | Array                           |       27320 |   1.68392816308215564902 |
| nldidlnorm.XX.Item5                                                                       | Object                          |       27057 |   1.66771758083872190248 |
| nldidlnorm.XX.Item5.Component                                                             | Object                          |       27057 |   1.66771758083872190248 |
| nldidlnorm.XX.Item5.Component.Resource                                                    | Array                           |       27057 |   1.66771758083872190248 |
| nldidlnorm.XX.Item5.Descriptor                                                            | Object                          |       27057 |   1.66771758083872190248 |
| nldidlnorm.XX.Item5.Descriptor.Statement                                                  | Object                          |       27057 |   1.66771758083872190248 |
| nldidlnorm.XX.Item5.Descriptor.Statement._attrs                                           | Array                           |       27057 |   1.66771758083872190248 |
| nldidlnorm.XX.Item5.Descriptor.Statement.type                                             | Array                           |       27057 |   1.66771758083872190248 |
| nldidlnorm.XX.Item.Component.Resource.mods.name1.role3                                    | Object                          |       26929 |   1.65982801989895190076 |
| nldidlnorm.XX.Item.Component.Resource.mods.name1.role3.roleTerm                           | Object                          |       26929 |   1.65982801989895190076 |
| nldidlnorm.XX.Item.Component.Resource.mods.name1.role3.roleTerm._attrs                    | Array                           |       26929 |   1.65982801989895190076 |
| nldidlnorm.XX.Item.Component.Resource.mods.name1.role3.roleTerm.text                      | Array                           |       26929 |   1.65982801989895190076 |
| Journal.XX.relatedItem1.part.text                                                         | Object                          |       26609 |   1.64010411754952700747 |
| Journal.XX.relatedItem1.part.text._attrs                                                  | Array                           |       26609 |   1.64010411754952700747 |
| Journal.XX.relatedItem1.part.text.text                                                    | Array                           |       26609 |   1.64010411754952700747 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem1.part.text                         | Object                          |       26609 |   1.64010411754952700747 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem1.part.text._attrs                  | Array                           |       26609 |   1.64010411754952700747 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem1.part.text.text                    | Array                           |       26609 |   1.64010411754952700747 |
| nldidlnorm.XX.Item.Component.Resource.mods.name7.nameIdentifier                           | Object                          |       25878 |   1.59504732812005944886 |
| nldidlnorm.XX.Item.Component.Resource.mods.name7.nameIdentifier._attrs                    | Array                           |       25878 |   1.59504732812005944886 |
| nldidlnorm.XX.Item.Component.Resource.mods.name7.nameIdentifier.text                      | Array                           |       25878 |   1.59504732812005944886 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject1.topic8                                | Array                           |       25436 |   1.56780368799991620676 |
| nldidlnorm.XX.Item4.Descriptor1                                                           | Object                          |       25128 |   1.54881943198859461930 |
| nldidlnorm.XX.Item4.Descriptor1.Statement                                                 | Object                          |       25128 |   1.54881943198859461930 |
| nldidlnorm.XX.Item4.Descriptor1.Statement._attrs                                          | Array                           |       25128 |   1.54881943198859461930 |
| nldidlnorm.XX.Item.Component.Resource.mods.name8.role2                                    | Object                          |       24631 |   1.51818574615214396673 |
| nldidlnorm.XX.Item.Component.Resource.mods.name8.role2.roleTerm                           | Object                          |       24631 |   1.51818574615214396673 |
| nldidlnorm.XX.Item.Component.Resource.mods.name8.role2.roleTerm._attrs                    | Array                           |       24631 |   1.51818574615214396673 |
| nldidlnorm.XX.Item.Component.Resource.mods.name8.role2.roleTerm.text                      | Array                           |       24631 |   1.51818574615214396673 |
| nldidlnorm.XX.Item.Component.Resource.mods.name5.role.roleTerm1                           | Object                          |       24020 |   1.48052542010371079861 |
| nldidlnorm.XX.Item.Component.Resource.mods.name5.role.roleTerm1._attrs                    | Array                           |       24020 |   1.48052542010371079861 |
| nldidlnorm.XX.Item.Component.Resource.mods.name5.role.roleTerm1.text                      | Array                           |       24020 |   1.48052542010371079861 |
| nldidlnorm.XX.Item.Component.Resource.mods.name6.displayForm                              | Array                           |       23663 |   1.45852094154513345359 |
| nldidlnorm.XX.Item3.Descriptor2                                                           | Object                          |       22816 |   1.40631423751399942113 |
| nldidlnorm.XX.Item3.Descriptor2.Statement                                                 | Object                          |       22816 |   1.40631423751399942113 |
| nldidlnorm.XX.Item3.Descriptor2.Statement._attrs                                          | Array                           |       22816 |   1.40631423751399942113 |
| Journal.XX.relatedItem.originInfo.place                                                   | Object                          |       21942 |   1.35244332922213250647 |
| Journal.XX.relatedItem.originInfo.place.placeTerm                                         | Object (15354),Array (6588)     |       21942 |   1.35244332922213250647 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.originInfo.place                   | Object                          |       21942 |   1.35244332922213250647 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.originInfo.place.placeTerm         | Object (15354),Array (6588)     |       21942 |   1.35244332922213250647 |
| Journal.XX.relatedItem.part.text                                                          | Object                          |       21798 |   1.34356757316489128229 |
| Journal.XX.relatedItem.part.text._attrs                                                   | Array                           |       21798 |   1.34356757316489128229 |
| Journal.XX.relatedItem.part.text.text                                                     | Array                           |       21798 |   1.34356757316489128229 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.part.text                          | Object                          |       21798 |   1.34356757316489128229 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.part.text._attrs                   | Array                           |       21798 |   1.34356757316489128229 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.part.text.text                     | Array                           |       21798 |   1.34356757316489128229 |
| nldidlnorm.XX.Item6                                                                       | Object                          |       21490 |   1.32458331715356969482 |
| nldidlnorm.XX.Item6.Component                                                             | Object                          |       21490 |   1.32458331715356969482 |
| nldidlnorm.XX.Item6.Component.Resource                                                    | Array                           |       21490 |   1.32458331715356969482 |
| nldidlnorm.XX.Item6.Descriptor                                                            | Object                          |       21490 |   1.32458331715356969482 |
| nldidlnorm.XX.Item6.Descriptor.Statement                                                  | Object                          |       21490 |   1.32458331715356969482 |
| nldidlnorm.XX.Item6.Descriptor.Statement._attrs                                           | Array                           |       21490 |   1.32458331715356969482 |
| nldidlnorm.XX.Item6.Descriptor.Statement.type                                             | Array                           |       21490 |   1.32458331715356969482 |
| nldidlnorm.XX.Item.Component.Resource.mods.titleInfo1._attrs                              | Array                           |       21186 |   1.30584560992161602400 |
| nldidlnorm.XX.Item4.Descriptor2                                                           | Object                          |       20898 |   1.28809409780713357563 |
| nldidlnorm.XX.Item4.Descriptor2.Statement                                                 | Object                          |       20898 |   1.28809409780713357563 |
| nldidlnorm.XX.Item4.Descriptor2.Statement._attrs                                          | Array                           |       20898 |   1.28809409780713357563 |
| nldidlnorm.XX.Item.Component.Resource.mods.name3.affiliation1                             | Array                           |       20361 |   1.25499492417700486691 |
| nldidlnorm.XX.Item.Component.Resource.mods.extension4                                     | Object                          |       19963 |   1.23046332062990737555 |
| nldidlnorm.XX.Item.Component.Resource.mods.extension4.rights                              | Object                          |       19963 |   1.23046332062990737555 |
| nldidlnorm.XX.Item.Component.Resource.mods.extension4.rights._attrs                       | Array                           |       19963 |   1.23046332062990737555 |
| nldidlnorm.XX.Item.Component.Resource.mods.extension4.rights.description                  | Array                           |       19963 |   1.23046332062990737555 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject4                                       | Object                          |       19907 |   1.22701163771875809694 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject4.topic                                 | Array (15217),Object (4690)     |       19907 |   1.22701163771875809694 |
| nldidlnorm.XX.Item.Component.Resource.mods.name7.affiliation                              | Array                           |       19855 |   1.22380650358697651292 |
| nldidlnorm.XX.Item.Component.Resource.mods.name9.role2                                    | Object                          |       19846 |   1.22325176883339903355 |
| nldidlnorm.XX.Item.Component.Resource.mods.name9.role2.roleTerm                           | Object                          |       19846 |   1.22325176883339903355 |
| nldidlnorm.XX.Item.Component.Resource.mods.name9.role2.roleTerm._attrs                    | Array                           |       19846 |   1.22325176883339903355 |
| nldidlnorm.XX.Item.Component.Resource.mods.name9.role2.roleTerm.text                      | Array                           |       19846 |   1.22325176883339903355 |
| nldidlnorm.XX.Item5.Descriptor1                                                           | Object                          |       19563 |   1.20580844269312636996 |
| nldidlnorm.XX.Item5.Descriptor1.Statement                                                 | Object                          |       19563 |   1.20580844269312636996 |
| nldidlnorm.XX.Item5.Descriptor1.Statement._attrs                                          | Array                           |       19563 |   1.20580844269312636996 |
| nldidlnorm.XX.Item7                                                                       | Object                          |       19385 |   1.19483702201125874254 |
| nldidlnorm.XX.Item7.Component                                                             | Object                          |       19385 |   1.19483702201125874254 |
| nldidlnorm.XX.Item7.Component.Resource                                                    | Array                           |       19385 |   1.19483702201125874254 |
| nldidlnorm.XX.Item7.Descriptor                                                            | Object                          |       19385 |   1.19483702201125874254 |
| nldidlnorm.XX.Item7.Descriptor.Statement                                                  | Object                          |       19385 |   1.19483702201125874254 |
| nldidlnorm.XX.Item7.Descriptor.Statement._attrs                                           | Array                           |       19385 |   1.19483702201125874254 |
| nldidlnorm.XX.Item7.Descriptor.Statement.type                                             | Array                           |       19385 |   1.19483702201125874254 |
| nldidlnorm.XX.Item.Component.Resource.mods.identifier8                                    | Object                          |       19259 |   1.18707073546117247709 |
| nldidlnorm.XX.Item.Component.Resource.mods.identifier8._attrs                             | Array                           |       19259 |   1.18707073546117247709 |
| nldidlnorm.XX.Item.Component.Resource.mods.identifier8.text                               | Array                           |       19259 |   1.18707073546117247709 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject1.topic9                                | Array                           |       19046 |   1.17394201295983657829 |
| nldidlnorm.XX.Item.Component.Resource.mods.name6.role.roleTerm1                           | Object                          |       18917 |   1.16599081482522470843 |
| nldidlnorm.XX.Item.Component.Resource.mods.name6.role.roleTerm1._attrs                    | Array                           |       18917 |   1.16599081482522470843 |
| nldidlnorm.XX.Item.Component.Resource.mods.name6.role.roleTerm1.text                      | Array                           |       18917 |   1.16599081482522470843 |
| nldidlnorm.XX.Item.Component.Resource.mods.name7.displayForm                              | Array                           |       18388 |   1.13338473875383161094 |
| nldidlnorm.XX.Item.Component.Resource.mods.name8.nameIdentifier                           | Object                          |       17658 |   1.08838958651920592047 |
| nldidlnorm.XX.Item.Component.Resource.mods.name8.nameIdentifier._attrs                    | Array                           |       17658 |   1.08838958651920592047 |
| nldidlnorm.XX.Item.Component.Resource.mods.name8.nameIdentifier.text                      | Array                           |       17658 |   1.08838958651920592047 |
| nldidlnorm.XX.Item6.Descriptor1                                                           | Object                          |       17562 |   1.08247241581437836366 |
| nldidlnorm.XX.Item6.Descriptor1.Statement                                                 | Object                          |       17562 |   1.08247241581437836366 |
| nldidlnorm.XX.Item6.Descriptor1.Statement._attrs                                          | Array                           |       17562 |   1.08247241581437836366 |
| nldidlnorm.XX.Item8                                                                       | Object                          |       17531 |   1.08056166277427778688 |
| nldidlnorm.XX.Item8.Component                                                             | Object                          |       17531 |   1.08056166277427778688 |
| nldidlnorm.XX.Item8.Component.Resource                                                    | Array                           |       17531 |   1.08056166277427778688 |
| nldidlnorm.XX.Item8.Descriptor                                                            | Object                          |       17531 |   1.08056166277427778688 |
| nldidlnorm.XX.Item8.Descriptor.Statement                                                  | Object                          |       17531 |   1.08056166277427778688 |
| nldidlnorm.XX.Item8.Descriptor.Statement._attrs                                           | Array                           |       17531 |   1.08056166277427778688 |
| nldidlnorm.XX.Item8.Descriptor.Statement.type                                             | Array                           |       17531 |   1.08056166277427778688 |
| nldidlnorm.XX.Item3.Descriptor1.Statement.Identifier                                      | Array                           |       17483 |   1.07760307742186411950 |
| nldidlnorm.XX.Item3.Descriptor2.Statement.accessRights                                    | Array                           |       17483 |   1.07760307742186411950 |
| nldidlnorm.XX.Item5.Descriptor2                                                           | Object                          |       17067 |   1.05196200436761166941 |
| nldidlnorm.XX.Item5.Descriptor2.Statement                                                 | Object                          |       17067 |   1.05196200436761166941 |
| nldidlnorm.XX.Item5.Descriptor2.Statement._attrs                                          | Array                           |       17067 |   1.05196200436761166941 |
| nldidlnorm.XX.Item9                                                                       | Object                          |       17004 |   1.04807886109256864771 |
| nldidlnorm.XX.Item9.Component                                                             | Object                          |       17004 |   1.04807886109256864771 |
| nldidlnorm.XX.Item9.Component.Resource                                                    | Array                           |       17004 |   1.04807886109256864771 |
| nldidlnorm.XX.Item9.Descriptor                                                            | Object                          |       17004 |   1.04807886109256864771 |
| nldidlnorm.XX.Item9.Descriptor.Statement                                                  | Object                          |       17004 |   1.04807886109256864771 |
| nldidlnorm.XX.Item9.Descriptor.Statement._attrs                                           | Array                           |       17004 |   1.04807886109256864771 |
| nldidlnorm.XX.Item9.Descriptor.Statement.type                                             | Array                           |       17004 |   1.04807886109256864771 |
| nldidlnorm.XX.Item0                                                                       | Object                          |       16563 |   1.02089685816726727374 |
| nldidlnorm.XX.Item0.Component                                                             | Object                          |       16563 |   1.02089685816726727374 |
| nldidlnorm.XX.Item0.Component.Resource                                                    | Array                           |       16563 |   1.02089685816726727374 |
| nldidlnorm.XX.Item0.Descriptor                                                            | Object                          |       16563 |   1.02089685816726727374 |
| nldidlnorm.XX.Item0.Descriptor.Statement                                                  | Object                          |       16563 |   1.02089685816726727374 |
| nldidlnorm.XX.Item0.Descriptor.Statement._attrs                                           | Array                           |       16563 |   1.02089685816726727374 |
| nldidlnorm.XX.Item0.Descriptor.Statement.type                                             | Array                           |       16563 |   1.02089685816726727374 |
| nldidlnorm.XX.Item4.Descriptor1.Statement.Identifier                                      | Array                           |       16091 |   0.99180410220186554504 |
| nldidlnorm.XX.Item4.Descriptor2.Statement.accessRights                                    | Array                           |       16091 |   0.99180410220186554504 |
| nldidlnorm.XX.Item.Component.Resource.mods.name0.role2                                    | Object                          |       16083 |   0.99131100464312993381 |
| nldidlnorm.XX.Item.Component.Resource.mods.name0.role2.roleTerm                           | Object                          |       16083 |   0.99131100464312993381 |
| nldidlnorm.XX.Item.Component.Resource.mods.name0.role2.roleTerm._attrs                    | Array                           |       16083 |   0.99131100464312993381 |
| nldidlnorm.XX.Item.Component.Resource.mods.name0.role2.roleTerm.text                      | Array                           |       16083 |   0.99131100464312993381 |
| nldidlnorm.XX.Item6.Descriptor2                                                           | Object                          |       16050 |   0.98927697721334539871 |
| nldidlnorm.XX.Item6.Descriptor2.Statement                                                 | Object                          |       16050 |   0.98927697721334539871 |
| nldidlnorm.XX.Item6.Descriptor2.Statement._attrs                                          | Array                           |       16050 |   0.98927697721334539871 |
| nldidlnorm.XX.Item7.Descriptor1                                                           | Object                          |       15943 |   0.98268179736525651524 |
| nldidlnorm.XX.Item7.Descriptor1.Statement                                                 | Object                          |       15943 |   0.98268179736525651524 |
| nldidlnorm.XX.Item7.Descriptor1.Statement._attrs                                          | Array                           |       15943 |   0.98268179736525651524 |
| nldidlnorm.XX.Item.Component.Resource.mods.titleInfo2                                     | Object                          |       15863 |   0.97775082177790018090 |
| nldidlnorm.XX.Item.Component.Resource.mods.titleInfo2.title                               | Array                           |       15863 |   0.97775082177790018090 |
| Journal.XX.relatedItem.name                                                               | Object                          |       15697 |   0.96751904743413608134 |
| Journal.XX.relatedItem.name.namePart                                                      | Array (15301),Object (396)      |       15697 |   0.96751904743413608134 |
| Journal.XX.relatedItem.name.role                                                          | Object                          |       15697 |   0.96751904743413608134 |
| Journal.XX.relatedItem.name.role.roleTerm                                                 | Object                          |       15697 |   0.96751904743413608134 |
| Journal.XX.relatedItem.name.role.roleTerm._attrs                                          | Array                           |       15697 |   0.96751904743413608134 |
| Journal.XX.relatedItem.name.role.roleTerm.text                                            | Array                           |       15697 |   0.96751904743413608134 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name                               | Object                          |       15697 |   0.96751904743413608134 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name.namePart                      | Array (15301),Object (396)      |       15697 |   0.96751904743413608134 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name.role                          | Object                          |       15697 |   0.96751904743413608134 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name.role.roleTerm                 | Object                          |       15697 |   0.96751904743413608134 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name.role.roleTerm._attrs          | Array                           |       15697 |   0.96751904743413608134 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name.role.roleTerm.text            | Array                           |       15697 |   0.96751904743413608134 |
| nldidlnorm.XX.Item8.Descriptor1                                                           | Object                          |       15620 |   0.96277298343130568448 |
| nldidlnorm.XX.Item8.Descriptor1.Statement                                                 | Object                          |       15620 |   0.96277298343130568448 |
| nldidlnorm.XX.Item8.Descriptor1.Statement._attrs                                          | Array                           |       15620 |   0.96277298343130568448 |
| Journal.XX.relatedItem.originInfo.place.placeTerm._attrs                                  | Array                           |       15354 |   0.94637748960334611148 |
| Journal.XX.relatedItem.originInfo.place.placeTerm.text                                    | Array                           |       15354 |   0.94637748960334611148 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.originInfo.place.placeTerm._attrs  | Array                           |       15354 |   0.94637748960334611148 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.originInfo.place.placeTerm.text    | Array                           |       15354 |   0.94637748960334611148 |
| nldidlnorm.XX.Item9.Descriptor1                                                           | Object                          |       15264 |   0.94083014206757042963 |
| nldidlnorm.XX.Item9.Descriptor1.Statement                                                 | Object                          |       15264 |   0.94083014206757042963 |
| nldidlnorm.XX.Item9.Descriptor1.Statement._attrs                                          | Array                           |       15264 |   0.94083014206757042963 |
| nldidlnorm.XX.Item.Component.Resource.mods.name7.role.roleTerm1                           | Object                          |       15162 |   0.93454314819369121992 |
| nldidlnorm.XX.Item.Component.Resource.mods.name7.role.roleTerm1._attrs                    | Array                           |       15162 |   0.93454314819369121992 |
| nldidlnorm.XX.Item.Component.Resource.mods.name7.role.roleTerm1.text                      | Array                           |       15162 |   0.93454314819369121992 |
| nldidlnorm.XX.Item.Component.Resource.mods.name4.affiliation1                             | Array                           |       15159 |   0.93435823660916528244 |
| nldidlnorm.XX.Item0.Descriptor1                                                           | Object                          |       15091 |   0.93016690735991247596 |
| nldidlnorm.XX.Item0.Descriptor1.Statement                                                 | Object                          |       15091 |   0.93016690735991247596 |
| nldidlnorm.XX.Item0.Descriptor1.Statement._attrs                                          | Array                           |       15091 |   0.93016690735991247596 |
| Journal.XX.relatedItem1.part.extent                                                       | Object (14970),Array (12)       |       14982 |   0.92344845312213963417 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem1.part.extent                       | Object (14970),Array (12)       |       14982 |   0.92344845312213963417 |
| Journal.XX.relatedItem1.part.extent._attrs                                                | Array                           |       14970 |   0.92270880678403621733 |
| Journal.XX.relatedItem1.part.extent.list                                                  | Array                           |       14970 |   0.92270880678403621733 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem1.part.extent._attrs                | Array                           |       14970 |   0.92270880678403621733 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem1.part.extent.list                  | Array                           |       14970 |   0.92270880678403621733 |
| nldidlnorm.XX.Item7.Descriptor2                                                           | Object                          |       14903 |   0.91857911472962539001 |
| nldidlnorm.XX.Item7.Descriptor2.Statement                                                 | Object                          |       14903 |   0.91857911472962539001 |
| nldidlnorm.XX.Item7.Descriptor2.Statement._attrs                                          | Array                           |       14903 |   0.91857911472962539001 |
| nldidlnorm.XX.Item.Component.Resource.mods.name8.affiliation                              | Array                           |       14897 |   0.91820929156057362608 |
| nldidlnorm.XX.Item.Component.Resource.mods.name1.affiliation2                             | Array                           |       14833 |   0.91426451109068862522 |
| nldidlnorm.XX.Item8.Descriptor2                                                           | Object                          |       14773 |   0.91056627940017142997 |
| nldidlnorm.XX.Item8.Descriptor2.Statement                                                 | Object                          |       14773 |   0.91056627940017142997 |
| nldidlnorm.XX.Item8.Descriptor2.Statement._attrs                                          | Array                           |       14773 |   0.91056627940017142997 |
| nldidlnorm.XX.Item.Component.Resource.mods.abstract._attrs                                | Array                           |       14656 |   0.90335472760366297695 |
| nldidlnorm.XX.Item.Component.Resource.mods.abstract.text                                  | Array                           |       14656 |   0.90335472760366297695 |
| nldidlnorm.XX.Item9.Descriptor2                                                           | Object                          |       14649 |   0.90292326723976934488 |
| nldidlnorm.XX.Item9.Descriptor2.Statement                                                 | Object                          |       14649 |   0.90292326723976934488 |
| nldidlnorm.XX.Item9.Descriptor2.Statement._attrs                                          | Array                           |       14649 |   0.90292326723976934488 |
| nldidlnorm.XX.Item.Component.Resource.mods.name8.displayForm                              | Array                           |       14544 |   0.89645136178136419769 |
| nldidlnorm.XX.Item0.Descriptor2                                                           | Object                          |       14461 |   0.89133547460948214791 |
| nldidlnorm.XX.Item0.Descriptor2.Statement                                                 | Object                          |       14461 |   0.89133547460948214791 |
| nldidlnorm.XX.Item0.Descriptor2.Statement._attrs                                          | Array                           |       14461 |   0.89133547460948214791 |
| Journal.XX.relatedItem.comment                                                            | Array                           |       14460 |   0.89127383741464016875 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.comment                            | Array                           |       14460 |   0.89127383741464016875 |
| Journal.XX.relatedItem.part.detail2                                                       | Object                          |       14346 |   0.88424719720265754219 |
| Journal.XX.relatedItem.part.detail2._attrs                                                | Array                           |       14346 |   0.88424719720265754219 |
| Journal.XX.relatedItem.part.detail2.number                                                | Array                           |       14346 |   0.88424719720265754219 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.part.detail2                       | Object                          |       14346 |   0.88424719720265754219 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.part.detail2._attrs                | Array                           |       14346 |   0.88424719720265754219 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.part.detail2.number                | Array                           |       14346 |   0.88424719720265754219 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject1.topic0                                | Array                           |       13679 |   0.84313518824307487343 |
| nldidlnorm.XX.Item.Component.Resource.mods.name2.role3                                    | Object                          |       13578 |   0.83690983156403764287 |
| nldidlnorm.XX.Item.Component.Resource.mods.name2.role3.roleTerm                           | Object                          |       13578 |   0.83690983156403764287 |
| nldidlnorm.XX.Item.Component.Resource.mods.name2.role3.roleTerm._attrs                    | Array                           |       13578 |   0.83690983156403764287 |
| nldidlnorm.XX.Item.Component.Resource.mods.name2.role3.roleTerm.text                      | Array                           |       13578 |   0.83690983156403764287 |
| nldidlnorm.XX.Item3.Descriptor1.Statement.accessRights                                    | Array                           |       13066 |   0.80535158780495774700 |
| nldidlnorm.XX.Item.Component.Resource.mods.name.namePart2                                 | Object (10152),Array (2792)     |       12944 |   0.79783185003423950921 |
| nldidlnorm.XX.Item5.Descriptor1.Statement.Identifier                                      | Array                           |       12532 |   0.77243732575935486473 |
| nldidlnorm.XX.Item5.Descriptor2.Statement.accessRights                                    | Array                           |       12532 |   0.77243732575935486473 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject2.topic._attrs                          | Array                           |       12456 |   0.76775289895136644702 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject2.topic.text                            | Array                           |       12456 |   0.76775289895136644702 |
| Journal.XX.relatedItem.titleInfo._attrs                                                   | Array                           |       12309 |   0.75869223130959928536 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.titleInfo._attrs                   | Array                           |       12309 |   0.75869223130959928536 |
| nldidlnorm.XX.Item.Component.Resource.mods.name8.role.roleTerm1                           | Object                          |       12249 |   0.75499399961908209011 |
| nldidlnorm.XX.Item.Component.Resource.mods.name8.role.roleTerm1._attrs                    | Array                           |       12249 |   0.75499399961908209011 |
| nldidlnorm.XX.Item.Component.Resource.mods.name8.role.roleTerm1.text                      | Array                           |       12249 |   0.75499399961908209011 |
| nldidlnorm.XX.Item.Component.Resource.mods.name9.nameIdentifier                           | Object                          |       11928 |   0.73520846007481521767 |
| nldidlnorm.XX.Item.Component.Resource.mods.name9.nameIdentifier._attrs                    | Array                           |       11928 |   0.73520846007481521767 |
| nldidlnorm.XX.Item.Component.Resource.mods.name9.nameIdentifier.text                      | Array                           |       11928 |   0.73520846007481521767 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject5                                       | Object                          |       11725 |   0.72269610952189877739 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject5.topic                                 | Array (8868),Object (2857)      |       11725 |   0.72269610952189877739 |
| nldidlnorm.XX.Item.Component.Resource.mods.extension5                                     | Object                          |       11603 |   0.71517637175118053960 |
| nldidlnorm.XX.Item.Component.Resource.mods.extension5.rights                              | Object                          |       11603 |   0.71517637175118053960 |
| nldidlnorm.XX.Item.Component.Resource.mods.extension5.rights._attrs                       | Array                           |       11603 |   0.71517637175118053960 |
| nldidlnorm.XX.Item.Component.Resource.mods.extension5.rights.description                  | Array                           |       11603 |   0.71517637175118053960 |
| nldidlnorm.XX.Item.Component.Resource.mods.name9.displayForm                              | Array                           |       11575 |   0.71345053029560578928 |
| nldidlnorm.XX.Item6.Descriptor1.Statement.Identifier                                      | Array                           |       11553 |   0.71209451200908280288 |
| nldidlnorm.XX.Item6.Descriptor2.Statement.accessRights                                    | Array                           |       11553 |   0.71209451200908280288 |
| nldidlnorm.XX.Item.Component.Resource.mods.name5.affiliation1                             | Array                           |       11351 |   0.69964379865100834177 |
| nldidlnorm.XX.Item.Component.Resource.mods.name9.affiliation                              | Array                           |       11261 |   0.69409645111523254890 |
| nldidlnorm.XX.Item2.Descriptor3                                                           | Object                          |       11196 |   0.69009003345050567990 |
| nldidlnorm.XX.Item2.Descriptor3.Statement                                                 | Object                          |       11196 |   0.69009003345050567990 |
| nldidlnorm.XX.Item2.Descriptor3.Statement._attrs                                          | Array                           |       11196 |   0.69009003345050567990 |
| Journal.XX.relatedItem.name1                                                              | Object                          |       10973 |   0.67634493900075010053 |
| Journal.XX.relatedItem.name1.namePart                                                     | Array (10680),Object (293)      |       10973 |   0.67634493900075010053 |
| Journal.XX.relatedItem.name1.role                                                         | Object                          |       10973 |   0.67634493900075010053 |
| Journal.XX.relatedItem.name1.role.roleTerm                                                | Object                          |       10973 |   0.67634493900075010053 |
| Journal.XX.relatedItem.name1.role.roleTerm._attrs                                         | Array                           |       10973 |   0.67634493900075010053 |
| Journal.XX.relatedItem.name1.role.roleTerm.text                                           | Array                           |       10973 |   0.67634493900075010053 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name1                              | Object                          |       10973 |   0.67634493900075010053 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name1.namePart                     | Array (10680),Object (293)      |       10973 |   0.67634493900075010053 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name1.role                         | Object                          |       10973 |   0.67634493900075010053 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name1.role.roleTerm                | Object                          |       10973 |   0.67634493900075010053 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name1.role.roleTerm._attrs         | Array                           |       10973 |   0.67634493900075010053 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name1.role.roleTerm.text           | Array                           |       10973 |   0.67634493900075010053 |
| nldidlnorm.XX.Item1.Descriptor2.Statement.description                                     | Array                           |       10761 |   0.66327785369425606987 |
| nldidlnorm.XX.Item.Component.Resource.mods.name3.role3                                    | Object                          |       10624 |   0.65483355800090847776 |
| nldidlnorm.XX.Item.Component.Resource.mods.name3.role3.roleTerm                           | Object                          |       10624 |   0.65483355800090847776 |
| nldidlnorm.XX.Item.Component.Resource.mods.name3.role3.roleTerm._attrs                    | Array                           |       10624 |   0.65483355800090847776 |
| nldidlnorm.XX.Item.Component.Resource.mods.name3.role3.roleTerm.text                      | Array                           |       10624 |   0.65483355800090847776 |
| nldidlnorm.XX.Item7.Descriptor1.Statement.Identifier                                      | Array                           |       10334 |   0.63695877149674218209 |
| nldidlnorm.XX.Item7.Descriptor2.Statement.accessRights                                    | Array                           |       10334 |   0.63695877149674218209 |
| nldidlnorm.XX.Item.Component.Resource.mods.name.namePart2._attrs                          | Array                           |       10152 |   0.62574080203550674906 |
| nldidlnorm.XX.Item.Component.Resource.mods.name.namePart2.text                            | Array                           |       10152 |   0.62574080203550674906 |
| nldidlnorm.XX.Item8.Descriptor1.Statement.Identifier                                      | Array                           |       10130 |   0.62438478374898376266 |
| nldidlnorm.XX.Item8.Descriptor2.Statement.accessRights                                    | Array                           |       10130 |   0.62438478374898376266 |
| nldidlnorm.XX.Item.Component.Resource.mods.titleInfo2._attrs                              | Array                           |       10113 |   0.62333695143667056104 |
| nldidlnorm.XX.Item9.Descriptor1.Statement.Identifier                                      | Array                           |        9931 |   0.61211898197543512801 |
| nldidlnorm.XX.Item9.Descriptor2.Statement.accessRights                                    | Array                           |        9931 |   0.61211898197543512801 |
| nldidlnorm.XX.Item.Component.Resource.mods.name9.role.roleTerm1                           | Object                          |        9894 |   0.60983840576628289831 |
| nldidlnorm.XX.Item.Component.Resource.mods.name9.role.roleTerm1._attrs                    | Array                           |        9894 |   0.60983840576628289831 |
| nldidlnorm.XX.Item.Component.Resource.mods.name9.role.roleTerm1.text                      | Array                           |        9894 |   0.60983840576628289831 |
| nldidlnorm.XX.Item0.Descriptor1.Statement.Identifier                                      | Array                           |        9709 |   0.59843552472052152780 |
| nldidlnorm.XX.Item0.Descriptor2.Statement.accessRights                                    | Array                           |        9709 |   0.59843552472052152780 |
| nldidlnorm.XX.Item.Component.Resource.mods.name2.affiliation2                             | Array                           |        9521 |   0.58684773209023444185 |
| nldidlnorm.XX.Item.Component.Resource.mods.name0.displayForm                              | Array                           |        9361 |   0.57698578091552188418 |
| nldidlnorm.XX.Item.Component.Resource.mods.classification1                                | Object                          |        9263 |   0.57094533582101048008 |
| nldidlnorm.XX.Item.Component.Resource.mods.classification1._attrs                         | Array                           |        9263 |   0.57094533582101048008 |
| nldidlnorm.XX.Item.Component.Resource.mods.classification1.text                           | Array                           |        9263 |   0.57094533582101048008 |
| nldidlnorm.XX.Item2.Descriptor2.Statement.modified                                        | Array                           |        9194 |   0.56669236937691580547 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject2.topic1                                | Array (2551),Object (6503)      |        9054 |   0.55806316209904238690 |
| nldidlnorm.XX.Item4.Descriptor1.Statement.accessRights                                    | Array                           |        9035 |   0.55689205539704522696 |
| nldidlnorm.XX.Item.Component.Resource.mods.name0.nameIdentifier                           | Object                          |        8878 |   0.54721501580685860677 |
| nldidlnorm.XX.Item.Component.Resource.mods.name0.nameIdentifier._attrs                    | Array                           |        8878 |   0.54721501580685860677 |
| nldidlnorm.XX.Item.Component.Resource.mods.name0.nameIdentifier.text                      | Array                           |        8878 |   0.54721501580685860677 |
| nldidlnorm.XX.Item.Component.Resource.mods.name0.affiliation                              | Array                           |        8842 |   0.54499607679254835624 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject3.topic._attrs                          | Array                           |        8640 |   0.53254536343447378410 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject3.topic.text                            | Array                           |        8640 |   0.53254536343447378410 |
| nldidlnorm.XX.Item.Component.Resource.mods.name.affiliation3                              | Array                           |        8633 |   0.53211390307058015203 |
| Journal.XX.relatedItem1.location                                                          | Object                          |        8590 |   0.52946350369237615840 |
| Journal.XX.relatedItem1.location.physicalLocation                                         | Array                           |        8590 |   0.52946350369237615840 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem1.location                          | Object                          |        8590 |   0.52946350369237615840 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem1.location.physicalLocation         | Array                           |        8590 |   0.52946350369237615840 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.dateValid                           | Object (8484),Array (14)        |        8498 |   0.52379288176691651824 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.dateValid._attrs                    | Array                           |        8484 |   0.52292996103912914307 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.dateValid.text                      | Array                           |        8484 |   0.52292996103912914307 |
| nldidlnorm.XX.Item.Component.Resource.mods.name6.affiliation1                             | Array                           |        8254 |   0.50875340622547993163 |
| nldidlnorm.XX.Item.Component.Resource.mods.name0.role.roleTerm1                           | Object                          |        8132 |   0.50123366845476169384 |
| nldidlnorm.XX.Item.Component.Resource.mods.name0.role.roleTerm1._attrs                    | Array                           |        8132 |   0.50123366845476169384 |
| nldidlnorm.XX.Item.Component.Resource.mods.name0.role.roleTerm1.text                      | Array                           |        8132 |   0.50123366845476169384 |
| nldidlnorm.XX.Item.Component.Resource.mods.identifier9                                    | Object                          |        7441 |   0.45864236681897219139 |
| nldidlnorm.XX.Item.Component.Resource.mods.identifier9._attrs                             | Array                           |        7441 |   0.45864236681897219139 |
| nldidlnorm.XX.Item.Component.Resource.mods.identifier9.text                               | Array                           |        7441 |   0.45864236681897219139 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject6                                       | Object                          |        7266 |   0.44785585772163039042 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject6.topic                                 | Array (5399),Object (1867)      |        7266 |   0.44785585772163039042 |
| nldidlnorm.XX.Item.Component.Resource.mods.abstract1                                      | Array (7188),Object (1)         |        7189 |   0.44310979371880004907 |
| nldidlnorm.XX.Item.Component.Resource.mods.extension6                                     | Object                          |        7053 |   0.43472713522029443611 |
| nldidlnorm.XX.Item.Component.Resource.mods.extension6.rights                              | Object                          |        7053 |   0.43472713522029443611 |
| nldidlnorm.XX.Item.Component.Resource.mods.extension6.rights._attrs                       | Array                           |        7053 |   0.43472713522029443611 |
| nldidlnorm.XX.Item.Component.Resource.mods.extension6.rights.description                  | Array                           |        7053 |   0.43472713522029443611 |
| nldidlnorm.XX.Item5.Descriptor1.Statement.accessRights                                    | Array                           |        7030 |   0.43330947973892947056 |
| nldidlnorm.XX.Item.Component.Resource.mods.name4.role3                                    | Object                          |        6956 |   0.42874832732062495566 |
| nldidlnorm.XX.Item.Component.Resource.mods.name4.role3.roleTerm                           | Object                          |        6956 |   0.42874832732062495566 |
| nldidlnorm.XX.Item.Component.Resource.mods.name4.role3.roleTerm._attrs                    | Array                           |        6956 |   0.42874832732062495566 |
| nldidlnorm.XX.Item.Component.Resource.mods.name4.role3.roleTerm.text                      | Array                           |        6956 |   0.42874832732062495566 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject3.temporal                              | Object                          |        6693 |   0.41253774507719132014 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject3.temporal._attrs                       | Array                           |        6693 |   0.41253774507719132014 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject3.temporal.text                         | Array                           |        6693 |   0.41253774507719132014 |
| nldidlnorm.XX.Item.Component.Resource.mods.name3.affiliation2                             | Array                           |        6593 |   0.40637402559299606875 |
| nldidlnorm.XX.Item.Component.Resource.mods.name1.namePart2                                | Object (4095),Array (2456)      |        6551 |   0.40378526340963399877 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject2.topic1._attrs                         | Array                           |        6503 |   0.40082667805722027587 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject2.topic1.text                           | Array                           |        6503 |   0.40082667805722027587 |
| nldidlnorm.XX.Item2.Descriptor3.Statement.modified                                        | Array                           |        6229 |   0.38393808667052514716 |
| Journal.XX.relatedItem.location                                                           | Object                          |        6096 |   0.37574033975654541617 |
| Journal.XX.relatedItem.location.physicalLocation                                          | Array                           |        6096 |   0.37574033975654541617 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.location                           | Object                          |        6096 |   0.37574033975654541617 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.location.physicalLocation          | Array                           |        6096 |   0.37574033975654541617 |
| nldidlnorm.XX.Item.Component.Resource.mods.language1                                      | Object                          |        6095 |   0.37567870256170343701 |
| nldidlnorm.XX.Item.Component.Resource.mods.language1.languageTerm                         | Object                          |        6095 |   0.37567870256170343701 |
| nldidlnorm.XX.Item.Component.Resource.mods.language1.languageTerm._attrs                  | Array                           |        6095 |   0.37567870256170343701 |
| nldidlnorm.XX.Item.Component.Resource.mods.language1.languageTerm.text                    | Array                           |        6095 |   0.37567870256170343701 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject1.topic._attrs                          | Array                           |        6089 |   0.37530887939265172859 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject1.topic.text                            | Array                           |        6089 |   0.37530887939265172859 |
| nldidlnorm.XX.Item6.Descriptor1.Statement.accessRights                                    | Array                           |        6009 |   0.37037790380529550527 |
| nldidlnorm.XX.Item.Component.Resource.mods.name7.affiliation1                             | Array                           |        5935 |   0.36581675138699099037 |
| nldidlnorm.XX.Item7.Descriptor1.Statement.accessRights                                    | Array                           |        5608 |   0.34566138867367235399 |
| nldidlnorm.XX.Item8.Descriptor1.Statement.accessRights                                    | Array                           |        5489 |   0.33832656248747994265 |
| nldidlnorm.XX.Item0.Descriptor1.Statement.accessRights                                    | Array                           |        5381 |   0.33166974544454902452 |
| nldidlnorm.XX.Item9.Descriptor1.Statement.accessRights                                    | Array                           |        5332 |   0.32864952289729332247 |
| Journal.XX.relatedItem.name2                                                              | Object                          |        5105 |   0.31465787966816999299 |
| Journal.XX.relatedItem.name2.namePart                                                     | Array (4835),Object (270)       |        5105 |   0.31465787966816999299 |
| Journal.XX.relatedItem.name2.role                                                         | Object                          |        5105 |   0.31465787966816999299 |
| Journal.XX.relatedItem.name2.role.roleTerm                                                | Object                          |        5105 |   0.31465787966816999299 |
| Journal.XX.relatedItem.name2.role.roleTerm._attrs                                         | Array                           |        5105 |   0.31465787966816999299 |
| Journal.XX.relatedItem.name2.role.roleTerm.text                                           | Array                           |        5105 |   0.31465787966816999299 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name2                              | Object                          |        5105 |   0.31465787966816999299 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name2.namePart                     | Array (4835),Object (270)       |        5105 |   0.31465787966816999299 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name2.role                         | Object                          |        5105 |   0.31465787966816999299 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name2.role.roleTerm                | Object                          |        5105 |   0.31465787966816999299 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name2.role.roleTerm._attrs         | Array                           |        5105 |   0.31465787966816999299 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name2.role.roleTerm.text           | Array                           |        5105 |   0.31465787966816999299 |
| nldidlnorm.XX.Item.Component.Resource.mods.name4.affiliation2                             | Array                           |        5072 |   0.31262385223838556891 |
| nldidlnorm.XX.Item.Component.Resource.mods.titleInfo3                                     | Object                          |        5008 |   0.30867907176850056805 |
| nldidlnorm.XX.Item.Component.Resource.mods.titleInfo3.title                               | Array                           |        5008 |   0.30867907176850056805 |
| nldidlnorm.XX.Item2.Descriptor3.Statement.description                                     | Array                           |        4967 |   0.30615194677998047723 |
| nldidlnorm.XX.Item.Component.Resource.mods.name5.role3                                    | Object                          |        4749 |   0.29271503830443473815 |
| nldidlnorm.XX.Item.Component.Resource.mods.name5.role3.roleTerm                           | Object                          |        4749 |   0.29271503830443473815 |
| nldidlnorm.XX.Item.Component.Resource.mods.name5.role3.roleTerm._attrs                    | Array                           |        4749 |   0.29271503830443473815 |
| nldidlnorm.XX.Item.Component.Resource.mods.name5.role3.roleTerm.text                      | Array                           |        4749 |   0.29271503830443473815 |
| nldidlnorm.XX.Item.Component.Resource.mods.titleInfo3._attrs                              | Array                           |        4735 |   0.29185211757664741850 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject4.topic._attrs                          | Array                           |        4690 |   0.28907844380875952206 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject4.topic.text                            | Array                           |        4690 |   0.28907844380875952206 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject7                                       | Object                          |        4687 |   0.28889353222423364009 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject7.topic                                 | Array (3522),Object (1165)      |        4687 |   0.28889353222423364009 |
| nldidlnorm.XX.Item.Component.Resource.mods.extension7                                     | Object                          |        4402 |   0.27132693169427707369 |
| nldidlnorm.XX.Item.Component.Resource.mods.extension7.rights                              | Object                          |        4402 |   0.27132693169427707369 |
| nldidlnorm.XX.Item.Component.Resource.mods.extension7.rights._attrs                       | Array                           |        4402 |   0.27132693169427707369 |
| nldidlnorm.XX.Item.Component.Resource.mods.extension7.rights.description                  | Array                           |        4402 |   0.27132693169427707369 |
| nldidlnorm.XX.Item.Component.Resource.mods.name8.affiliation1                             | Array                           |        4375 |   0.26966272743354430252 |
| nldidlnorm.XX.Item2.Descriptor2.Statement.description                                     | Array                           |        4233 |   0.26091024576598698115 |
| nldidlnorm.XX.Item.Component.Resource.mods.classification2                                | Object                          |        4189 |   0.25819820919294106387 |
| nldidlnorm.XX.Item.Component.Resource.mods.classification2._attrs                         | Array                           |        4189 |   0.25819820919294106387 |
| nldidlnorm.XX.Item.Component.Resource.mods.classification2.text                           | Array                           |        4189 |   0.25819820919294106387 |
| nldidlnorm.XX.Item.Component.Resource.mods.name1.namePart2._attrs                         | Array                           |        4095 |   0.25240431287779746539 |
| nldidlnorm.XX.Item.Component.Resource.mods.name1.namePart2.text                           | Array                           |        4095 |   0.25240431287779746539 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject2.temporal                              | Object                          |        4079 |   0.25141811776032624293 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject2.temporal._attrs                       | Array                           |        4079 |   0.25141811776032624293 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject2.temporal.text                         | Array                           |        4079 |   0.25141811776032624293 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject4.temporal                              | Object                          |        3940 |   0.24285054767729477576 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject4.temporal._attrs                       | Array                           |        3940 |   0.24285054767729477576 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject4.temporal.text                         | Array                           |        3940 |   0.24285054767729477576 |
| nldidlnorm.XX.Item.Component.Resource.mods.name5.affiliation2                             | Array                           |        3826 |   0.23582390746531212145 |
| nldidlnorm.XX.Item3.Descriptor2.Statement.description                                     | Array                           |        3798 |   0.23409806600973745438 |
| nldidlnorm.XX.Item.Component.Resource.mods.titleInfo4                                     | Object                          |        3636 |   0.22411284044534104942 |
| nldidlnorm.XX.Item.Component.Resource.mods.titleInfo4.title                               | Array                           |        3636 |   0.22411284044534104942 |
| nldidlnorm.XX.Item.Component.Resource.mods.name2.namePart2                                | Object (1338),Array (2251)      |        3589 |   0.22121589228776927794 |
| nldidlnorm.XX.Item0.Descriptor2.Statement.description                                     | Array                           |        3476 |   0.21425088927062857502 |
| nldidlnorm.XX.Item9.Descriptor2.Statement.description                                     | Array                           |        3456 |   0.21301814537378951919 |
| nldidlnorm.XX.Item.Component.Resource.mods.titleInfo4._attrs                              | Array                           |        3420 |   0.21079920635947921315 |
| Journal.XX.relatedItem1.identifier                                                        | Object                          |        3417 |   0.21061429477495335894 |
| Journal.XX.relatedItem1.identifier._attrs                                                 | Array                           |        3417 |   0.21061429477495335894 |
| Journal.XX.relatedItem1.identifier.text                                                   | Array                           |        3417 |   0.21061429477495335894 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem1.identifier                        | Object                          |        3417 |   0.21061429477495335894 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem1.identifier._attrs                 | Array                           |        3417 |   0.21061429477495335894 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem1.identifier.text                   | Array                           |        3417 |   0.21061429477495335894 |
| nldidlnorm.XX.Item8.Descriptor2.Statement.description                                     | Array                           |        3385 |   0.20864190454001085850 |
| nldidlnorm.XX.Item4.Descriptor2.Statement.description                                     | Array                           |        3365 |   0.20740916064317180267 |
| nldidlnorm.XX.Item7.Descriptor2.Statement.description                                     | Array                           |        3294 |   0.20303291980939314199 |
| nldidlnorm.XX.Item.Component.Resource.mods.name6.role3                                    | Object                          |        3250 |   0.20032088323634719695 |
| nldidlnorm.XX.Item.Component.Resource.mods.name6.role3.roleTerm                           | Object                          |        3250 |   0.20032088323634719695 |
| nldidlnorm.XX.Item.Component.Resource.mods.name6.role3.roleTerm._attrs                    | Array                           |        3250 |   0.20032088323634719695 |
| nldidlnorm.XX.Item.Component.Resource.mods.name6.role3.roleTerm.text                      | Array                           |        3250 |   0.20032088323634719695 |
| nldidlnorm.XX.Item.Component.Resource.mods.name1.affiliation3                             | Array                           |        3237 |   0.19951959970340182871 |
| nldidlnorm.XX.Item.Component.Resource.mods.name9.affiliation1                             | Array                           |        3225 |   0.19877995336529838410 |
| Journal.XX.relatedItem.titleInfo.title1                                                   | Array                           |        3187 |   0.19643773996130417525 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.titleInfo.title1                   | Array                           |        3187 |   0.19643773996130417525 |
| nldidlnorm.XX.Item6.Descriptor2.Statement.description                                     | Array                           |        3182 |   0.19612955398709439048 |
| nldidlnorm.XX.Item5.Descriptor2.Statement.description                                     | Array                           |        3141 |   0.19360242899857432741 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject8                                       | Object                          |        3124 |   0.19255459668626112579 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject8.topic                                 | Array (2401),Object (723)       |        3124 |   0.19255459668626112579 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.dateIssued1                         | Object (2579),Array (455)       |        3034 |   0.18700724915048536068 |
| nldidlnorm.XX.Item.Component.Resource.mods.extension8                                     | Object                          |        2891 |   0.17819313028808608790 |
| nldidlnorm.XX.Item.Component.Resource.mods.extension8.rights                              | Object                          |        2891 |   0.17819313028808608790 |
| nldidlnorm.XX.Item.Component.Resource.mods.extension8.rights._attrs                       | Array                           |        2891 |   0.17819313028808608790 |
| nldidlnorm.XX.Item.Component.Resource.mods.extension8.rights.description                  | Array                           |        2891 |   0.17819313028808608790 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject5.topic._attrs                          | Array                           |        2857 |   0.17609746566345968466 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject5.topic.text                            | Array                           |        2857 |   0.17609746566345968466 |
| nldidlnorm.XX.Item.Component.Resource.mods.name6.affiliation2                             | Array                           |        2812 |   0.17332379189557178822 |
| nldidlnorm.XX.Item.Component.Resource.mods.name3.namePart2                                | Object (736),Array (1962)       |        2698 |   0.16629715168358916166 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.dateIssued1._attrs                  | Array                           |        2579 |   0.15896232549739675033 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.dateIssued1.text                    | Array                           |        2579 |   0.15896232549739675033 |
| Journal.XX.relatedItem1.part.extent.end                                                   | Array                           |        2469 |   0.15218223406478192938 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem1.part.extent.end                   | Array                           |        2469 |   0.15218223406478192938 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject5.temporal                              | Object                          |        2449 |   0.15094949016794287355 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject5.temporal._attrs                       | Array                           |        2449 |   0.15094949016794287355 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject5.temporal.text                         | Array                           |        2449 |   0.15094949016794287355 |
| Journal.XX.relatedItem1.part.extent.start                                                 | Array                           |        2437 |   0.15020984382983942895 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem1.part.extent.start                 | Array                           |        2437 |   0.15020984382983942895 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject3.topic1                                | Array (1207),Object (1226)      |        2433 |   0.14996329505047162334 |
| nldidlnorm.XX.Item.Component.Resource.mods.name0.affiliation1                             | Array                           |        2369 |   0.14601851458058662248 |
| nldidlnorm.XX.Item.Component.Resource.mods.name2.affiliation3                             | Array                           |        2332 |   0.14373793837143436503 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject1.topic1._attrs                         | Array                           |        2277 |   0.14034789265512695455 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject1.topic1.text                           | Array                           |        2277 |   0.14034789265512695455 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject2.topic2                                | Array                           |        2274 |   0.14016298107060110034 |
| nldidlnorm.XX.Item.Component.Resource.mods.name7.role3                                    | Object                          |        2162 |   0.13325961524830234883 |
| nldidlnorm.XX.Item.Component.Resource.mods.name7.role3.roleTerm                           | Object                          |        2162 |   0.13325961524830234883 |
| nldidlnorm.XX.Item.Component.Resource.mods.name7.role3.roleTerm._attrs                    | Array                           |        2162 |   0.13325961524830234883 |
| nldidlnorm.XX.Item.Component.Resource.mods.name7.role3.roleTerm.text                      | Array                           |        2162 |   0.13325961524830234883 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject9                                       | Object                          |        2078 |   0.12808209088157831990 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject9.topic                                 | Array (1613),Object (465)       |        2078 |   0.12808209088157831990 |
| Journal.XX.relatedItem.name3                                                              | Object                          |        2067 |   0.12740408173831682670 |
| Journal.XX.relatedItem.name3.namePart                                                     | Array (1847),Object (220)       |        2067 |   0.12740408173831682670 |
| Journal.XX.relatedItem.name3.role                                                         | Object                          |        2067 |   0.12740408173831682670 |
| Journal.XX.relatedItem.name3.role.roleTerm                                                | Object                          |        2067 |   0.12740408173831682670 |
| Journal.XX.relatedItem.name3.role.roleTerm._attrs                                         | Array                           |        2067 |   0.12740408173831682670 |
| Journal.XX.relatedItem.name3.role.roleTerm.text                                           | Array                           |        2067 |   0.12740408173831682670 |
| nldidlnorm.XX.Item.Component.Resource.mods.identifier0                                    | Object                          |        2067 |   0.12740408173831682670 |
| nldidlnorm.XX.Item.Component.Resource.mods.identifier0._attrs                             | Array                           |        2067 |   0.12740408173831682670 |
| nldidlnorm.XX.Item.Component.Resource.mods.identifier0.text                               | Array                           |        2067 |   0.12740408173831682670 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name3                              | Object                          |        2067 |   0.12740408173831682670 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name3.namePart                     | Array (1847),Object (220)       |        2067 |   0.12740408173831682670 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name3.role                         | Object                          |        2067 |   0.12740408173831682670 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name3.role.roleTerm                | Object                          |        2067 |   0.12740408173831682670 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name3.role.roleTerm._attrs         | Array                           |        2067 |   0.12740408173831682670 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name3.role.roleTerm.text           | Array                           |        2067 |   0.12740408173831682670 |
| nldidlnorm.XX.Item.Component.Resource.mods.name4.namePart2                                | Object (354),Array (1638)       |        1992 |   0.12278129212517034652 |
| nldidlnorm.XX.Item.Component.Resource.mods.extension9                                     | Object                          |        1979 |   0.12198000859222496439 |
| nldidlnorm.XX.Item.Component.Resource.mods.extension9.rights                              | Object                          |        1979 |   0.12198000859222496439 |
| nldidlnorm.XX.Item.Component.Resource.mods.extension9.rights._attrs                       | Array                           |        1979 |   0.12198000859222496439 |
| nldidlnorm.XX.Item.Component.Resource.mods.extension9.rights.description                  | Array                           |        1979 |   0.12198000859222496439 |
| nldidlnorm.XX.Item.Component.Resource.mods.name.affiliation4                              | Array                           |        1969 |   0.12136363664380543648 |
| nldidlnorm.XX.Item.Component.Resource.mods.name7.affiliation2                             | Array                           |        1939 |   0.11951452079854683885 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject2.topic3                                | Array                           |        1903 |   0.11729558178423653281 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject6.topic._attrs                          | Array                           |        1867 |   0.11507664276992622676 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject6.topic.text                            | Array                           |        1867 |   0.11507664276992622676 |
| Journal.XX.relatedItem1.part.extent.total                                                 | Array                           |        1827 |   0.11261115497624810122 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem1.part.extent.total                 | Array                           |        1827 |   0.11261115497624810122 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject6.temporal                              | Object                          |        1643 |   0.10126991112532875150 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject6.temporal._attrs                       | Array                           |        1643 |   0.10126991112532875150 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject6.temporal.text                         | Array                           |        1643 |   0.10126991112532875150 |
| nldidlnorm.XX.Item.Component.Resource.mods.classification3                                | Object                          |        1638 |   0.10096172515111899448 |
| nldidlnorm.XX.Item.Component.Resource.mods.classification3._attrs                         | Array                           |        1638 |   0.10096172515111899448 |
| nldidlnorm.XX.Item.Component.Resource.mods.classification3.text                           | Array                           |        1638 |   0.10096172515111899448 |
| nldidlnorm.XX.Item.Component.Resource.mods.name5.namePart2                                | Object (287),Array (1332)       |        1619 |   0.09979061844912189005 |
| nldidlnorm.XX.Item.Component.Resource.mods.name3.affiliation3                             | Array                           |        1618 |   0.09972898125427993865 |
| nldidlnorm.XX.Item.Component.Resource.mods.name8.role3                                    | Object                          |        1589 |   0.09794150260386329243 |
| nldidlnorm.XX.Item.Component.Resource.mods.name8.role3.roleTerm                           | Object                          |        1589 |   0.09794150260386329243 |
| nldidlnorm.XX.Item.Component.Resource.mods.name8.role3.roleTerm._attrs                    | Array                           |        1589 |   0.09794150260386329243 |
| nldidlnorm.XX.Item.Component.Resource.mods.name8.role3.roleTerm.text                      | Array                           |        1589 |   0.09794150260386329243 |
| Journal.XX.relatedItem.physicalDescription                                                | Object                          |        1546 |   0.09529110322565931268 |
| Journal.XX.relatedItem.physicalDescription.form                                           | Array                           |        1546 |   0.09529110322565931268 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.physicalDescription                | Object                          |        1546 |   0.09529110322565931268 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.physicalDescription.form           | Array                           |        1546 |   0.09529110322565931268 |
| nldidlnorm.XX.Item.Component.Resource.mods.titleInfo1.subTitle                            | Array                           |        1543 |   0.09510619164113345847 |
| nldidlnorm.XX.Item3.Descriptor2.Statement.modified                                        | Array                           |        1535 |   0.09461309408239783336 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject2.topic4                                | Array                           |        1530 |   0.09430490810818807634 |
| nldidlnorm.XX.Item.Component.Resource.mods.name8.affiliation2                             | Array                           |        1519 |   0.09362689896492658315 |
| nldidlnorm.XX.Item4.Descriptor2.Statement.modified                                        | Array                           |        1442 |   0.08888083496209620016 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject0                                       | Object                          |        1417 |   0.08733990509104738731 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject0.topic                                 | Array (1137),Object (280)       |        1417 |   0.08733990509104738731 |
| nldidlnorm.XX.Item5.Descriptor2.Statement.modified                                        | Array                           |        1394 |   0.08592224960968246339 |
| nldidlnorm.XX.Item.Component.Resource.mods.name2.namePart2._attrs                         | Array                           |        1338 |   0.08247056669853310151 |
| nldidlnorm.XX.Item.Component.Resource.mods.name2.namePart2.text                           | Array                           |        1338 |   0.08247056669853310151 |
| nldidlnorm.XX.Item.Component.Resource.mods.name6.namePart2                                | Object (222),Array (1111)       |        1333 |   0.08216238072432333062 |
| nldidlnorm.XX.Item6.Descriptor2.Statement.modified                                        | Array                           |        1315 |   0.08105291121716817759 |
| nldidlnorm.XX.Item0.Descriptor2.Statement.modified                                        | Array                           |        1276 |   0.07864906061833201734 |
| nldidlnorm.XX.Item7.Descriptor2.Statement.modified                                        | Array                           |        1275 |   0.07858742342349005205 |
| nldidlnorm.XX.Item9.Descriptor2.Statement.modified                                        | Array                           |        1262 |   0.07778613989054466993 |
| nldidlnorm.XX.Item8.Descriptor2.Statement.modified                                        | Array                           |        1258 |   0.07753959111117686431 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject2.topic5                                | Array                           |        1251 |   0.07710813074728319061 |
| nldidlnorm.XX.Item.Component.Resource.mods.name4.affiliation3                             | Array                           |        1239 |   0.07636848440917974601 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject3.topic1._attrs                         | Array                           |        1226 |   0.07556720087623436388 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject3.topic1.text                           | Array                           |        1226 |   0.07556720087623436388 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject3.topic2                                | Array                           |        1169 |   0.07205388077024303672 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject7.topic._attrs                          | Array                           |        1165 |   0.07180733199087523111 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject7.topic.text                            | Array                           |        1165 |   0.07180733199087523111 |
| nldidlnorm.XX.Item.Component.Resource.mods.name7.namePart2                                | Object (183),Array (944)        |        1127 |   0.06946511858688102226 |
| nldidlnorm.XX.Item.Component.Resource.mods.name9.role3                                    | Object                          |        1118 |   0.06891038383330344574 |
| nldidlnorm.XX.Item.Component.Resource.mods.name9.role3.roleTerm                           | Object                          |        1118 |   0.06891038383330344574 |
| nldidlnorm.XX.Item.Component.Resource.mods.name9.role3.roleTerm._attrs                    | Array                           |        1118 |   0.06891038383330344574 |
| nldidlnorm.XX.Item.Component.Resource.mods.name9.role3.roleTerm.text                      | Array                           |        1118 |   0.06891038383330344574 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject3.topic3                                | Array                           |        1104 |   0.06804746310551609834 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.publisher1                          | Array                           |        1097 |   0.06761600274162242463 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.copyrightDate                       | Object (1010),Array (42)        |        1052 |   0.06484232897373454207 |
| nldidlnorm.XX.Item.Component.Resource.mods.name9.affiliation2                             | Array                           |        1048 |   0.06459578019436673646 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject3.topic4                                | Array                           |        1039 |   0.06404104544078915995 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject7.temporal                              | Object                          |        1024 |   0.06311648751815986114 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject7.temporal._attrs                       | Array                           |        1024 |   0.06311648751815986114 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject7.temporal.text                         | Array                           |        1024 |   0.06311648751815986114 |
| nldidlnorm.XX.Item3.Descriptor3                                                           | Object                          |        1013 |   0.06243847837489837488 |
| nldidlnorm.XX.Item3.Descriptor3.Statement                                                 | Object                          |        1013 |   0.06243847837489837488 |
| nldidlnorm.XX.Item3.Descriptor3.Statement._attrs                                          | Array                           |        1013 |   0.06243847837489837488 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.copyrightDate._attrs                | Array                           |        1010 |   0.06225356679037251373 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.copyrightDate.text                  | Array                           |        1010 |   0.06225356679037251373 |
| nldidlnorm.XX.Item.Component.Resource.mods.name5.affiliation3                             | Array                           |        1009 |   0.06219192959553056232 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject2.topic6                                | Array                           |        1009 |   0.06219192959553056232 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject3.topic5                                | Array                           |         971 |   0.05984971619153635347 |
| nldidlnorm.XX.Item3.Descriptor3.Statement.modified                                        | Array                           |         941 |   0.05800060034627776279 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.dateCreated1                        | Object (895),Array (6)          |         901 |   0.05553511255259964419 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.dateCreated1._attrs                 | Array                           |         895 |   0.05516528938354792189 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.dateCreated1.text                   | Array                           |         895 |   0.05516528938354792189 |
| nldidlnorm.XX.Item.Component.Resource.mods.name6.affiliation3                             | Array                           |         882 |   0.05436400585060253282 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject3.topic6                                | Array                           |         872 |   0.05374763390218300491 |
| nldidlnorm.XX.Item.Component.Resource.mods.name8.namePart2                                | Object (71),Array (789)         |         860 |   0.05300798756407956724 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject2.topic7                                | Array                           |         857 |   0.05282307597955371303 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.dateOther1                          | Object                          |         856 |   0.05276143878471175469 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.dateOther1._attrs                   | Array                           |         856 |   0.05276143878471175469 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.dateOther1.text                     | Array                           |         856 |   0.05276143878471175469 |
| nldidlnorm.XX.Item.Component.Resource.mods.name0.role3                                    | Object                          |         817 |   0.05035758818587558749 |
| nldidlnorm.XX.Item.Component.Resource.mods.name0.role3.roleTerm                           | Object                          |         817 |   0.05035758818587558749 |
| nldidlnorm.XX.Item.Component.Resource.mods.name0.role3.roleTerm._attrs                    | Array                           |         817 |   0.05035758818587558749 |
| nldidlnorm.XX.Item.Component.Resource.mods.name0.role3.roleTerm.text                      | Array                           |         817 |   0.05035758818587558749 |
| nldidlnorm.XX.Item.Component.Resource.mods.name.nameIdentifier1                           | Object                          |         802 |   0.04943303026324629562 |
| nldidlnorm.XX.Item.Component.Resource.mods.name.nameIdentifier1._attrs                    | Array                           |         802 |   0.04943303026324629562 |
| nldidlnorm.XX.Item.Component.Resource.mods.name.nameIdentifier1.text                      | Array                           |         802 |   0.04943303026324629562 |
| Journal.XX.relatedItem.name4                                                              | Object                          |         791 |   0.04875502111998480936 |
| Journal.XX.relatedItem.name4.namePart                                                     | Array (789),Object (2)          |         791 |   0.04875502111998480936 |
| Journal.XX.relatedItem.name4.role                                                         | Object                          |         791 |   0.04875502111998480936 |
| Journal.XX.relatedItem.name4.role.roleTerm                                                | Object                          |         791 |   0.04875502111998480936 |
| Journal.XX.relatedItem.name4.role.roleTerm._attrs                                         | Array                           |         791 |   0.04875502111998480936 |
| Journal.XX.relatedItem.name4.role.roleTerm.text                                           | Array                           |         791 |   0.04875502111998480936 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name4                              | Object                          |         791 |   0.04875502111998480936 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name4.namePart                     | Array (789),Object (2)          |         791 |   0.04875502111998480936 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name4.role                         | Object                          |         791 |   0.04875502111998480936 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name4.role.roleTerm                | Object                          |         791 |   0.04875502111998480936 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name4.role.roleTerm._attrs         | Array                           |         791 |   0.04875502111998480936 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name4.role.roleTerm.text           | Array                           |         791 |   0.04875502111998480936 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject3.topic7                                | Array                           |         791 |   0.04875502111998480936 |
| nldidlnorm.XX.Item.Component.Resource.mods.name0.affiliation2                             | Array                           |         770 |   0.04746064002830380213 |
| Journal.XX.relatedItem.part.extent1                                                       | Object                          |         762 |   0.04696754246956817702 |
| Journal.XX.relatedItem.part.extent1._attrs                                                | Array                           |         762 |   0.04696754246956817702 |
| Journal.XX.relatedItem.part.extent1.total                                                 | Array                           |         762 |   0.04696754246956817702 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.part.extent1                       | Object                          |         762 |   0.04696754246956817702 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.part.extent1._attrs                | Array                           |         762 |   0.04696754246956817702 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.part.extent1.total                 | Array                           |         762 |   0.04696754246956817702 |
| nldidlnorm.XX.Item.Component.Resource.mods.classification4                                | Object                          |         740 |   0.04561152418304521144 |
| nldidlnorm.XX.Item.Component.Resource.mods.classification4._attrs                         | Array                           |         740 |   0.04561152418304521144 |
| nldidlnorm.XX.Item.Component.Resource.mods.classification4.text                           | Array                           |         740 |   0.04561152418304521144 |
| nldidlnorm.XX.Item.Component.Resource.mods.name3.namePart2._attrs                         | Array                           |         736 |   0.04536497540367739889 |
| nldidlnorm.XX.Item.Component.Resource.mods.name3.namePart2.text                           | Array                           |         736 |   0.04536497540367739889 |
| nldidlnorm.XX.Item.Component.Resource.mods.name9.namePart2                                | Object (60),Array (664)         |         724 |   0.04462532906557396123 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject8.topic._attrs                          | Array                           |         723 |   0.04456369187073200983 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject8.topic.text                            | Array                           |         723 |   0.04456369187073200983 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject2.topic8                                | Array                           |         717 |   0.04419386870168029446 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo1                                    | Object                          |         714 |   0.04400895711715443331 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo1.dateIssued                         | Object                          |         714 |   0.04400895711715443331 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo1.dateIssued._attrs                  | Array                           |         714 |   0.04400895711715443331 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo1.dateIssued.text                    | Array                           |         714 |   0.04400895711715443331 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject3.topic8                                | Array                           |         700 |   0.04314603638936709284 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject8.temporal                              | Object                          |         671 |   0.04135855773895045356 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject8.temporal._attrs                       | Array                           |         671 |   0.04135855773895045356 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject8.temporal.text                         | Array                           |         671 |   0.04135855773895045356 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject4.topic1                                | Object (357),Array (306)        |         663 |   0.04086546018021482846 |
| nldidlnorm.XX.Item.Component.Resource.mods.name0.namePart2                                | Array (584),Object (41)         |         625 |   0.03852324677622061960 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject3.topic9                                | Array                           |         586 |   0.03611939617738445241 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject2.topic9                                | Array                           |         572 |   0.03525647544959711194 |
| nldidlnorm.XX.Item.Component.Resource.mods.name7.affiliation3                             | Array                           |         543 |   0.03346899679918047266 |
| Journal.XX.relatedItem.originInfo.dateOther                                               | Object                          |         536 |   0.03303753643528679895 |
| Journal.XX.relatedItem.originInfo.dateOther._attrs                                        | Array                           |         536 |   0.03303753643528679895 |
| Journal.XX.relatedItem.originInfo.dateOther.text                                          | Array                           |         536 |   0.03303753643528679895 |
| Journal.XX.relatedItem.originInfo.dateOther1                                              | Object (528),Array (8)          |         536 |   0.03303753643528679895 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.originInfo.dateOther               | Object                          |         536 |   0.03303753643528679895 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.originInfo.dateOther._attrs        | Array                           |         536 |   0.03303753643528679895 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.originInfo.dateOther.text          | Array                           |         536 |   0.03303753643528679895 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.originInfo.dateOther1              | Object (528),Array (8)          |         536 |   0.03303753643528679895 |
| Journal.XX.relatedItem.originInfo.dateOther1._attrs                                       | Array                           |         528 |   0.03254443887655117384 |
| Journal.XX.relatedItem.originInfo.dateOther1.text                                         | Array                           |         528 |   0.03254443887655117384 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.originInfo.dateOther1._attrs       | Array                           |         528 |   0.03254443887655117384 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.originInfo.dateOther1.text         | Array                           |         528 |   0.03254443887655117384 |
| nldidlnorm.XX.Item4.Descriptor3                                                           | Object                          |         513 |   0.03161988095392188197 |
| nldidlnorm.XX.Item4.Descriptor3.Statement                                                 | Object                          |         513 |   0.03161988095392188197 |
| nldidlnorm.XX.Item4.Descriptor3.Statement._attrs                                          | Array                           |         513 |   0.03161988095392188197 |
| nldidlnorm.XX.Item.Component.Resource.mods.name.affiliation5                              | Array                           |         505 |   0.03112678339518625686 |
| nldidlnorm.XX.Item4.Descriptor3.Statement.modified                                        | Array                           |         505 |   0.03112678339518625686 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject1.temporal                              | Object                          |         471 |   0.02903111877055985710 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject1.temporal._attrs                       | Array                           |         471 |   0.02903111877055985710 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject1.temporal.text                         | Array                           |         471 |   0.02903111877055985710 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject9.topic._attrs                          | Array                           |         465 |   0.02866129560150813826 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject9.topic.text                            | Array                           |         465 |   0.02866129560150813826 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.edition                             | Array                           |         437 |   0.02693545414593345386 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject9.temporal                              | Object                          |         433 |   0.02668890536656564477 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject9.temporal._attrs                       | Array                           |         433 |   0.02668890536656564477 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject9.temporal.text                         | Array                           |         433 |   0.02668890536656564477 |
| nldidlnorm.XX.Item.Component.Resource.mods.name8.affiliation3                             | Array                           |         424 |   0.02613417061298806479 |
| nldidlnorm.XX.Item.Component.Resource.mods.name1.affiliation4                             | Array                           |         417 |   0.02570271024909439456 |
| Journal.XX.relatedItem.name5                                                              | Object                          |         397 |   0.02446996635225533526 |
| Journal.XX.relatedItem.name5.namePart                                                     | Array (395),Object (2)          |         397 |   0.02446996635225533526 |
| Journal.XX.relatedItem.name5.role                                                         | Object                          |         397 |   0.02446996635225533526 |
| Journal.XX.relatedItem.name5.role.roleTerm                                                | Object                          |         397 |   0.02446996635225533526 |
| Journal.XX.relatedItem.name5.role.roleTerm._attrs                                         | Array                           |         397 |   0.02446996635225533526 |
| Journal.XX.relatedItem.name5.role.roleTerm.text                                           | Array                           |         397 |   0.02446996635225533526 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name5                              | Object                          |         397 |   0.02446996635225533526 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name5.namePart                     | Array (395),Object (2)          |         397 |   0.02446996635225533526 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name5.role                         | Object                          |         397 |   0.02446996635225533526 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name5.role.roleTerm                | Object                          |         397 |   0.02446996635225533526 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name5.role.roleTerm._attrs         | Array                           |         397 |   0.02446996635225533526 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name5.role.roleTerm.text           | Array                           |         397 |   0.02446996635225533526 |
| Journal.XX.relatedItem.name._attrs                                                        | Array                           |         396 |   0.02440832915741338385 |
| Journal.XX.relatedItem.name.namePart._attrs                                               | Array                           |         396 |   0.02440832915741338385 |
| Journal.XX.relatedItem.name.namePart.text                                                 | Array                           |         396 |   0.02440832915741338385 |
| Journal.XX.relatedItem.name.namePart1                                                     | Object (393),Array (3)          |         396 |   0.02440832915741338385 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name._attrs                        | Array                           |         396 |   0.02440832915741338385 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name.namePart._attrs               | Array                           |         396 |   0.02440832915741338385 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name.namePart.text                 | Array                           |         396 |   0.02440832915741338385 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name.namePart1                     | Object (393),Array (3)          |         396 |   0.02440832915741338385 |
| Journal.XX.relatedItem.name.namePart1._attrs                                              | Array                           |         393 |   0.02422341757288752270 |
| Journal.XX.relatedItem.name.namePart1.text                                                | Array                           |         393 |   0.02422341757288752270 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name.namePart1._attrs              | Array                           |         393 |   0.02422341757288752270 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name.namePart1.text                | Array                           |         393 |   0.02422341757288752270 |
| nldidlnorm.XX.Item.Component.Resource.mods.name.affiliation6                              | Array                           |         384 |   0.02366868281930994619 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject4.topic1._attrs                         | Array                           |         357 |   0.02200447855857721666 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject4.topic1.text                           | Array                           |         357 |   0.02200447855857721666 |
| nldidlnorm.XX.Item.Component.Resource.mods.name1.nameIdentifier1                          | Object                          |         356 |   0.02194284136373526178 |
| nldidlnorm.XX.Item.Component.Resource.mods.name1.nameIdentifier1._attrs                   | Array                           |         356 |   0.02194284136373526178 |
| nldidlnorm.XX.Item.Component.Resource.mods.name1.nameIdentifier1.text                     | Array                           |         356 |   0.02194284136373526178 |
| nldidlnorm.XX.Item.Component.Resource.mods.name.affiliation7                              | Array                           |         354 |   0.02181956697405135898 |
| nldidlnorm.XX.Item.Component.Resource.mods.name4.namePart2._attrs                         | Array                           |         354 |   0.02181956697405135898 |
| nldidlnorm.XX.Item.Component.Resource.mods.name4.namePart2.text                           | Array                           |         354 |   0.02181956697405135898 |
| nldidlnorm.XX.Item.Component.Resource.mods.name9.affiliation3                             | Array                           |         345 |   0.02126483222047377900 |
| nldidlnorm.XX.Item.Component.Resource.mods.name.affiliation8                              | Array                           |         344 |   0.02120319502563182759 |
| nldidlnorm.XX.Item5.Descriptor3                                                           | Object                          |         339 |   0.02089500905142206363 |
| nldidlnorm.XX.Item5.Descriptor3.Statement                                                 | Object                          |         339 |   0.02089500905142206363 |
| nldidlnorm.XX.Item5.Descriptor3.Statement._attrs                                          | Array                           |         339 |   0.02089500905142206363 |
| nldidlnorm.XX.Item1.Descriptor1.Statement.modified                                        | Array                           |         336 |   0.02071009746689620248 |
| nldidlnorm.XX.Item5.Descriptor3.Statement.modified                                        | Array                           |         336 |   0.02071009746689620248 |
| nldidlnorm.XX.Item.Component.Resource.mods.language2                                      | Object                          |         316 |   0.01947735357005714318 |
| nldidlnorm.XX.Item.Component.Resource.mods.language2.languageTerm                         | Object                          |         316 |   0.01947735357005714318 |
| nldidlnorm.XX.Item.Component.Resource.mods.language2.languageTerm._attrs                  | Array                           |         316 |   0.01947735357005714318 |
| nldidlnorm.XX.Item.Component.Resource.mods.language2.languageTerm.text                    | Array                           |         316 |   0.01947735357005714318 |
| nldidlnorm.XX.Item.Component.Resource.mods.name0.affiliation3                             | Array                           |         306 |   0.01886098162163761527 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.dateOther2                          | Object                          |         301 |   0.01855279564742784784 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.dateOther2._attrs                   | Array                           |         301 |   0.01855279564742784784 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.dateOther2.text                     | Array                           |         301 |   0.01855279564742784784 |
| Journal.XX.relatedItem.name1._attrs                                                       | Array                           |         293 |   0.01805969808869222620 |
| Journal.XX.relatedItem.name1.namePart._attrs                                              | Array                           |         293 |   0.01805969808869222620 |
| Journal.XX.relatedItem.name1.namePart.text                                                | Array                           |         293 |   0.01805969808869222620 |
| Journal.XX.relatedItem.name1.namePart1                                                    | Object (290),Array (3)          |         293 |   0.01805969808869222620 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name1._attrs                       | Array                           |         293 |   0.01805969808869222620 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name1.namePart._attrs              | Array                           |         293 |   0.01805969808869222620 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name1.namePart.text                | Array                           |         293 |   0.01805969808869222620 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name1.namePart1                    | Object (290),Array (3)          |         293 |   0.01805969808869222620 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject4.topic2                                | Array                           |         292 |   0.01799806089385027133 |
| Journal.XX.relatedItem.name1.namePart1._attrs                                             | Array                           |         290 |   0.01787478650416636505 |
| Journal.XX.relatedItem.name1.namePart1.text                                               | Array                           |         290 |   0.01787478650416636505 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name1.namePart1._attrs             | Array                           |         290 |   0.01787478650416636505 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name1.namePart1.text               | Array                           |         290 |   0.01787478650416636505 |
| nldidlnorm.XX.Item.Component.Resource.mods.name5.namePart2._attrs                         | Array                           |         287 |   0.01768987491964050737 |
| nldidlnorm.XX.Item.Component.Resource.mods.name5.namePart2.text                           | Array                           |         287 |   0.01768987491964050737 |
| Journal.XX.relatedItem.relatedItem                                                        | Object                          |         283 |   0.01744332614027269482 |
| Journal.XX.relatedItem.relatedItem._attrs                                                 | Array                           |         283 |   0.01744332614027269482 |
| Journal.XX.relatedItem.relatedItem.titleInfo                                              | Object                          |         283 |   0.01744332614027269482 |
| Journal.XX.relatedItem.relatedItem.titleInfo.title                                        | Array                           |         283 |   0.01744332614027269482 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.relatedItem                        | Object                          |         283 |   0.01744332614027269482 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.relatedItem._attrs                 | Array                           |         283 |   0.01744332614027269482 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.relatedItem.titleInfo              | Object                          |         283 |   0.01744332614027269482 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.relatedItem.titleInfo.title        | Array                           |         283 |   0.01744332614027269482 |
| nldidlnorm.XX.Item.Component.Resource.mods.titleInfo5                                     | Object                          |         283 |   0.01744332614027269482 |
| nldidlnorm.XX.Item.Component.Resource.mods.titleInfo5.title                               | Array                           |         283 |   0.01744332614027269482 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject0.topic._attrs                          | Array                           |         280 |   0.01725841455574683714 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject0.topic.text                            | Array                           |         280 |   0.01725841455574683714 |
| nldidlnorm.XX.Item.Component.Resource.mods.titleInfo5._attrs                              | Array                           |         278 |   0.01713514016606293086 |
| nldidlnorm.XX.Item.Component.Resource.mods.titleInfo6                                     | Object                          |         276 |   0.01701186577637902458 |
| nldidlnorm.XX.Item.Component.Resource.mods.titleInfo6.title                               | Array                           |         276 |   0.01701186577637902458 |
| nldidlnorm.XX.Item.Component.Resource.mods.titleInfo6._attrs                              | Array                           |         272 |   0.01676531699701121203 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject0.temporal                              | Object                          |         271 |   0.01670367980216926063 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject0.temporal._attrs                       | Array                           |         271 |   0.01670367980216926063 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject0.temporal.text                         | Array                           |         271 |   0.01670367980216926063 |
| Journal.XX.relatedItem.name2._attrs                                                       | Array                           |         270 |   0.01664204260732730575 |
| Journal.XX.relatedItem.name2.namePart._attrs                                              | Array                           |         270 |   0.01664204260732730575 |
| Journal.XX.relatedItem.name2.namePart.text                                                | Array                           |         270 |   0.01664204260732730575 |
| Journal.XX.relatedItem.name2.namePart1                                                    | Object                          |         270 |   0.01664204260732730575 |
| Journal.XX.relatedItem.name2.namePart1._attrs                                             | Array                           |         270 |   0.01664204260732730575 |
| Journal.XX.relatedItem.name2.namePart1.text                                               | Array                           |         270 |   0.01664204260732730575 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name2._attrs                       | Array                           |         270 |   0.01664204260732730575 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name2.namePart._attrs              | Array                           |         270 |   0.01664204260732730575 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name2.namePart.text                | Array                           |         270 |   0.01664204260732730575 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name2.namePart1                    | Object                          |         270 |   0.01664204260732730575 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name2.namePart1._attrs             | Array                           |         270 |   0.01664204260732730575 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name2.namePart1.text               | Array                           |         270 |   0.01664204260732730575 |
| nldidlnorm.XX.Item.Component.Resource.mods.classification5                                | Object                          |         257 |   0.01584075907438191669 |
| nldidlnorm.XX.Item.Component.Resource.mods.classification5._attrs                         | Array                           |         257 |   0.01584075907438191669 |
| nldidlnorm.XX.Item.Component.Resource.mods.classification5.text                           | Array                           |         257 |   0.01584075907438191669 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject2.topic0                                | Array                           |         253 |   0.01559421029501410587 |
| nldidlnorm.XX.Item6.Descriptor3                                                           | Object                          |         252 |   0.01553257310017215273 |
| nldidlnorm.XX.Item6.Descriptor3.Statement                                                 | Object                          |         252 |   0.01553257310017215273 |
| nldidlnorm.XX.Item6.Descriptor3.Statement._attrs                                          | Array                           |         252 |   0.01553257310017215273 |
| nldidlnorm.XX.Item6.Descriptor3.Statement.modified                                        | Array                           |         250 |   0.01540929871048824645 |
| nldidlnorm.XX.Item.Component.Resource.mods.name2.affiliation4                             | Array                           |         247 |   0.01522438712596238704 |
| nldidlnorm.XX.Item.Component.Resource.mods.name2.nameIdentifier1                          | Object                          |         234 |   0.01442310359301699971 |
| nldidlnorm.XX.Item.Component.Resource.mods.name2.nameIdentifier1._attrs                   | Array                           |         234 |   0.01442310359301699971 |
| nldidlnorm.XX.Item.Component.Resource.mods.name2.nameIdentifier1.text                     | Array                           |         234 |   0.01442310359301699971 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo1.publisher                          | Array                           |         228 |   0.01405328042396528088 |
| Journal.XX.relatedItem1.titleInfo.subTitle                                                | Array                           |         226 |   0.01393000603428137460 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem1.titleInfo.subTitle                | Array                           |         226 |   0.01393000603428137460 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject5.topic1                                | Object (123),Array (103)        |         226 |   0.01393000603428137460 |
| nldidlnorm.XX.Item.Component.Resource.mods.name6.namePart2._attrs                         | Array                           |         222 |   0.01368345725491356378 |
| nldidlnorm.XX.Item.Component.Resource.mods.name6.namePart2.text                           | Array                           |         222 |   0.01368345725491356378 |
| Journal.XX.relatedItem.name6                                                              | Object                          |         221 |   0.01362182006007161064 |
| Journal.XX.relatedItem.name6.namePart                                                     | Array (220),Object (1)          |         221 |   0.01362182006007161064 |
| Journal.XX.relatedItem.name6.role                                                         | Object                          |         221 |   0.01362182006007161064 |
| Journal.XX.relatedItem.name6.role.roleTerm                                                | Object                          |         221 |   0.01362182006007161064 |
| Journal.XX.relatedItem.name6.role.roleTerm._attrs                                         | Array                           |         221 |   0.01362182006007161064 |
| Journal.XX.relatedItem.name6.role.roleTerm.text                                           | Array                           |         221 |   0.01362182006007161064 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name6                              | Object                          |         221 |   0.01362182006007161064 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name6.namePart                     | Array (220),Object (1)          |         221 |   0.01362182006007161064 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name6.role                         | Object                          |         221 |   0.01362182006007161064 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name6.role.roleTerm                | Object                          |         221 |   0.01362182006007161064 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name6.role.roleTerm._attrs         | Array                           |         221 |   0.01362182006007161064 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name6.role.roleTerm.text           | Array                           |         221 |   0.01362182006007161064 |
| Journal.XX.relatedItem.name3._attrs                                                       | Array                           |         220 |   0.01356018286522965750 |
| Journal.XX.relatedItem.name3.namePart._attrs                                              | Array                           |         220 |   0.01356018286522965750 |
| Journal.XX.relatedItem.name3.namePart.text                                                | Array                           |         220 |   0.01356018286522965750 |
| Journal.XX.relatedItem.name3.namePart1                                                    | Object                          |         220 |   0.01356018286522965750 |
| Journal.XX.relatedItem.name3.namePart1._attrs                                             | Array                           |         220 |   0.01356018286522965750 |
| Journal.XX.relatedItem.name3.namePart1.text                                               | Array                           |         220 |   0.01356018286522965750 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name3._attrs                       | Array                           |         220 |   0.01356018286522965750 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name3.namePart._attrs              | Array                           |         220 |   0.01356018286522965750 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name3.namePart.text                | Array                           |         220 |   0.01356018286522965750 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name3.namePart1                    | Object                          |         220 |   0.01356018286522965750 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name3.namePart1._attrs             | Array                           |         220 |   0.01356018286522965750 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name3.namePart1.text               | Array                           |         220 |   0.01356018286522965750 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.dateModified                        | Object (207),Array (3)          |         210 |   0.01294381091681012785 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject4.topic3                                | Array                           |         209 |   0.01288217372196817471 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.dateModified._attrs                 | Array                           |         207 |   0.01275889933228426844 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.dateModified.text                   | Array                           |         207 |   0.01275889933228426844 |
| nldidlnorm.XX.Item7.Descriptor3                                                           | Object                          |         199 |   0.01226580177354864507 |
| nldidlnorm.XX.Item7.Descriptor3.Statement                                                 | Object                          |         199 |   0.01226580177354864507 |
| nldidlnorm.XX.Item7.Descriptor3.Statement._attrs                                          | Array                           |         199 |   0.01226580177354864507 |
| nldidlnorm.XX.Item7.Descriptor3.Statement.modified                                        | Array                           |         198 |   0.01220416457870669193 |
| nldidlnorm.XX.Item.Component.Resource.mods.name7.namePart2._attrs                         | Array                           |         183 |   0.01127960665607739658 |
| nldidlnorm.XX.Item.Component.Resource.mods.name7.namePart2.text                           | Array                           |         183 |   0.01127960665607739658 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject4.topic4                                | Array                           |         181 |   0.01115633226639349031 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject3.topic0                                | Array                           |         177 |   0.01090978348702567949 |
| nldidlnorm.XX.Item.Component.Resource.mods.name3.affiliation4                             | Array                           |         169 |   0.01041668592829005438 |
| nldidlnorm.XX.Item8.Descriptor3                                                           | Object                          |         168 |   0.01035504873344810124 |
| nldidlnorm.XX.Item8.Descriptor3.Statement                                                 | Object                          |         168 |   0.01035504873344810124 |
| nldidlnorm.XX.Item8.Descriptor3.Statement._attrs                                          | Array                           |         168 |   0.01035504873344810124 |
| nldidlnorm.XX.Item8.Descriptor3.Statement.modified                                        | Array                           |         167 |   0.01029341153860614810 |
| nldidlnorm.XX.Item.Component.Resource.mods.language3                                      | Object                          |         163 |   0.01004686275923833728 |
| nldidlnorm.XX.Item.Component.Resource.mods.language3.languageTerm                         | Object                          |         163 |   0.01004686275923833728 |
| nldidlnorm.XX.Item.Component.Resource.mods.language3.languageTerm._attrs                  | Array                           |         163 |   0.01004686275923833728 |
| nldidlnorm.XX.Item.Component.Resource.mods.language3.languageTerm.text                    | Array                           |         163 |   0.01004686275923833728 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject6.topic1                                | Array (111),Object (46)         |         157 |   0.00967703959018661845 |
| nldidlnorm.XX.Item9.Descriptor3                                                           | Object                          |         144 |   0.00887575605724122939 |
| nldidlnorm.XX.Item9.Descriptor3.Statement                                                 | Object                          |         144 |   0.00887575605724122939 |
| nldidlnorm.XX.Item9.Descriptor3.Statement._attrs                                          | Array                           |         144 |   0.00887575605724122939 |
| nldidlnorm.XX.Item9.Descriptor3.Statement.modified                                        | Array                           |         143 |   0.00881411886239927798 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject4.topic5                                | Array                           |         138 |   0.00850593288818951229 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject5.topic1._attrs                         | Array                           |         123 |   0.00758137496556021695 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject5.topic1.text                           | Array                           |         123 |   0.00758137496556021695 |
| nldidlnorm.XX.Item0.Descriptor3                                                           | Object                          |         122 |   0.00751973777071826468 |
| nldidlnorm.XX.Item0.Descriptor3.Statement                                                 | Object                          |         122 |   0.00751973777071826468 |
| nldidlnorm.XX.Item0.Descriptor3.Statement._attrs                                          | Array                           |         122 |   0.00751973777071826468 |
| nldidlnorm.XX.Item0.Descriptor3.Statement.modified                                        | Array                           |         121 |   0.00745810057587631154 |
| nldidlnorm.XX.Item.Component.Resource.mods.classification6                                | Object                          |         118 |   0.00727318899135045212 |
| nldidlnorm.XX.Item.Component.Resource.mods.classification6._attrs                         | Array                           |         118 |   0.00727318899135045212 |
| nldidlnorm.XX.Item.Component.Resource.mods.classification6.text                           | Array                           |         118 |   0.00727318899135045212 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.dateCreated2                        | Object (113),Array (1)          |         114 |   0.00702664021198264044 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.dateCreated2._attrs                 | Array                           |         113 |   0.00696500301714068730 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.dateCreated2.text                   | Array                           |         113 |   0.00696500301714068730 |
| nldidlnorm.XX.Item.Component.Resource.mods.name4.affiliation4                             | Array                           |         112 |   0.00690336582229873416 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject6.topic2                                | Array                           |         110 |   0.00678009143261482875 |
| Journal.XX.relatedItem.name7                                                              | Object                          |         108 |   0.00665681704293092247 |
| Journal.XX.relatedItem.name7.namePart                                                     | Array (107),Object (1)          |         108 |   0.00665681704293092247 |
| Journal.XX.relatedItem.name7.role                                                         | Object                          |         108 |   0.00665681704293092247 |
| Journal.XX.relatedItem.name7.role.roleTerm                                                | Object                          |         108 |   0.00665681704293092247 |
| Journal.XX.relatedItem.name7.role.roleTerm._attrs                                         | Array                           |         108 |   0.00665681704293092247 |
| Journal.XX.relatedItem.name7.role.roleTerm.text                                           | Array                           |         108 |   0.00665681704293092247 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name7                              | Object                          |         108 |   0.00665681704293092247 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name7.namePart                     | Array (107),Object (1)          |         108 |   0.00665681704293092247 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name7.role                         | Object                          |         108 |   0.00665681704293092247 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name7.role.roleTerm                | Object                          |         108 |   0.00665681704293092247 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name7.role.roleTerm._attrs         | Array                           |         108 |   0.00665681704293092247 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name7.role.roleTerm.text           | Array                           |         108 |   0.00665681704293092247 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject6.topic3                                | Array                           |         108 |   0.00665681704293092247 |
| nldidlnorm.XX.Item.Component.Resource.mods.name1.affiliation5                             | Array                           |         104 |   0.00641026826356311079 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject6.topic4                                | Array                           |         104 |   0.00641026826356311079 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject4.topic6                                | Array                           |         102 |   0.00628699387387920451 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject5.topic2                                | Array                           |         101 |   0.00622535667903725137 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject6.topic5                                | Array                           |         101 |   0.00622535667903725137 |
| nldidlnorm.XX.Item.Component.Resource.mods.name5.affiliation4                             | Array                           |         100 |   0.00616371948419529910 |
| nldidlnorm.XX.Item.Component.Resource.mods.name3.nameIdentifier1                          | Object                          |          95 |   0.00585553350998553341 |
| nldidlnorm.XX.Item.Component.Resource.mods.name3.nameIdentifier1._attrs                   | Array                           |          95 |   0.00585553350998553341 |
| nldidlnorm.XX.Item.Component.Resource.mods.name3.nameIdentifier1.text                     | Array                           |          95 |   0.00585553350998553341 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject5.topic3                                | Array                           |          95 |   0.00585553350998553341 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject6.topic6                                | Array                           |          93 |   0.00573225912030162800 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject7.topic1                                | Array (64),Object (26)          |          90 |   0.00554734753577576858 |
| nldidlnorm.XX.Item.Component.Resource.mods.name6.affiliation4                             | Array                           |          89 |   0.00548571034093381545 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject6.topic7                                | Array                           |          89 |   0.00548571034093381545 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject8.topic1                                | Array (77),Object (12)          |          89 |   0.00548571034093381545 |
| Journal.XX.relatedItem1.originInfo                                                        | Object                          |          85 |   0.00523916156156600376 |
| Journal.XX.relatedItem1.originInfo.dateOther                                              | Object                          |          85 |   0.00523916156156600376 |
| Journal.XX.relatedItem1.originInfo.dateOther._attrs                                       | Array                           |          85 |   0.00523916156156600376 |
| Journal.XX.relatedItem1.originInfo.dateOther.text                                         | Array                           |          85 |   0.00523916156156600376 |
| Journal.XX.relatedItem1.originInfo.dateOther1                                             | Object                          |          85 |   0.00523916156156600376 |
| Journal.XX.relatedItem1.originInfo.dateOther1._attrs                                      | Array                           |          85 |   0.00523916156156600376 |
| Journal.XX.relatedItem1.originInfo.dateOther1.text                                        | Array                           |          85 |   0.00523916156156600376 |
| Journal.XX.relatedItem1.originInfo.place                                                  | Object                          |          85 |   0.00523916156156600376 |
| Journal.XX.relatedItem1.originInfo.place.placeTerm                                        | Object                          |          85 |   0.00523916156156600376 |
| Journal.XX.relatedItem1.originInfo.place.placeTerm._attrs                                 | Array                           |          85 |   0.00523916156156600376 |
| Journal.XX.relatedItem1.originInfo.place.placeTerm.text                                   | Array                           |          85 |   0.00523916156156600376 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem1.originInfo                        | Object                          |          85 |   0.00523916156156600376 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem1.originInfo.dateOther              | Object                          |          85 |   0.00523916156156600376 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem1.originInfo.dateOther._attrs       | Array                           |          85 |   0.00523916156156600376 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem1.originInfo.dateOther.text         | Array                           |          85 |   0.00523916156156600376 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem1.originInfo.dateOther1             | Object                          |          85 |   0.00523916156156600376 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem1.originInfo.dateOther1._attrs      | Array                           |          85 |   0.00523916156156600376 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem1.originInfo.dateOther1.text        | Array                           |          85 |   0.00523916156156600376 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem1.originInfo.place                  | Object                          |          85 |   0.00523916156156600376 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem1.originInfo.place.placeTerm        | Object                          |          85 |   0.00523916156156600376 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem1.originInfo.place.placeTerm._attrs | Array                           |          85 |   0.00523916156156600376 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem1.originInfo.place.placeTerm.text   | Array                           |          85 |   0.00523916156156600376 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject4.topic7                                | Array                           |          85 |   0.00523916156156600376 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject5.topic4                                | Array                           |          85 |   0.00523916156156600376 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.publisher2                          | Array                           |          83 |   0.00511588717188209748 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo1.place                              | Object                          |          83 |   0.00511588717188209748 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo1.place.placeTerm                    | Object                          |          83 |   0.00511588717188209748 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo1.place.placeTerm._attrs             | Array                           |          83 |   0.00511588717188209748 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo1.place.placeTerm.text               | Array                           |          83 |   0.00511588717188209748 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject5.topic5                                | Array                           |          80 |   0.00493097558735623893 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject5.topic6                                | Array                           |          79 |   0.00486933839251428580 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject6.topic8                                | Array                           |          78 |   0.00480770119767233266 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject8.topic2                                | Array                           |          77 |   0.00474606400283038039 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject8.topic3                                | Array                           |          77 |   0.00474606400283038039 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject8.topic4                                | Array                           |          77 |   0.00474606400283038039 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject8.topic5                                | Array                           |          77 |   0.00474606400283038039 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject8.topic6                                | Array                           |          77 |   0.00474606400283038039 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject8.topic7                                | Array                           |          75 |   0.00462278961314647411 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject8.topic8                                | Array                           |          75 |   0.00462278961314647411 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject8.topic9                                | Array                           |          72 |   0.00443787802862061469 |
| nldidlnorm.XX.Item3.Descriptor3.Statement.description                                     | Array                           |          72 |   0.00443787802862061469 |
| nldidlnorm.XX.Item.Component.Resource.mods.name8.namePart2._attrs                         | Array                           |          71 |   0.00437624083377866242 |
| nldidlnorm.XX.Item.Component.Resource.mods.name8.namePart2.text                           | Array                           |          71 |   0.00437624083377866242 |
| nldidlnorm.XX.Item.Component.Resource.mods.name2.affiliation5                             | Array                           |          69 |   0.00425296644409475615 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject4.topic8                                | Array                           |          69 |   0.00425296644409475615 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject5.topic7                                | Array                           |          68 |   0.00419132924925280301 |
| nldidlnorm.XX.Item.Component.Resource.mods.abstract2                                      | Array                           |          66 |   0.00406805485956889673 |
| nldidlnorm.XX.Item.Component.Resource.mods.extension.rights.rights                        | Array                           |          64 |   0.00394478046988499132 |
| nldidlnorm.XX.Item.Component.Resource.mods.name7.affiliation4                             | Array                           |          64 |   0.00394478046988499132 |
| nldidlnorm.XX.Item.Component.Resource.mods.name8.affiliation4                             | Array                           |          64 |   0.00394478046988499132 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.dateIssued2                         | Object                          |          64 |   0.00394478046988499132 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.dateIssued2._attrs                  | Array                           |          64 |   0.00394478046988499132 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.dateIssued2.text                    | Array                           |          64 |   0.00394478046988499132 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject6.topic9                                | Array                           |          64 |   0.00394478046988499132 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject7.topic2                                | Array                           |          63 |   0.00388314327504303818 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject5.topic8                                | Array                           |          61 |   0.00375986888535913234 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject8.topic0                                | Array                           |          61 |   0.00375986888535913234 |
| nldidlnorm.XX.Item.Component.Resource.mods.name9.namePart2._attrs                         | Array                           |          60 |   0.00369823169051717920 |
| nldidlnorm.XX.Item.Component.Resource.mods.name9.namePart2.text                           | Array                           |          60 |   0.00369823169051717920 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject7.topic3                                | Array                           |          60 |   0.00369823169051717920 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject7.topic4                                | Array                           |          60 |   0.00369823169051717920 |
| Journal.XX.relatedItem.name8                                                              | Object                          |          59 |   0.00363659449567522606 |
| Journal.XX.relatedItem.name8.namePart                                                     | Array (58),Object (1)           |          59 |   0.00363659449567522606 |
| Journal.XX.relatedItem.name8.role                                                         | Object                          |          59 |   0.00363659449567522606 |
| Journal.XX.relatedItem.name8.role.roleTerm                                                | Object                          |          59 |   0.00363659449567522606 |
| Journal.XX.relatedItem.name8.role.roleTerm._attrs                                         | Array                           |          59 |   0.00363659449567522606 |
| Journal.XX.relatedItem.name8.role.roleTerm.text                                           | Array                           |          59 |   0.00363659449567522606 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name8                              | Object                          |          59 |   0.00363659449567522606 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name8.namePart                     | Array (58),Object (1)           |          59 |   0.00363659449567522606 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name8.role                         | Object                          |          59 |   0.00363659449567522606 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name8.role.roleTerm                | Object                          |          59 |   0.00363659449567522606 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name8.role.roleTerm._attrs         | Array                           |          59 |   0.00363659449567522606 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name8.role.roleTerm.text           | Array                           |          59 |   0.00363659449567522606 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject7.topic5                                | Array                           |          59 |   0.00363659449567522606 |
| nldidlnorm.XX.Item.Component.Resource.mods.name9.affiliation4                             | Array                           |          57 |   0.00351332010599132022 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject7.topic6                                | Array                           |          57 |   0.00351332010599132022 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject7.topic7                                | Array                           |          56 |   0.00345168291114936708 |
| nldidlnorm.XX.Item.Component.Resource.mods.name3.affiliation5                             | Array                           |          55 |   0.00339004571630741438 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject4.topic9                                | Array                           |          54 |   0.00332840852146546124 |
| nldidlnorm.XX.Item.Component.Resource.mods.name4.nameIdentifier1                          | Object                          |          53 |   0.00326677132662350810 |
| nldidlnorm.XX.Item.Component.Resource.mods.name4.nameIdentifier1._attrs                   | Array                           |          53 |   0.00326677132662350810 |
| nldidlnorm.XX.Item.Component.Resource.mods.name4.nameIdentifier1.text                     | Array                           |          53 |   0.00326677132662350810 |
| Journal.XX.relatedItem.titleInfo1                                                         | Object                          |          52 |   0.00320513413178155539 |
| Journal.XX.relatedItem.titleInfo1.title                                                   | Array                           |          52 |   0.00320513413178155539 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.titleInfo1                         | Object                          |          52 |   0.00320513413178155539 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.titleInfo1.title                   | Array                           |          52 |   0.00320513413178155539 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject7.topic8                                | Array                           |          52 |   0.00320513413178155539 |
| nldidlnorm.XX.Item.Component.Resource.mods.name0.affiliation4                             | Array                           |          51 |   0.00314349693693960226 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject6.topic0                                | Array                           |          51 |   0.00314349693693960226 |
| nldidlnorm.XX.Item.Component.Resource.mods.name4.affiliation5                             | Array                           |          49 |   0.00302022254725569641 |
| Journal.XX.relatedItem2                                                                   | Object                          |          48 |   0.00295858535241374327 |
| Journal.XX.relatedItem2._attrs                                                            | Array                           |          48 |   0.00295858535241374327 |
| Journal.XX.relatedItem2.titleInfo                                                         | Object                          |          48 |   0.00295858535241374327 |
| Journal.XX.relatedItem2.titleInfo.title                                                   | Array                           |          48 |   0.00295858535241374327 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem2                                   | Object                          |          48 |   0.00295858535241374327 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem2._attrs                            | Array                           |          48 |   0.00295858535241374327 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem2.titleInfo                         | Object                          |          48 |   0.00295858535241374327 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem2.titleInfo.title                   | Array                           |          48 |   0.00295858535241374327 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject5.topic9                                | Array                           |          47 |   0.00289694815757179057 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject7.topic9                                | Array                           |          47 |   0.00289694815757179057 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject6.topic1._attrs                         | Array                           |          46 |   0.00283531096272983743 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject6.topic1.text                           | Array                           |          46 |   0.00283531096272983743 |
| nldidlnorm.XX.Item.Component.Resource.mods.name1.affiliation6                             | Array                           |          45 |   0.00277367376788788429 |
| nldidlnorm.XX.Item.Component.Resource.mods.classification7                                | Object                          |          43 |   0.00265039937820397845 |
| nldidlnorm.XX.Item.Component.Resource.mods.classification7._attrs                         | Array                           |          43 |   0.00265039937820397845 |
| nldidlnorm.XX.Item.Component.Resource.mods.classification7.text                           | Array                           |          43 |   0.00265039937820397845 |
| nldidlnorm.XX.Item.Component.Resource.mods.name0.namePart2._attrs                         | Array                           |          41 |   0.00252712498852007261 |
| nldidlnorm.XX.Item.Component.Resource.mods.name0.namePart2.text                           | Array                           |          41 |   0.00252712498852007261 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject7.topic0                                | Array                           |          40 |   0.00246548779367811947 |
| nldidlnorm.XX.Item.Component.Resource.mods.name2.affiliation6                             | Array                           |          39 |   0.00240385059883616633 |
| nldidlnorm.XX.Item.Component.Resource.mods.name5.affiliation5                             | Array                           |          39 |   0.00240385059883616633 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject4.topic0                                | Array                           |          39 |   0.00240385059883616633 |
| Journal.XX.relatedItem.name9                                                              | Object                          |          37 |   0.00228057620915226049 |
| Journal.XX.relatedItem.name9.namePart                                                     | Array                           |          37 |   0.00228057620915226049 |
| Journal.XX.relatedItem.name9.role                                                         | Object                          |          37 |   0.00228057620915226049 |
| Journal.XX.relatedItem.name9.role.roleTerm                                                | Object                          |          37 |   0.00228057620915226049 |
| Journal.XX.relatedItem.name9.role.roleTerm._attrs                                         | Array                           |          37 |   0.00228057620915226049 |
| Journal.XX.relatedItem.name9.role.roleTerm.text                                           | Array                           |          37 |   0.00228057620915226049 |
| Journal.XX.relatedItem.originInfo.edition                                                 | Array                           |          37 |   0.00228057620915226049 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name9                              | Object                          |          37 |   0.00228057620915226049 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name9.namePart                     | Array                           |          37 |   0.00228057620915226049 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name9.role                         | Object                          |          37 |   0.00228057620915226049 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name9.role.roleTerm                | Object                          |          37 |   0.00228057620915226049 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name9.role.roleTerm._attrs         | Array                           |          37 |   0.00228057620915226049 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name9.role.roleTerm.text           | Array                           |          37 |   0.00228057620915226049 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.originInfo.edition                 | Array                           |          37 |   0.00228057620915226049 |
| nldidlnorm.XX.Item.Component.Resource.mods.name1.affiliation7                             | Array                           |          36 |   0.00221893901431030735 |
| nldidlnorm.XX.Item.Component.Resource.mods.name6.affiliation5                             | Array                           |          36 |   0.00221893901431030735 |
| Journal.XX.relatedItem.relatedItem.part                                                   | Object                          |          34 |   0.00209566462462640150 |
| Journal.XX.relatedItem.relatedItem.part.detail                                            | Object                          |          34 |   0.00209566462462640150 |
| Journal.XX.relatedItem.relatedItem.part.detail._attrs                                     | Array                           |          34 |   0.00209566462462640150 |
| Journal.XX.relatedItem.relatedItem.part.detail.number                                     | Array                           |          34 |   0.00209566462462640150 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.relatedItem.part                   | Object                          |          34 |   0.00209566462462640150 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.relatedItem.part.detail            | Object                          |          34 |   0.00209566462462640150 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.relatedItem.part.detail._attrs     | Array                           |          34 |   0.00209566462462640150 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.relatedItem.part.detail.number     | Array                           |          34 |   0.00209566462462640150 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject5.topic0                                | Array                           |          33 |   0.00203402742978444837 |
| Journal.XX.relatedItem3                                                                   | Object                          |          30 |   0.00184911584525858960 |
| Journal.XX.relatedItem3._attrs                                                            | Array                           |          30 |   0.00184911584525858960 |
| Journal.XX.relatedItem3.titleInfo                                                         | Object                          |          30 |   0.00184911584525858960 |
| Journal.XX.relatedItem3.titleInfo.title                                                   | Array                           |          30 |   0.00184911584525858960 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem3                                   | Object                          |          30 |   0.00184911584525858960 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem3._attrs                            | Array                           |          30 |   0.00184911584525858960 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem3.titleInfo                         | Object                          |          30 |   0.00184911584525858960 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem3.titleInfo.title                   | Array                           |          30 |   0.00184911584525858960 |
| nldidlnorm.XX.Item2.Descriptor1.Statement.modified                                        | Array                           |          29 |   0.00178747865041663668 |
| nldidlnorm.XX.Item.Component.Resource.mods.name2.affiliation7                             | Array                           |          28 |   0.00172584145557468354 |
| nldidlnorm.XX.Item.Component.Resource.mods.name8.affiliation5                             | Array                           |          27 |   0.00166420426073273062 |
| nldidlnorm.XX.Item.Component.Resource.mods.name7.affiliation5                             | Array                           |          26 |   0.00160256706589077770 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.dateCreated3                        | Object                          |          26 |   0.00160256706589077770 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.dateCreated3._attrs                 | Array                           |          26 |   0.00160256706589077770 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.dateCreated3.text                   | Array                           |          26 |   0.00160256706589077770 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject7.topic1._attrs                         | Array                           |          26 |   0.00160256706589077770 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject7.topic1.text                           | Array                           |          26 |   0.00160256706589077770 |
| nldidlnorm.XX.Item.Component.Resource.mods.name4.affiliation6                             | Array                           |          25 |   0.00154092987104882478 |
| nldidlnorm.XX.Item.Component.Resource.mods.name5.nameIdentifier1                          | Object                          |          25 |   0.00154092987104882478 |
| nldidlnorm.XX.Item.Component.Resource.mods.name5.nameIdentifier1._attrs                   | Array                           |          25 |   0.00154092987104882478 |
| nldidlnorm.XX.Item.Component.Resource.mods.name5.nameIdentifier1.text                     | Array                           |          25 |   0.00154092987104882478 |
| nldidlnorm.XX.Item.Component.Resource.mods.name0.affiliation5                             | Array                           |          24 |   0.00147929267620687164 |
| nldidlnorm.XX.Item.Component.Resource.mods.name1.affiliation8                             | Array                           |          24 |   0.00147929267620687164 |
| nldidlnorm.XX.Item.Component.Resource.mods.name9.affiliation5                             | Array                           |          24 |   0.00147929267620687164 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject9.topic1                                | Array (16),Object (8)           |          24 |   0.00147929267620687164 |
| nldidlnorm.XX.Item.Component.Resource.mods.classification8                                | Object                          |          23 |   0.00141765548136491872 |
| nldidlnorm.XX.Item.Component.Resource.mods.classification8._attrs                         | Array                           |          23 |   0.00141765548136491872 |
| nldidlnorm.XX.Item.Component.Resource.mods.classification8.text                           | Array                           |          23 |   0.00141765548136491872 |
| nldidlnorm.XX.Item.Component.Resource.mods.name6.affiliation6                             | Array                           |          23 |   0.00141765548136491872 |
| Journal.XX.relatedItem.name0                                                              | Object                          |          21 |   0.00129438109168101266 |
| Journal.XX.relatedItem.name0.namePart                                                     | Array                           |          21 |   0.00129438109168101266 |
| Journal.XX.relatedItem.name0.role                                                         | Object                          |          21 |   0.00129438109168101266 |
| Journal.XX.relatedItem.name0.role.roleTerm                                                | Object                          |          21 |   0.00129438109168101266 |
| Journal.XX.relatedItem.name0.role.roleTerm._attrs                                         | Array                           |          21 |   0.00129438109168101266 |
| Journal.XX.relatedItem.name0.role.roleTerm.text                                           | Array                           |          21 |   0.00129438109168101266 |
| nldidlnorm.XX.Item.Component.Resource.mods.name.nameIdentifier2                           | Object                          |          21 |   0.00129438109168101266 |
| nldidlnorm.XX.Item.Component.Resource.mods.name.nameIdentifier2._attrs                    | Array                           |          21 |   0.00129438109168101266 |
| nldidlnorm.XX.Item.Component.Resource.mods.name.nameIdentifier2.text                      | Array                           |          21 |   0.00129438109168101266 |
| nldidlnorm.XX.Item.Component.Resource.mods.name2.affiliation8                             | Array                           |          21 |   0.00129438109168101266 |
| nldidlnorm.XX.Item.Component.Resource.mods.name3.affiliation6                             | Array                           |          21 |   0.00129438109168101266 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name0                              | Object                          |          21 |   0.00129438109168101266 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name0.namePart                     | Array                           |          21 |   0.00129438109168101266 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name0.role                         | Object                          |          21 |   0.00129438109168101266 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name0.role.roleTerm                | Object                          |          21 |   0.00129438109168101266 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name0.role.roleTerm._attrs         | Array                           |          21 |   0.00129438109168101266 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name0.role.roleTerm.text           | Array                           |          21 |   0.00129438109168101266 |
| Journal.XX.relatedItem.titleInfo.title2                                                   | Array                           |          20 |   0.00123274389683905973 |
| nldidlnorm.XX.Item.Component.Resource.mods.name5.affiliation6                             | Array                           |          20 |   0.00123274389683905973 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.titleInfo.title2                   | Array                           |          20 |   0.00123274389683905973 |
| nldidlnorm.XX.Item.Component.Resource.mods.classification9                                | Object                          |          19 |   0.00117110670199710681 |
| nldidlnorm.XX.Item.Component.Resource.mods.classification9._attrs                         | Array                           |          19 |   0.00117110670199710681 |
| nldidlnorm.XX.Item.Component.Resource.mods.classification9.text                           | Array                           |          19 |   0.00117110670199710681 |
| nldidlnorm.XX.Item.Component.Resource.mods.name8.affiliation6                             | Array                           |          17 |   0.00104783231231320075 |
| Journal.XX.relatedItem.titleInfo1._attrs                                                  | Array                           |          16 |   0.00098619511747124783 |
| nldidlnorm.XX.Item.Component.Resource.mods.classification0                                | Object                          |          16 |   0.00098619511747124783 |
| nldidlnorm.XX.Item.Component.Resource.mods.classification0._attrs                         | Array                           |          16 |   0.00098619511747124783 |
| nldidlnorm.XX.Item.Component.Resource.mods.classification0.text                           | Array                           |          16 |   0.00098619511747124783 |
| nldidlnorm.XX.Item.Component.Resource.mods.name2.affiliation9                             | Array                           |          16 |   0.00098619511747124783 |
| nldidlnorm.XX.Item.Component.Resource.mods.name6.nameIdentifier1                          | Object                          |          16 |   0.00098619511747124783 |
| nldidlnorm.XX.Item.Component.Resource.mods.name6.nameIdentifier1._attrs                   | Array                           |          16 |   0.00098619511747124783 |
| nldidlnorm.XX.Item.Component.Resource.mods.name6.nameIdentifier1.text                     | Array                           |          16 |   0.00098619511747124783 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.titleInfo1._attrs                  | Array                           |          16 |   0.00098619511747124783 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject9.topic2                                | Array                           |          16 |   0.00098619511747124783 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject9.topic3                                | Array                           |          16 |   0.00098619511747124783 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject9.topic4                                | Array                           |          16 |   0.00098619511747124783 |
| nldidlnorm.XX.Item.Component.Resource.mods.name2.affiliation0                             | Array                           |          15 |   0.00092455792262929480 |
| nldidlnorm.XX.Item.Component.Resource.mods.name3.affiliation7                             | Array                           |          15 |   0.00092455792262929480 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject9.topic5                                | Array                           |          15 |   0.00092455792262929480 |
| nldidlnorm.XX.Item.Component.Resource.mods.name1.nameIdentifier2                          | Object                          |          14 |   0.00086292072778734177 |
| nldidlnorm.XX.Item.Component.Resource.mods.name1.nameIdentifier2._attrs                   | Array                           |          14 |   0.00086292072778734177 |
| nldidlnorm.XX.Item.Component.Resource.mods.name1.nameIdentifier2.text                     | Array                           |          14 |   0.00086292072778734177 |
| nldidlnorm.XX.Item.Component.Resource.mods.name7.affiliation6                             | Array                           |          14 |   0.00086292072778734177 |
| nldidlnorm.XX.Item.Component.Resource.mods.name9.affiliation6                             | Array                           |          14 |   0.00086292072778734177 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.publisher3                          | Array                           |          14 |   0.00086292072778734177 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject9.topic6                                | Array                           |          14 |   0.00086292072778734177 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject9.topic7                                | Array                           |          14 |   0.00086292072778734177 |
| nldidlnorm.XX.Item.Component.Resource.mods.titleInfo7                                     | Object                          |          14 |   0.00086292072778734177 |
| nldidlnorm.XX.Item.Component.Resource.mods.titleInfo7._attrs                              | Array                           |          14 |   0.00086292072778734177 |
| nldidlnorm.XX.Item.Component.Resource.mods.titleInfo7.title                               | Array                           |          14 |   0.00086292072778734177 |
| nldidlnorm.XX.Item.Component.Resource.mods.titleInfo8                                     | Object                          |          13 |   0.00080128353294538885 |
| nldidlnorm.XX.Item.Component.Resource.mods.titleInfo8._attrs                              | Array                           |          13 |   0.00080128353294538885 |
| nldidlnorm.XX.Item.Component.Resource.mods.titleInfo8.title                               | Array                           |          13 |   0.00080128353294538885 |
| nldidlnorm.XX.Item.Component.Resource.mods.name0.affiliation6                             | Array                           |          12 |   0.00073964633810343582 |
| nldidlnorm.XX.Item.Component.Resource.mods.name4.affiliation7                             | Array                           |          12 |   0.00073964633810343582 |
| nldidlnorm.XX.Item.Component.Resource.mods.name5.affiliation7                             | Array                           |          12 |   0.00073964633810343582 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject8.topic1._attrs                         | Array                           |          12 |   0.00073964633810343582 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject8.topic1.text                           | Array                           |          12 |   0.00073964633810343582 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject9.topic8                                | Array                           |          12 |   0.00073964633810343582 |
| nldidlnorm.XX.Item.Component.Resource.mods.abstract3                                      | Array                           |          11 |   0.00067800914326148290 |
| nldidlnorm.XX.Item.Component.Resource.mods.name.affiliation9                              | Array                           |          11 |   0.00067800914326148290 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject9.topic9                                | Array                           |          11 |   0.00067800914326148290 |
| Journal.XX.relatedItem.titleInfo.title3                                                   | Array                           |          10 |   0.00061637194841952987 |
| nldidlnorm.XX.Item.Component.Resource.mods.name6.affiliation7                             | Array                           |          10 |   0.00061637194841952987 |
| nldidlnorm.XX.Item.Component.Resource.mods.name7.affiliation7                             | Array                           |          10 |   0.00061637194841952987 |
| nldidlnorm.XX.Item.Component.Resource.mods.name8.affiliation7                             | Array                           |          10 |   0.00061637194841952987 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.titleInfo.title3                   | Array                           |          10 |   0.00061637194841952987 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject9.topic0                                | Array                           |          10 |   0.00061637194841952987 |
| Journal.XX.relatedItem1.titleInfo1                                                        | Object                          |           9 |   0.00055473475357757684 |
| Journal.XX.relatedItem1.titleInfo1._attrs                                                 | Array                           |           9 |   0.00055473475357757684 |
| Journal.XX.relatedItem1.titleInfo1.title                                                  | Array                           |           9 |   0.00055473475357757684 |
| nldidlnorm.XX.Item.Component.Resource.mods.name2.nameIdentifier2                          | Object                          |           9 |   0.00055473475357757684 |
| nldidlnorm.XX.Item.Component.Resource.mods.name2.nameIdentifier2._attrs                   | Array                           |           9 |   0.00055473475357757684 |
| nldidlnorm.XX.Item.Component.Resource.mods.name2.nameIdentifier2.text                     | Array                           |           9 |   0.00055473475357757684 |
| nldidlnorm.XX.Item.Component.Resource.mods.name3.affiliation8                             | Array                           |           9 |   0.00055473475357757684 |
| nldidlnorm.XX.Item.Component.Resource.mods.name7.nameIdentifier1                          | Object                          |           9 |   0.00055473475357757684 |
| nldidlnorm.XX.Item.Component.Resource.mods.name7.nameIdentifier1._attrs                   | Array                           |           9 |   0.00055473475357757684 |
| nldidlnorm.XX.Item.Component.Resource.mods.name7.nameIdentifier1.text                     | Array                           |           9 |   0.00055473475357757684 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem1.titleInfo1                        | Object                          |           9 |   0.00055473475357757684 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem1.titleInfo1._attrs                 | Array                           |           9 |   0.00055473475357757684 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem1.titleInfo1.title                  | Array                           |           9 |   0.00055473475357757684 |
| nldidlnorm.XX.Item.Component.Resource.mods.name.affiliation0                              | Array                           |           8 |   0.00049309755873562392 |
| nldidlnorm.XX.Item.Component.Resource.mods.name0.affiliation7                             | Array                           |           8 |   0.00049309755873562392 |
| nldidlnorm.XX.Item.Component.Resource.mods.name9.affiliation7                             | Array                           |           8 |   0.00049309755873562392 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.dateCreated4                        | Object                          |           8 |   0.00049309755873562392 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.dateCreated4._attrs                 | Array                           |           8 |   0.00049309755873562392 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.dateCreated4.text                   | Array                           |           8 |   0.00049309755873562392 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject0.topic1                                | Array (3),Object (5)            |           8 |   0.00049309755873562392 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject9.topic1._attrs                         | Array                           |           8 |   0.00049309755873562392 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject9.topic1.text                           | Array                           |           8 |   0.00049309755873562392 |
| nldidlnorm.XX.Item.Component.Resource.mods.titleInfo9                                     | Object                          |           8 |   0.00049309755873562392 |
| nldidlnorm.XX.Item.Component.Resource.mods.titleInfo9._attrs                              | Array                           |           8 |   0.00049309755873562392 |
| nldidlnorm.XX.Item.Component.Resource.mods.titleInfo9.title                               | Array                           |           8 |   0.00049309755873562392 |
| nldidlnorm.XX.Item4.Descriptor3.Statement.description                                     | Array                           |           8 |   0.00049309755873562392 |
| Journal.XX.relatedItem.originInfo.dateIssued1                                             | Object                          |           7 |   0.00043146036389367089 |
| Journal.XX.relatedItem.originInfo.dateIssued1._attrs                                      | Array                           |           7 |   0.00043146036389367089 |
| Journal.XX.relatedItem.originInfo.dateIssued1.text                                        | Array                           |           7 |   0.00043146036389367089 |
| nldidlnorm.XX.Item.Component.Resource.mods.name.role.roleTerm2                            | Object                          |           7 |   0.00043146036389367089 |
| nldidlnorm.XX.Item.Component.Resource.mods.name.role.roleTerm2._attrs                     | Array                           |           7 |   0.00043146036389367089 |
| nldidlnorm.XX.Item.Component.Resource.mods.name.role.roleTerm2.text                       | Array                           |           7 |   0.00043146036389367089 |
| nldidlnorm.XX.Item.Component.Resource.mods.name1.affiliation9                             | Array                           |           7 |   0.00043146036389367089 |
| nldidlnorm.XX.Item.Component.Resource.mods.name4.affiliation8                             | Array                           |           7 |   0.00043146036389367089 |
| nldidlnorm.XX.Item.Component.Resource.mods.name5.affiliation8                             | Array                           |           7 |   0.00043146036389367089 |
| nldidlnorm.XX.Item.Component.Resource.mods.name8.nameIdentifier1                          | Object                          |           7 |   0.00043146036389367089 |
| nldidlnorm.XX.Item.Component.Resource.mods.name8.nameIdentifier1._attrs                   | Array                           |           7 |   0.00043146036389367089 |
| nldidlnorm.XX.Item.Component.Resource.mods.name8.nameIdentifier1.text                     | Array                           |           7 |   0.00043146036389367089 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.dateIssued3                         | Object                          |           7 |   0.00043146036389367089 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.dateIssued3._attrs                  | Array                           |           7 |   0.00043146036389367089 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.dateIssued3.text                    | Array                           |           7 |   0.00043146036389367089 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.originInfo.dateIssued1             | Object                          |           7 |   0.00043146036389367089 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.originInfo.dateIssued1._attrs      | Array                           |           7 |   0.00043146036389367089 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.originInfo.dateIssued1.text        | Array                           |           7 |   0.00043146036389367089 |
| nldidlnorm.XX.Item.Component.Resource.mods.extension.grantAgreementList                   | Object                          |           6 |   0.00036982316905171791 |
| nldidlnorm.XX.Item.Component.Resource.mods.extension.grantAgreementList._attrs            | Array                           |           6 |   0.00036982316905171791 |
| nldidlnorm.XX.Item.Component.Resource.mods.extension.grantAgreementList.grantAgreement    | Array                           |           6 |   0.00036982316905171791 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo1.edition                            | Array                           |           6 |   0.00036982316905171791 |
| nldidlnorm.XX.Item.Component.Resource.mods.abstract4                                      | Array                           |           5 |   0.00030818597420976493 |
| nldidlnorm.XX.Item.Component.Resource.mods.name0.affiliation8                             | Array                           |           5 |   0.00030818597420976493 |
| nldidlnorm.XX.Item.Component.Resource.mods.name1.affiliation0                             | Array                           |           5 |   0.00030818597420976493 |
| nldidlnorm.XX.Item.Component.Resource.mods.name3.nameIdentifier2                          | Object                          |           5 |   0.00030818597420976493 |
| nldidlnorm.XX.Item.Component.Resource.mods.name3.nameIdentifier2._attrs                   | Array                           |           5 |   0.00030818597420976493 |
| nldidlnorm.XX.Item.Component.Resource.mods.name3.nameIdentifier2.text                     | Array                           |           5 |   0.00030818597420976493 |
| nldidlnorm.XX.Item.Component.Resource.mods.name4.affiliation9                             | Array                           |           5 |   0.00030818597420976493 |
| nldidlnorm.XX.Item.Component.Resource.mods.name8.affiliation8                             | Array                           |           5 |   0.00030818597420976493 |
| nldidlnorm.XX.Item.Component.Resource.mods.name9.affiliation8                             | Array                           |           5 |   0.00030818597420976493 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject0.topic1._attrs                         | Array                           |           5 |   0.00030818597420976493 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject0.topic1.text                           | Array                           |           5 |   0.00030818597420976493 |
| nldidlnorm.XX.Item.Component.Resource.mods.name1.role.roleTerm2                           | Object                          |           4 |   0.00024654877936781196 |
| nldidlnorm.XX.Item.Component.Resource.mods.name1.role.roleTerm2._attrs                    | Array                           |           4 |   0.00024654877936781196 |
| nldidlnorm.XX.Item.Component.Resource.mods.name1.role.roleTerm2.text                      | Array                           |           4 |   0.00024654877936781196 |
| nldidlnorm.XX.Item.Component.Resource.mods.name3.affiliation9                             | Array                           |           4 |   0.00024654877936781196 |
| nldidlnorm.XX.Item.Component.Resource.mods.name4.role4                                    | Object                          |           4 |   0.00024654877936781196 |
| nldidlnorm.XX.Item.Component.Resource.mods.name4.role4.roleTerm                           | Object                          |           4 |   0.00024654877936781196 |
| nldidlnorm.XX.Item.Component.Resource.mods.name4.role4.roleTerm._attrs                    | Array                           |           4 |   0.00024654877936781196 |
| nldidlnorm.XX.Item.Component.Resource.mods.name4.role4.roleTerm.text                      | Array                           |           4 |   0.00024654877936781196 |
| nldidlnorm.XX.Item.Component.Resource.mods.name6.affiliation8                             | Array                           |           4 |   0.00024654877936781196 |
| nldidlnorm.XX.Item.Component.Resource.mods.name7.affiliation8                             | Array                           |           4 |   0.00024654877936781196 |
| nldidlnorm.XX.Item.Component.Resource.mods.titleInfo.subTitle1                            | Array                           |           4 |   0.00024654877936781196 |
| Journal.XX.relatedItem.part.detail3                                                       | Object                          |           3 |   0.00018491158452585895 |
| Journal.XX.relatedItem.part.detail3._attrs                                                | Array                           |           3 |   0.00018491158452585895 |
| Journal.XX.relatedItem.part.detail3.number                                                | Array                           |           3 |   0.00018491158452585895 |
| nldidlnorm.XX.Item.Component.Resource.mods.name3.affiliation0                             | Array                           |           3 |   0.00018491158452585895 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.copyrightDate1                      | Object                          |           3 |   0.00018491158452585895 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.copyrightDate1._attrs               | Array                           |           3 |   0.00018491158452585895 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.copyrightDate1.text                 | Array                           |           3 |   0.00018491158452585895 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.publisher4                          | Array                           |           3 |   0.00018491158452585895 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.part.detail3                       | Object                          |           3 |   0.00018491158452585895 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.part.detail3._attrs                | Array                           |           3 |   0.00018491158452585895 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.part.detail3.number                | Array                           |           3 |   0.00018491158452585895 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject0.topic2                                | Array                           |           3 |   0.00018491158452585895 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject0.topic3                                | Array                           |           3 |   0.00018491158452585895 |
| nldidlnorm.XX.Item.Component.Resource.mods.titleInfo0                                     | Object                          |           3 |   0.00018491158452585895 |
| nldidlnorm.XX.Item.Component.Resource.mods.titleInfo0._attrs                              | Array                           |           3 |   0.00018491158452585895 |
| nldidlnorm.XX.Item.Component.Resource.mods.titleInfo0.title                               | Array                           |           3 |   0.00018491158452585895 |
| nldidlnorm.XX.Item3.Descriptor1.Statement.modified                                        | Array                           |           3 |   0.00018491158452585895 |
| nldidlnorm.XX.Item5.Descriptor3.Statement.description                                     | Array                           |           3 |   0.00018491158452585895 |
| Journal.XX.relatedItem.identifier2                                                        | Object                          |           2 |   0.00012327438968390598 |
| Journal.XX.relatedItem.identifier2._attrs                                                 | Array                           |           2 |   0.00012327438968390598 |
| Journal.XX.relatedItem.identifier2.text                                                   | Array                           |           2 |   0.00012327438968390598 |
| Journal.XX.relatedItem.name4._attrs                                                       | Array                           |           2 |   0.00012327438968390598 |
| Journal.XX.relatedItem.name4.namePart._attrs                                              | Array                           |           2 |   0.00012327438968390598 |
| Journal.XX.relatedItem.name4.namePart.text                                                | Array                           |           2 |   0.00012327438968390598 |
| Journal.XX.relatedItem.name4.namePart1                                                    | Object                          |           2 |   0.00012327438968390598 |
| Journal.XX.relatedItem.name4.namePart1._attrs                                             | Array                           |           2 |   0.00012327438968390598 |
| Journal.XX.relatedItem.name4.namePart1.text                                               | Array                           |           2 |   0.00012327438968390598 |
| Journal.XX.relatedItem.name5._attrs                                                       | Array                           |           2 |   0.00012327438968390598 |
| Journal.XX.relatedItem.name5.namePart._attrs                                              | Array                           |           2 |   0.00012327438968390598 |
| Journal.XX.relatedItem.name5.namePart.text                                                | Array                           |           2 |   0.00012327438968390598 |
| Journal.XX.relatedItem.name5.namePart1                                                    | Object                          |           2 |   0.00012327438968390598 |
| Journal.XX.relatedItem.name5.namePart1._attrs                                             | Array                           |           2 |   0.00012327438968390598 |
| Journal.XX.relatedItem.name5.namePart1.text                                               | Array                           |           2 |   0.00012327438968390598 |
| Journal.XX.relatedItem1.identifier1                                                       | Object                          |           2 |   0.00012327438968390598 |
| Journal.XX.relatedItem1.identifier1._attrs                                                | Array                           |           2 |   0.00012327438968390598 |
| Journal.XX.relatedItem1.identifier1.text                                                  | Array                           |           2 |   0.00012327438968390598 |
| nldidlnorm.XX.Item.Component.Resource.mods.language4                                      | Object                          |           2 |   0.00012327438968390598 |
| nldidlnorm.XX.Item.Component.Resource.mods.language4.languageTerm                         | Object                          |           2 |   0.00012327438968390598 |
| nldidlnorm.XX.Item.Component.Resource.mods.language4.languageTerm._attrs                  | Array                           |           2 |   0.00012327438968390598 |
| nldidlnorm.XX.Item.Component.Resource.mods.language4.languageTerm.text                    | Array                           |           2 |   0.00012327438968390598 |
| nldidlnorm.XX.Item.Component.Resource.mods.language5                                      | Object                          |           2 |   0.00012327438968390598 |
| nldidlnorm.XX.Item.Component.Resource.mods.language5.languageTerm                         | Object                          |           2 |   0.00012327438968390598 |
| nldidlnorm.XX.Item.Component.Resource.mods.language5.languageTerm._attrs                  | Array                           |           2 |   0.00012327438968390598 |
| nldidlnorm.XX.Item.Component.Resource.mods.language5.languageTerm.text                    | Array                           |           2 |   0.00012327438968390598 |
| nldidlnorm.XX.Item.Component.Resource.mods.language6                                      | Object                          |           2 |   0.00012327438968390598 |
| nldidlnorm.XX.Item.Component.Resource.mods.language6.languageTerm                         | Object                          |           2 |   0.00012327438968390598 |
| nldidlnorm.XX.Item.Component.Resource.mods.language6.languageTerm._attrs                  | Array                           |           2 |   0.00012327438968390598 |
| nldidlnorm.XX.Item.Component.Resource.mods.language6.languageTerm.text                    | Array                           |           2 |   0.00012327438968390598 |
| nldidlnorm.XX.Item.Component.Resource.mods.name0.affiliation9                             | Array                           |           2 |   0.00012327438968390598 |
| nldidlnorm.XX.Item.Component.Resource.mods.name0.nameIdentifier1                          | Object                          |           2 |   0.00012327438968390598 |
| nldidlnorm.XX.Item.Component.Resource.mods.name0.nameIdentifier1._attrs                   | Array                           |           2 |   0.00012327438968390598 |
| nldidlnorm.XX.Item.Component.Resource.mods.name0.nameIdentifier1.text                     | Array                           |           2 |   0.00012327438968390598 |
| nldidlnorm.XX.Item.Component.Resource.mods.name2.role.roleTerm2                           | Object                          |           2 |   0.00012327438968390598 |
| nldidlnorm.XX.Item.Component.Resource.mods.name2.role.roleTerm2._attrs                    | Array                           |           2 |   0.00012327438968390598 |
| nldidlnorm.XX.Item.Component.Resource.mods.name2.role.roleTerm2.text                      | Array                           |           2 |   0.00012327438968390598 |
| nldidlnorm.XX.Item.Component.Resource.mods.name2.role4                                    | Object                          |           2 |   0.00012327438968390598 |
| nldidlnorm.XX.Item.Component.Resource.mods.name2.role4.roleTerm                           | Object                          |           2 |   0.00012327438968390598 |
| nldidlnorm.XX.Item.Component.Resource.mods.name2.role4.roleTerm._attrs                    | Array                           |           2 |   0.00012327438968390598 |
| nldidlnorm.XX.Item.Component.Resource.mods.name2.role4.roleTerm.text                      | Array                           |           2 |   0.00012327438968390598 |
| nldidlnorm.XX.Item.Component.Resource.mods.name3.role.roleTerm2                           | Object                          |           2 |   0.00012327438968390598 |
| nldidlnorm.XX.Item.Component.Resource.mods.name3.role.roleTerm2._attrs                    | Array                           |           2 |   0.00012327438968390598 |
| nldidlnorm.XX.Item.Component.Resource.mods.name3.role.roleTerm2.text                      | Array                           |           2 |   0.00012327438968390598 |
| nldidlnorm.XX.Item.Component.Resource.mods.name4.affiliation0                             | Array                           |           2 |   0.00012327438968390598 |
| nldidlnorm.XX.Item.Component.Resource.mods.name7.affiliation0                             | Array                           |           2 |   0.00012327438968390598 |
| nldidlnorm.XX.Item.Component.Resource.mods.name7.affiliation9                             | Array                           |           2 |   0.00012327438968390598 |
| nldidlnorm.XX.Item.Component.Resource.mods.name8.affiliation9                             | Array                           |           2 |   0.00012327438968390598 |
| nldidlnorm.XX.Item.Component.Resource.mods.name9.affiliation9                             | Array                           |           2 |   0.00012327438968390598 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.dateCreated5                        | Object                          |           2 |   0.00012327438968390598 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.dateCreated5._attrs                 | Array                           |           2 |   0.00012327438968390598 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.dateCreated5.text                   | Array                           |           2 |   0.00012327438968390598 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.identifier2                        | Object                          |           2 |   0.00012327438968390598 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.identifier2._attrs                 | Array                           |           2 |   0.00012327438968390598 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.identifier2.text                   | Array                           |           2 |   0.00012327438968390598 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name4._attrs                       | Array                           |           2 |   0.00012327438968390598 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name4.namePart._attrs              | Array                           |           2 |   0.00012327438968390598 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name4.namePart.text                | Array                           |           2 |   0.00012327438968390598 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name4.namePart1                    | Object                          |           2 |   0.00012327438968390598 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name4.namePart1._attrs             | Array                           |           2 |   0.00012327438968390598 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name4.namePart1.text               | Array                           |           2 |   0.00012327438968390598 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name5._attrs                       | Array                           |           2 |   0.00012327438968390598 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name5.namePart._attrs              | Array                           |           2 |   0.00012327438968390598 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name5.namePart.text                | Array                           |           2 |   0.00012327438968390598 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name5.namePart1                    | Object                          |           2 |   0.00012327438968390598 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name5.namePart1._attrs             | Array                           |           2 |   0.00012327438968390598 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name5.namePart1.text               | Array                           |           2 |   0.00012327438968390598 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem1.identifier1                       | Object                          |           2 |   0.00012327438968390598 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem1.identifier1._attrs                | Array                           |           2 |   0.00012327438968390598 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem1.identifier1.text                  | Array                           |           2 |   0.00012327438968390598 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject0.topic4                                | Array                           |           2 |   0.00012327438968390598 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject0.topic5                                | Array                           |           2 |   0.00012327438968390598 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject0.topic6                                | Array                           |           2 |   0.00012327438968390598 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject0.topic7                                | Array                           |           2 |   0.00012327438968390598 |
| nldidlnorm.XX.Item4.Descriptor1.Statement.modified                                        | Array                           |           2 |   0.00012327438968390598 |
| nldidlnorm.XX.Item6.Descriptor3.Statement.description                                     | Array                           |           2 |   0.00012327438968390598 |
| Journal.XX.relatedItem.name6._attrs                                                       | Array                           |           1 |   0.00006163719484195299 |
| Journal.XX.relatedItem.name6.namePart._attrs                                              | Array                           |           1 |   0.00006163719484195299 |
| Journal.XX.relatedItem.name6.namePart.text                                                | Array                           |           1 |   0.00006163719484195299 |
| Journal.XX.relatedItem.name6.namePart1                                                    | Object                          |           1 |   0.00006163719484195299 |
| Journal.XX.relatedItem.name6.namePart1._attrs                                             | Array                           |           1 |   0.00006163719484195299 |
| Journal.XX.relatedItem.name6.namePart1.text                                               | Array                           |           1 |   0.00006163719484195299 |
| Journal.XX.relatedItem.name7._attrs                                                       | Array                           |           1 |   0.00006163719484195299 |
| Journal.XX.relatedItem.name7.namePart._attrs                                              | Array                           |           1 |   0.00006163719484195299 |
| Journal.XX.relatedItem.name7.namePart.text                                                | Array                           |           1 |   0.00006163719484195299 |
| Journal.XX.relatedItem.name7.namePart1                                                    | Object                          |           1 |   0.00006163719484195299 |
| Journal.XX.relatedItem.name7.namePart1._attrs                                             | Array                           |           1 |   0.00006163719484195299 |
| Journal.XX.relatedItem.name7.namePart1.text                                               | Array                           |           1 |   0.00006163719484195299 |
| Journal.XX.relatedItem.name8._attrs                                                       | Array                           |           1 |   0.00006163719484195299 |
| Journal.XX.relatedItem.name8.namePart._attrs                                              | Array                           |           1 |   0.00006163719484195299 |
| Journal.XX.relatedItem.name8.namePart.text                                                | Array                           |           1 |   0.00006163719484195299 |
| Journal.XX.relatedItem.name8.namePart1                                                    | Object                          |           1 |   0.00006163719484195299 |
| Journal.XX.relatedItem.name8.namePart1._attrs                                             | Array                           |           1 |   0.00006163719484195299 |
| Journal.XX.relatedItem.name8.namePart1.text                                               | Array                           |           1 |   0.00006163719484195299 |
| Journal.XX.relatedItem.titleInfo2                                                         | Object                          |           1 |   0.00006163719484195299 |
| Journal.XX.relatedItem.titleInfo2.title                                                   | Array                           |           1 |   0.00006163719484195299 |
| Journal.XX.relatedItem1.part.detail                                                       | Object                          |           1 |   0.00006163719484195299 |
| Journal.XX.relatedItem1.part.detail._attrs                                                | Array                           |           1 |   0.00006163719484195299 |
| Journal.XX.relatedItem1.part.detail.number                                                | Array                           |           1 |   0.00006163719484195299 |
| Journal.XX.relatedItem1.part.detail1                                                      | Object                          |           1 |   0.00006163719484195299 |
| Journal.XX.relatedItem1.part.detail1._attrs                                               | Array                           |           1 |   0.00006163719484195299 |
| Journal.XX.relatedItem1.part.detail1.number                                               | Array                           |           1 |   0.00006163719484195299 |
| Journal.XX.relatedItem4                                                                   | Object                          |           1 |   0.00006163719484195299 |
| Journal.XX.relatedItem4._attrs                                                            | Array                           |           1 |   0.00006163719484195299 |
| Journal.XX.relatedItem4.titleInfo                                                         | Object                          |           1 |   0.00006163719484195299 |
| Journal.XX.relatedItem4.titleInfo.title                                                   | Array                           |           1 |   0.00006163719484195299 |
| Journal.XX.relatedItem5                                                                   | Object                          |           1 |   0.00006163719484195299 |
| Journal.XX.relatedItem5._attrs                                                            | Array                           |           1 |   0.00006163719484195299 |
| Journal.XX.relatedItem5.titleInfo                                                         | Object                          |           1 |   0.00006163719484195299 |
| Journal.XX.relatedItem5.titleInfo.title                                                   | Array                           |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.abstract1._attrs                               | Array                           |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.abstract1.text                                 | Array                           |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.abstract5                                      | Array                           |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.extension.daiList                              | Object                          |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.extension.daiList._attrs                       | Array                           |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.extension.daiList.identifier                   | Object                          |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.extension.daiList.identifier._attrs            | Array                           |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.extension.daiList.identifier.text              | Array                           |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.extension.daiList.identifier1                  | Object                          |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.extension.daiList.identifier1._attrs           | Array                           |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.extension.daiList.identifier1.text             | Array                           |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.language7                                      | Object                          |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.language7.languageTerm                         | Object                          |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.language7.languageTerm._attrs                  | Array                           |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.language7.languageTerm.text                    | Array                           |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.name0.affiliation0                             | Array                           |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.name3.role4                                    | Object                          |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.name3.role4.roleTerm                           | Object                          |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.name3.role4.roleTerm._attrs                    | Array                           |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.name3.role4.roleTerm.text                      | Array                           |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.name4.nameIdentifier2                          | Object                          |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.name4.nameIdentifier2._attrs                   | Array                           |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.name4.nameIdentifier2.text                     | Array                           |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.name5.role.roleTerm2                           | Object                          |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.name5.role.roleTerm2._attrs                    | Array                           |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.name5.role.roleTerm2.text                      | Array                           |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.name6.affiliation0                             | Array                           |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.name6.affiliation9                             | Array                           |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.name8.affiliation0                             | Array                           |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.name9.affiliation0                             | Array                           |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.dateCreated6                        | Object                          |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.dateCreated6._attrs                 | Array                           |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.dateCreated6.text                   | Array                           |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.dateCreated7                        | Object                          |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.dateCreated7._attrs                 | Array                           |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.dateCreated7.text                   | Array                           |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.dateCreated8                        | Object                          |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.dateCreated8._attrs                 | Array                           |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.dateCreated8.text                   | Array                           |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.dateCreated9                        | Object                          |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.dateCreated9._attrs                 | Array                           |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.dateCreated9.text                   | Array                           |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.dateValid1                          | Object                          |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.dateValid1._attrs                   | Array                           |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.dateValid1.text                     | Array                           |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.place1                              | Object                          |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.place1.placeTerm                    | Object                          |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.place1.placeTerm._attrs             | Array                           |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.place1.placeTerm.text               | Array                           |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.originInfo.publisher5                          | Array                           |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name6._attrs                       | Array                           |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name6.namePart._attrs              | Array                           |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name6.namePart.text                | Array                           |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name6.namePart1                    | Object                          |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name6.namePart1._attrs             | Array                           |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name6.namePart1.text               | Array                           |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name7._attrs                       | Array                           |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name7.namePart._attrs              | Array                           |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name7.namePart.text                | Array                           |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name7.namePart1                    | Object                          |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name7.namePart1._attrs             | Array                           |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name7.namePart1.text               | Array                           |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name8._attrs                       | Array                           |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name8.namePart._attrs              | Array                           |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name8.namePart.text                | Array                           |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name8.namePart1                    | Object                          |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name8.namePart1._attrs             | Array                           |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.name8.namePart1.text               | Array                           |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.titleInfo2                         | Object                          |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem.titleInfo2.title                   | Array                           |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem1.part.detail                       | Object                          |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem1.part.detail._attrs                | Array                           |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem1.part.detail.number                | Array                           |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem1.part.detail1                      | Object                          |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem1.part.detail1._attrs               | Array                           |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem1.part.detail1.number               | Array                           |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem4                                   | Object                          |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem4._attrs                            | Array                           |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem4.titleInfo                         | Object                          |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem4.titleInfo.title                   | Array                           |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem5                                   | Object                          |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem5._attrs                            | Array                           |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem5.titleInfo                         | Object                          |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem5.titleInfo.title                   | Array                           |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject0.topic0                                | Array                           |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject0.topic8                                | Array                           |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject0.topic9                                | Array                           |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item0.Descriptor1.Statement.modified                                        | Array                           |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item0.Descriptor3.Statement.description                                     | Array                           |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item5.Descriptor1.Statement.modified                                        | Array                           |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item7.Descriptor1.Statement.modified                                        | Array                           |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item7.Descriptor3.Statement.description                                     | Array                           |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item8.Descriptor1.Statement.modified                                        | Array                           |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item8.Descriptor3.Statement.description                                     | Array                           |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item9.Descriptor1.Statement.modified                                        | Array                           |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item9.Descriptor3.Statement.description                                     | Array                           |           1 |   0.00006163719484195299 |
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------+
In [ ]:

```
